# Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
project_dir = '/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/'
data_dir = project_dir + 'data/'

# Main

In [3]:
#stable
#%%writefile "/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/main.py"
import ipywidgets as widgets
from google.colab import drive
drive.mount('/content/drive')
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
import time
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService  # ✅ import the new class

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager  # ✅ import the new class

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
project_dir = '/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/'
data_dir = project_dir + 'data/'
emotion_dir=f"{data_dir}emotion_results.csv"
emotion_csv = pd.read_csv(emotion_dir)
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################
def start_music_recommender():
    print("DEBUG: Starting enhanced multi-page recommender with improved UI/UX")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService(emotion_csv)
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    # Define common styles
    main_title_style = "font-size:24px; font-weight:bold; margin-bottom:15px; text-align:center; color:teal;"
    subtitle_style = "font-size:18px; font-weight:bold; margin-top:15px; margin-bottom:10px; text-align:center; color:teal;"

    # Simplified button styles with proper spacing
    button_layout = widgets.Layout(
        margin="8px",
        width="auto",
        min_width="120px"
    )

    # Define common layout for all containers - consistent width and centered
    container_layout = widgets.Layout(
        display="flex",
        flex_flow="column",
        align_items="center",
        width="100%",
        max_width="600px",  # Slightly wider maximum for better spacing
        margin="0 auto"
    )

    # Form fields layout
    form_layout = widgets.Layout(
        display="flex",
        flex_flow="column",
        align_items="flex-start",
        width="100%",
        max_width="450px",
        margin="0 auto"
    )

    # Apply a clean stylesheet that addresses common issues
    display(HTML("""
    <style>
    /* Base styling for widgets */
    .widget-button, .widget-text, .widget-password,
    .widget-dropdown, .widget-label, .widget-html {
        transition: all 0.2s ease;
    }

    /* Button styling */
    .widget-button {
        border-radius: 4px;
        font-weight: 500;
        box-shadow: 0 1px 3px rgba(0,0,0,0.1);
    }

    .widget-button:hover {
        transform: translateY(-1px);
        box-shadow: 0 2px 5px rgba(0,0,0,0.15);
    }

    /* Input field styling */
    .widget-text input,
    .widget-textarea textarea,
    .widget-password input {
        border-radius: 4px;
        border: 1px solid #ccc;
        padding: 6px 8px;
        width: 100%;
    }

    /* Label styling */
    .widget-label {
        font-weight: 500;
    }

    /* Remove focus outlines */
    .widget-box:focus,
    .jupyter-widgets:focus {
        outline: none !important;
    }

    /* Page transition animation */
    @keyframes fadeIn {
        from { opacity: 0; transform: translateY(5px); }
        to { opacity: 1; transform: translateY(0); }
    }

    .fade-in {
        animation: fadeIn 0.3s ease-in-out;
    }

    /* Keep content visible in both light and dark modes */
    .jupyter-widgets {
        color: #333;
    }

    /* Fix for HTML content */
    .widget-html-content {
        width: 100%;
        text-align: center;
    }

    /* Status message styling */
    .status-message {
        padding: 10px;
        border-radius: 6px;
        margin: 10px 0;
        text-align: center;
        width: 100%;
        max-width: 500px;
    }

    .status-info {
        background-color: rgba(33, 150, 243, 0.1);
        border-left: 4px solid #2196F3;
    }

    .status-success {
        background-color: rgba(76, 175, 80, 0.1);
        border-left: 4px solid #4CAF50;
    }

    .status-warning {
        background-color: rgba(255, 152, 0, 0.1);
        border-left: 4px solid #FF9800;
    }

    .status-error {
        background-color: rgba(244, 67, 54, 0.1);
        border-left: 4px solid #F44336;
    }

    /* Fix for dropdown button */
    .widget-dropdown > select {
        height: 32px;
        border-radius: 4px;
        padding: 0 8px;
    }

    /* Fix dark mode compatibility */
    .dark-mode .widget-label,
    .dark-mode .widget-html-content {
        color: #f0f0f0 !important;
    }

    .dark-mode .widget-text input,
    .dark-mode .widget-password input,
    .dark-mode .widget-dropdown > select {
        background: #242424 !important;
        color: #f0f0f0 !important;
        border-color: #555 !important;
    }
    </style>
    """))

    # Navigation history
    nav_history = []

    def show_status(message, status='info'):
        """Display a styled status message"""
        if not message:
            with output:
                clear_output()
            return

        status_classes = {
            'info': 'status-info',
            'success': 'status-success',
            'error': 'status-error',
            'warning': 'status-warning'
        }

        icons = {
            'info': 'ℹ️',
            'success': '✅',
            'error': '❌',
            'warning': '⚠️'
        }

        with output:
            clear_output()
            display(HTML(f"""
            <div class="status-message {status_classes[status]}">
                <span style="font-weight:bold">{icons[status]} {message}</span>
            </div>
            """))

    def show_loading(message="Loading..."):
        """Display a loading spinner with message"""
        with output:
            clear_output()
            display(HTML(f"""
            <div style="text-align:center; padding:15px;">
                <div style="font-weight:bold; margin-bottom:15px;">{message}</div>
                <div style="display:inline-block; width:30px; height:30px; border:3px solid #f3f3f3;
                     border-top:3px solid teal; border-radius:50%; animation:spin 1s linear infinite;"></div>
                <style>@keyframes spin {{0% {{ transform: rotate(0deg); }} 100% {{ transform: rotate(360deg); }}}}</style>
            </div>
            """))

    def switch_to_page(page, save_history=True):
        """Switch to a different page in the application"""
        if save_history and session['current_page'] != page:
            nav_history.append(session['current_page'])
        session['current_page'] = page
        show_status("")

        # Add fade transition between pages
        display(HTML("<div class='fade-in'>"))

        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

        display(HTML("</div>"))

    def go_back():
        """Navigate to the previous page"""
        if nav_history:
            previous_page = nav_history.pop()
            switch_to_page(previous_page, save_history=False)
        else:
            switch_to_page('start', save_history=False)

    def show_start_page():
        """Display the start/welcome page"""
        clear_output()

        # Create elements
        start_label = widgets.HTML(f"<h1 style='{main_title_style}'>🎵 Music Recommender</h1>")
        welcome_text = widgets.HTML("<p style='text-align:center;'>Welcome to your personal music recommendation system.</p>")

        # Buttons
        signin_button = widgets.Button(description='Sign In', button_style='primary', layout=button_layout)
        signup_button = widgets.Button(description='Sign Up', layout=button_layout)

        # Button handlers
        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        # Button container
        buttons = widgets.HBox(
            [signin_button, signup_button],
            layout=widgets.Layout(justify_content='center', margin='15px 0')
        )

        # Main container with simplified structure
        main_container = widgets.VBox([
            start_label,
            welcome_text,
            buttons,
            output
        ], layout=container_layout)

        display(main_container)

    def show_signin_page():
        """Display the sign in page"""
        clear_output()

        # Create elements
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🔑 Sign In</h1>")

        # Form fields with consistent styling
        form_style = {'description_width': '80px'}

        si_username = widgets.Text(description='Username', style=form_style)
        si_password = widgets.Password(description='Password', style=form_style)

        # Buttons
        si_button = widgets.Button(description='Sign In', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Back', layout=button_layout)

        def sign_in_action(b):
            """Handle sign in button click"""
            show_loading("Signing in...")

            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]

            if user_row.empty:
                show_status("Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: go_back())

        # Form container
        signin_form = widgets.VBox([
            si_username,
            si_password
        ], layout=form_layout)

        # Button container
        buttons = widgets.HBox([
            si_button,
            back_button
        ], layout=widgets.Layout(justify_content='center', margin='15px 0'))

        # Main container with simplified structure
        main_container = widgets.VBox([
            label,
            signin_form,
            buttons,
            output
        ], layout=container_layout)

        display(main_container)

    def show_signup_page():
        """Display the sign up page"""
        clear_output()

        # Create elements
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🔐 Sign Up</h1>")

        # Form fields with consistent styling
        form_style = {'description_width': '100px'}

        su_username = widgets.Text(description='Username', style=form_style)
        su_password = widgets.Password(description='Password', style=form_style)
        su_firstname = widgets.Text(description='First Name', style=form_style)
        su_lastname = widgets.Text(description='Last Name', style=form_style)
        su_age = widgets.IntText(description='Age', style=form_style)
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender', style=form_style)

        selected_artists = []

        # Artist selection fields
        artist_header = widgets.HTML(f"<h2 style='{subtitle_style}'>🎵 Select Favorite Artists</h2>")
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(description='Add', layout=widgets.Layout(width='80px'))

        # Artist list container
        artist_box = widgets.VBox(layout=widgets.Layout(
            width='100%',
            max_width='400px',
            margin='10px auto'
        ))

        def refresh_artist_table():
            """Refresh the list of selected artists"""
            rows = []
            for a in selected_artists:
                label = widgets.Label(a, layout=widgets.Layout(width='auto', margin='0 10px 0 0'))
                remove_btn = widgets.Button(
                    description='Remove',
                    button_style='danger',
                    layout=widgets.Layout(width='80px')
                )
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn], layout=widgets.Layout(justify_content='space-between', width='100%')))
            artist_box.children = rows

        def on_add(b):
            """Handle add artist button click"""
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)

        # Create account button and back button
        su_button = widgets.Button(description='Create Account', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Back', layout=button_layout)

        def sign_up_action(b):
            """Handle create account button click"""
            show_loading("Creating account...")

            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            if len(su_password.value) < 4:
                show_status("Password should be at least 4 characters long.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: go_back())

        # Artist input container
        artist_input_container = widgets.HBox(
            [artist_input, add_button],
            layout=widgets.Layout(width='100%', max_width='400px', margin='0 auto', justify_content='space-between')
        )

        # Form container
        user_form = widgets.VBox([
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender
        ], layout=form_layout)

        # Button container
        buttons = widgets.HBox([
            su_button, back_button
        ], layout=widgets.Layout(justify_content='center', margin='15px 0'))

        # Main container with simplified structure and better spacing
        main_container = widgets.VBox([
            label,
            user_form,
            widgets.HTML("<div style='height:15px'></div>"),  # Spacer
            artist_header,
            artist_input_container,
            artist_box,
            widgets.HTML("<div style='height:10px'></div>"),  # Spacer
            buttons,
            output
        ], layout=container_layout)

        display(main_container)

    def show_edit_favorites_page():
        """Display page for editing favorite artists"""
        clear_output()

        # Create elements
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🎨 Edit Favorite Artists</h1>")

        # Get current favorites
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        # Artist selection fields
        form_style = {'description_width': '100px'}
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(description='Add', layout=widgets.Layout(width='80px'))

        # Artist list container
        artist_box = widgets.VBox(layout=widgets.Layout(
            width='100%',
            max_width='400px',
            margin='10px auto'
        ))

        def refresh_artist_table():
            """Refresh the list of selected artists"""
            rows = []
            for a in selected_artists:
                label = widgets.Label(a, layout=widgets.Layout(width='auto', margin='0 10px 0 0'))
                remove_btn = widgets.Button(
                    description='Remove',
                    button_style='danger',
                    layout=widgets.Layout(width='80px')
                )
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn], layout=widgets.Layout(justify_content='space-between', width='100%')))
            artist_box.children = rows

        def on_add(b):
            """Handle add artist button click"""
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)
        refresh_artist_table()

        # Buttons
        confirm_button = widgets.Button(description='Save Changes', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Cancel', layout=button_layout)

        def save_favorites(b):
            """Handle save changes button click"""
            show_loading("Saving your preferences...")

            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists)
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(session['user_id'], selected_artists)
            show_status("Your favorite artists have been updated successfully!", 'success')
            switch_to_page('session')

        confirm_button.on_click(save_favorites)
        back_button.on_click(lambda b: switch_to_page('session'))

        # Artist input container
        artist_input_container = widgets.HBox(
            [artist_input, add_button],
            layout=widgets.Layout(width='100%', max_width='400px', margin='0 auto', justify_content='space-between')
        )

        # Header
        artist_header = widgets.HTML(f"<p style='{subtitle_style}'>Your Current Favorite Artists:</p>")

        # Button container
        buttons = widgets.HBox(
            [confirm_button, back_button],
            layout=widgets.Layout(justify_content='center', margin='15px 0')
        )

        # Main container with simplified structure
        main_container = widgets.VBox([
            label,
            artist_input_container,
            artist_header,
            artist_box,
            widgets.HTML("<div style='height:10px'></div>"),  # Spacer
            buttons,
            output
        ], layout=container_layout)

        display(main_container)

    def show_session_page():
        """Display the main session page after login"""
        clear_output()

        # Create elements
        profile_label = widgets.HTML(f"<h1 style='{main_title_style}'>🎵 Music Recommender</h1>")
        user_info = widgets.HTML(f"<p style='text-align:center; margin-bottom:15px;'><b>👤 Signed in as:</b> {session['username']}</p>")

        # Mode selector
        mode_selector = widgets.ToggleButtons(
            options=[
                ('Private Mode', 'none'),
                ('Log My Activity', 'log'),
                ('Recommend Me Music', 'recommend')
            ],
            value=session['mode'],
            description='Mode:',
            button_style='info',
            tooltips=[
                'Private Mode: No logging or recommendations',
                'Log Mode: Track your listening activity',
                'Recommend Mode: Get personalized music recommendations'
            ],
            layout=widgets.Layout(width='auto', margin='0 auto')
        )

        # Mode labels
        mode_labels = {
            'none': 'Private Mode',
            'log': 'Log My Activity',
            'recommend': 'Recommend Me Music'
        }

        # Mode description box
        mode_description = widgets.HTML(
            f"""<div id="mode-desc-box" style='margin:15px auto; padding:12px; background:#f8f8f8; border-radius:8px; width:100%; max-width:500px; text-align:center;'>
                <p id='mode-desc' style='margin:0;'>Select a mode to continue.</p>
            </div>"""
        )

        # Action buttons
        play_button = widgets.Button(
            description='Play Random Song',
            icon='play',
            button_style='success',
            layout=button_layout
        )
        sad_button = widgets.Button(
            description='I Feel Sad 😢',
            button_style='info',
            layout=button_layout
        )
        happy_button = widgets.Button(
            description='I Feel Happy 😊',
            button_style='info',
            layout=button_layout
        )

        # Profile buttons
        fav_edit_button = widgets.Button(
            description='Edit Favorites',
            icon='edit',
            layout=button_layout
        )
        exit_button = widgets.Button(
            description='Log Out',
            icon='sign-out',
            button_style='danger',
            layout=button_layout
        )

        def change_mode(change):
            """Handle mode selection change"""
            session['mode'] = change['new']

            # Mode descriptions
            mode_descriptions = {
                'none': "Private Mode: Your activity won't be logged or used for recommendations.",
                'log': "Log Mode: Your listening activity will be tracked to improve recommendations.",
                'recommend': "Recommend Mode: Tell us how you're feeling and get personalized recommendations!"
            }

            # Update description box
            mode_description.value = f"""<div id="mode-desc-box" style='margin:15px auto; padding:12px; background:#f8f8f8; border-radius:8px; width:100%; max-width:500px; text-align:center;'>
                <p id='mode-desc' style='margin:0;'>{mode_descriptions[session['mode']]}</p>
            </div>"""

            show_status(f"Mode changed to: {mode_labels[session['mode']]}", 'info')

        def play_song(b):
            """Handle play random song button click"""
            show_status("")
            show_loading("Finding a song...")

            if session['mode'] == 'none':
                show_status("Privacy mode → not logging song play.", 'info')
                return

            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)

            time.sleep(1)  # Reduced delay for better responsiveness
            switch_to_page('session')
            show_status(f"Now playing: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            """Handle I feel sad button click"""
            show_status("")

            if session['mode'] != 'recommend':
                show_status("Not in recommend mode — please switch to 'Recommend Me Music' mode first.", 'warning')
                return

            show_loading("Finding songs to lift your mood...")
            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("No songs found to recommend. Try adding more favorite artists.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]

            # Create song selection UI
            song_header = widgets.HTML("<h3 style='margin-top:15px; text-align:center;'>🎵 Songs to cheer you up:</h3>")
            song_options = widgets.RadioButtons(
                options=options,
                layout=widgets.Layout(width='auto', margin='10px auto')
            )
            confirm_button = widgets.Button(
                description='Play Selected Song',
                button_style='success',
                layout=widgets.Layout(margin='10px auto', width='auto', min_width='150px')
            )

            def on_confirm_click(_):
                """Handle play selected song button click"""
                if not song_options.value:
                    show_status("Please select a song first", 'warning')
                    return

                show_loading("Playing your selection...")
                selected_label = song_options.value
                selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(
                    session['user_id'],
                    selected_row['song_id'],
                    mood_before,
                    mood_after,
                    is_recommended=1
                )
                time.sleep(1)  # Reduced delay for better responsiveness
                switch_to_page('session')
                show_status(f"Now playing: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

            confirm_button.on_click(on_confirm_click)

            with music_recommender_output:
                clear_output()
                display(widgets.VBox([
                    song_header,
                    song_options,
                    confirm_button
                ], layout=widgets.Layout(align_items='center', width='100%', max_width='500px', margin='0 auto')))

        def feel_happy(b):
            """Handle I feel happy button click"""
            show_status("")
            show_status("Thanks for sharing! We're glad you're feeling happy today. 😊", 'success')

        def exit_session(b):
            """Handle log out button click"""
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        # Set up event handlers
        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        fav_edit_button.on_click(lambda b: switch_to_page('edit_favorites'))
        exit_button.on_click(exit_session)

        # Initialize mode description
        change_mode({'new': session['mode']})

        # Section titles
        mode_title = widgets.HTML(f"<h2 style='{subtitle_style}'>Select Your Experience Mode:</h2>")
        action_title = widgets.HTML(f"<h2 style='{subtitle_style}'>Actions:</h2>")
        profile_title = widgets.HTML(f"<h2 style='{subtitle_style}'>Profile Settings:</h2>")

        # Button containers with clean layout
        action_buttons = widgets.HBox(
            [play_button, sad_button, happy_button],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )

        profile_buttons = widgets.HBox(
            [fav_edit_button, exit_button],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )

        # Create sections with minimal nesting and clear spacing
        mode_section = widgets.VBox([
            mode_title,
            mode_selector,
            mode_description
        ], layout=widgets.Layout(width='100%', margin='10px 0'))

        action_section = widgets.VBox([
            action_title,
            action_buttons
        ], layout=widgets.Layout(width='100%', margin='10px 0'))

        profile_section = widgets.VBox([
            profile_title,
            profile_buttons
        ], layout=widgets.Layout(width='100%', margin='10px 0'))

        # Main container with simplified structure
        main_container = widgets.VBox([
            profile_label,
            user_info,
            mode_section,
            action_section,
            profile_section,
            output
        ], layout=container_layout)

        display(main_container)

    # Start the application at the start page
    switch_to_page('start')

#if __name__ == "__main__":
#    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
start_music_recommender()

# Session handalling

In [ ]:
def preview_session_layouts():
    """
    Shows a preview of all three layout options with the ability to switch between them.
    This function creates a demo environment to visualize each layout option.
    """
    clear_output()

    # Create a preview container with tabs
    preview_tabs = widgets.Tab()
    preview_output = widgets.Output()

    # Create temporary session data for preview
    demo_session = {
        'user_id': 1,
        'username': 'Demo User',
        'mode': 'recommend',
        'current_page': 'session'
    }

    # Create a mock show_status function for the preview
    def mock_show_status(message, status='info'):
        colors = {'info': '#ff9900', 'success': '#00aa00', 'error': '#cc0000', 'warning': '#ff6600'}
        icons = {'info': 'ℹ️', 'success': '✅', 'error': '❌', 'warning': '⚠️'}
        with preview_output:
            clear_output()
            if message:
                display(HTML(f"""
                <div style='padding:10px; border-radius:8px; background-color:{colors[status]}15;
                     border-left:4px solid {colors[status]}; margin:15px auto; max-width:500px; text-align:center;'>
                    <span style='color:{colors[status]}; font-weight:bold'>{icons[status]} {message}</span>
                </div>
                """))

    # Mock functions for the preview
    def mock_switch_to_page(page):
        with preview_output:
            display(HTML(f"<div style='text-align:center; padding:5px;'>Would navigate to '{page}' page</div>"))

    def mock_show_loading(message="Loading..."):
        with preview_output:
            clear_output()
            display(HTML(f"""
            <div style='text-align:center; padding:15px;'>
                <div style='font-weight:bold; margin-bottom:15px;'>{message}</div>
                <div style='display:inline-block; width:40px; height:40px; border:5px solid #f3f3f3;
                     border-top:5px solid #3498db; border-radius:50%; animation:spin 1s linear infinite;'></div>
                <style>@keyframes spin {{0% {{ transform: rotate(0deg); }} 100% {{ transform: rotate(360deg); }}}}</style>
            </div>
            """))

    # Function to render the grid layout preview
    def render_grid_layout(session=demo_session):
        with preview_output:
            clear_output()

            # Compact header with title and user info
            header = widgets.HTML(
                f"""<div style='display:flex; justify-content:space-between; align-items:center; margin-bottom:15px; width:100%;'>
                    <span style='font-size:20px; font-weight:bold; color:teal;'>🎵 Music Recommender</span>
                    <span style='font-size:14px; background-color:#f0f8ff; padding:4px 10px;
                           border-radius:20px; display:inline-block; border:1px solid #d0e0ff;'>
                        <b>👤</b> {session['username']}
                    </span>
                </div>"""
            )

            # Mode selector with compact toggle buttons
            mode_selector = widgets.ToggleButtons(
                options=[
                    ('Private', 'none'),
                    ('Log', 'log'),
                    ('Recommend', 'recommend')
                ],
                value=session['mode'],
                button_style='info',
                tooltips=[
                    'Private Mode: No logging or recommendations',
                    'Log Mode: Track your listening activity',
                    'Recommend Mode: Get personalized music recommendations'
                ],
                layout=widgets.Layout(width='100%')
            )

            # Left column: Mode controls
            left_column = widgets.VBox([
                widgets.HTML("<p style='margin:5px 0; font-weight:bold; font-size:14px;'>🎚️ Mode:</p>"),
                mode_selector,
                widgets.Button(
                    description='Edit Favorites',
                    icon='edit',
                    button_style='primary',
                    layout=widgets.Layout(width='100%', margin='10px 0 0 0')
                ),
                widgets.Button(
                    description='Log Out',
                    icon='sign-out',
                    button_style='danger',
                    layout=widgets.Layout(width='100%', margin='5px 0 0 0')
                )
            ], layout=widgets.Layout(
                width='48%',
                padding='8px',
                border='1px solid #e0e0e0',
                border_radius='8px'
            ))

            # Right column: Action buttons in vertical layout
            right_column = widgets.VBox([
                widgets.HTML("<p style='margin:5px 0; font-weight:bold; font-size:14px;'>🎧 Actions:</p>"),
                widgets.Button(
                    description='Play Random Song',
                    icon='play',
                    button_style='success',
                    layout=widgets.Layout(width='100%', margin='0 0 5px 0')
                ),
                widgets.Button(
                    description='I Feel Sad 😢',
                    button_style='info',
                    layout=widgets.Layout(width='100%', margin='5px 0')
                ),
                widgets.Button(
                    description='I Feel Happy 😊',
                    button_style='info',
                    layout=widgets.Layout(width='100%', margin='5px 0 0 0')
                )
            ], layout=widgets.Layout(
                width='48%',
                padding='8px',
                border='1px solid #e0e0e0',
                border_radius='8px'
            ))

            # Create a two-column grid layout
            grid_container = widgets.HBox([
                left_column,
                right_column
            ], layout=widgets.Layout(
                width='100%',
                justify_content='space-between'
            ))

            # Get button references for event handlers
            play_button = right_column.children[1]
            sad_button = right_column.children[2]
            happy_button = right_column.children[3]
            fav_edit_button = left_column.children[2]
            exit_button = left_column.children[3]

            # Add mock event handlers for preview
            play_button.on_click(lambda b: mock_show_status("Playing a random song", 'success'))
            sad_button.on_click(lambda b: mock_show_status("Finding songs to cheer you up", 'info'))
            happy_button.on_click(lambda b: mock_show_status("Thanks for sharing! We're glad you're feeling happy today. 😊", 'success'))
            fav_edit_button.on_click(lambda b: mock_show_status("Would navigate to Edit Favorites page", 'info'))
            exit_button.on_click(lambda b: mock_show_status("Would log out and return to start page", 'info'))
            mode_selector.observe(lambda change: mock_show_status(f"Mode changed to: {change.new}", 'info'), names='value')

            # Show the grid layout
            main_container = widgets.VBox([
                header,
                grid_container,
                widgets.HTML("<div style='height: 15px;'></div>"),  # Space for status messages
                widgets.HTML("<div style='text-align:center; margin-top:10px; padding:5px; background-color:#f0f7ff; border-radius:5px;'><b>Grid Layout</b> - Two-column design with all controls visible</div>")
            ], layout=widgets.Layout(
                display="flex",
                flex_flow="column",
                align_items="center",
                width="100%",
                max_width="550px",
                margin="0 auto",
                padding="10px",
                border="1px solid #e0e0e0",
                border_radius="8px"
            ))

            display(main_container)

    # Function to render the tabs layout preview
    def render_tabs_layout(session=demo_session):
        with preview_output:
            clear_output()

            # Compact header with user info in the corner
            header = widgets.HTML(
                f"""<div style='display:flex; justify-content:space-between; align-items:center; margin-bottom:10px; width:100%;'>
                    <span style='font-size:20px; font-weight:bold; color:teal;'>🎵 Music Recommender</span>
                    <span style='font-size:14px; background-color:#f0f8ff; padding:4px 10px;
                           border-radius:20px; display:inline-block; border:1px solid #d0e0ff;'>
                        <b>👤</b> {session['username']}
                    </span>
                </div>"""
            )

            # Mode selector with compact toggle buttons
            mode_selector = widgets.ToggleButtons(
                options=[
                    ('Private', 'none'),
                    ('Log', 'log'),
                    ('Recommend', 'recommend')
                ],
                value=session['mode'],
                button_style='info',
                tooltips=[
                    'Private Mode: No logging or recommendations',
                    'Log Mode: Track your listening activity',
                    'Recommend Mode: Get personalized music recommendations'
                ],
                layout=widgets.Layout(width='100%')
            )

            # Music action tab content
            music_tab = widgets.VBox([
                widgets.Button(
                    description='Play Random Song',
                    icon='play',
                    button_style='success',
                    layout=widgets.Layout(width='100%', margin='5px 0')
                ),
                widgets.Button(
                    description='I Feel Sad 😢',
                    button_style='info',
                    layout=widgets.Layout(width='100%', margin='5px 0')
                ),
                widgets.Button(
                    description='I Feel Happy 😊',
                    button_style='info',
                    layout=widgets.Layout(width='100%', margin='5px 0')
                )
            ], layout=widgets.Layout(padding='10px'))

            # Profile tab content
            profile_tab = widgets.VBox([
                widgets.Button(
                    description='Edit Favorite Artists',
                    icon='edit',
                    button_style='primary',
                    layout=widgets.Layout(width='100%', margin='5px 0')
                ),
                widgets.Button(
                    description='Log Out',
                    icon='sign-out',
                    button_style='danger',
                    layout=widgets.Layout(width='100%', margin='5px 0')
                )
            ], layout=widgets.Layout(padding='10px'))

            # Create tabs widget
            tabs = widgets.Tab()
            tabs.children = [music_tab, profile_tab]
            tabs.set_title(0, '🎧 Music')
            tabs.set_title(1, '👤 Profile')
            tabs.layout.width = '100%'
            tabs.layout.height = 'auto'

            # Mode container
            mode_container = widgets.VBox([
                widgets.HTML("<p style='margin:5px 0; font-weight:bold; font-size:14px;'>🎚️ Experience Mode:</p>"),
                mode_selector
            ], layout=widgets.Layout(
                width='100%',
                padding='8px',
                margin='0 0 10px 0',
                border='1px solid #e0e0e0',
                border_radius='8px'
            ))

            # Get button references
            play_button = music_tab.children[0]
            sad_button = music_tab.children[1]
            happy_button = music_tab.children[2]
            fav_edit_button = profile_tab.children[0]
            exit_button = profile_tab.children[1]

            # Add mock event handlers for preview
            play_button.on_click(lambda b: mock_show_status("Playing a random song", 'success'))
            sad_button.on_click(lambda b: mock_show_status("Finding songs to cheer you up", 'info'))
            happy_button.on_click(lambda b: mock_show_status("Thanks for sharing! We're glad you're feeling happy today. 😊", 'success'))
            fav_edit_button.on_click(lambda b: mock_show_status("Would navigate to Edit Favorites page", 'info'))
            exit_button.on_click(lambda b: mock_show_status("Would log out and return to start page", 'info'))
            mode_selector.observe(lambda change: mock_show_status(f"Mode changed to: {change.new}", 'info'), names='value')

            # Show the tabs layout
            main_container = widgets.VBox([
                header,
                mode_container,
                tabs,
                widgets.HTML("<div style='height: 15px;'></div>"),  # Space for status messages
                widgets.HTML("<div style='text-align:center; margin-top:10px; padding:5px; background-color:#f0f7ff; border-radius:5px;'><b>Tab Layout</b> - Tabs separate music and profile features</div>")
            ], layout=widgets.Layout(
                display="flex",
                flex_flow="column",
                align_items="center",
                width="100%",
                max_width="550px",
                margin="0 auto",
                padding="10px",
                border="1px solid #e0e0e0",
                border_radius="8px"
            ))

            display(main_container)

    # Function to render the dashboard layout preview
    def render_dashboard_layout(session=demo_session):
        with preview_output:
            clear_output()

            # Dashboard header with username and logout
            header = widgets.HBox([
                widgets.HTML(f"<span style='font-size:20px; font-weight:bold; color:teal;'>🎵 Music Recommender</span>"),
                widgets.HBox([
                    widgets.HTML(f"<span style='font-size:14px; padding:4px; margin-right:8px;'><b>👤 {session['username']}</b></span>"),
                    widgets.Button(
                        description='',
                        icon='sign-out',
                        button_style='danger',
                        layout=widgets.Layout(width='32px', height='32px', padding='0px')
                    )
                ], layout=widgets.Layout(justify_content='flex-end', align_items='center'))
            ], layout=widgets.Layout(
                justify_content='space-between',
                align_items='center',
                width='100%',
                margin='0 0 10px 0'
            ))

            # Mode card - horizontal toggle buttons
            mode_card = widgets.VBox([
                widgets.HTML("<p style='margin:3px 0; font-weight:bold; font-size:13px;'>🎚️ Mode</p>"),
                widgets.ToggleButtons(
                    options=[
                        ('Private', 'none'),
                        ('Log', 'log'),
                        ('Recommend', 'recommend')
                    ],
                    value=session['mode'],
                    button_style='info',
                    tooltips=[
                        'Private Mode: No logging or recommendations',
                        'Log Mode: Track your listening activity',
                        'Recommend Mode: Get personalized music recommendations'
                    ],
                    layout=widgets.Layout(width='100%')
                )
            ], layout=widgets.Layout(
                width='100%',
                padding='8px',
                margin='0 0 10px 0',
                border='1px solid #e0e0e0',
                border_radius='8px',
                background_color='#fafafa'
            ))

            # Play music card
            play_card = widgets.VBox([
                widgets.HTML("<p style='margin:3px 0; font-weight:bold; font-size:13px;'>🎧 Play Music</p>"),
                widgets.Button(
                    description='Random Song',
                    icon='play',
                    button_style='success',
                    layout=widgets.Layout(width='100%')
                )
            ], layout=widgets.Layout(
                width='48%',
                padding='8px',
                border='1px solid #e0e0e0',
                border_radius='8px',
                background_color='#fafafa'
            ))

            # Mood card
            mood_card = widgets.VBox([
                widgets.HTML("<p style='margin:3px 0; font-weight:bold; font-size:13px;'>😊 Mood</p>"),
                widgets.HBox([
                    widgets.Button(
                        description='Sad',
                        button_style='info',
                        layout=widgets.Layout(flex='1', height='auto')
                    ),
                    widgets.Button(
                        description='Happy',
                        button_style='info',
                        layout=widgets.Layout(flex='1', height='auto')
                    )
                ], layout=widgets.Layout(width='100%'))
            ], layout=widgets.Layout(
                width='48%',
                padding='8px',
                border='1px solid #e0e0e0',
                border_radius='8px',
                background_color='#fafafa'
            ))

            # User preferences card
            prefs_card = widgets.VBox([
                widgets.HTML("<p style='margin:3px 0; font-weight:bold; font-size:13px;'>👤 Preferences</p>"),
                widgets.Button(
                    description='Edit Favorites',
                    icon='edit',
                    button_style='primary',
                    layout=widgets.Layout(width='100%')
                )
            ], layout=widgets.Layout(
                width='100%',
                padding='8px',
                margin='10px 0 0 0',
                border='1px solid #e0e0e0',
                border_radius='8px',
                background_color='#fafafa'
            ))

            # Create cards grid
            cards_row1 = widgets.HBox([
                play_card,
                mood_card
            ], layout=widgets.Layout(
                width='100%',
                justify_content='space-between'
            ))

            # Get button references
            play_button = play_card.children[1]
            sad_button = mood_card.children[1].children[0]
            happy_button = mood_card.children[1].children[1]
            fav_edit_button = prefs_card.children[1]
            exit_button = header.children[1].children[1]
            mode_selector = mode_card.children[1]

            # Add mock event handlers for preview
            play_button.on_click(lambda b: mock_show_status("Playing a random song", 'success'))
            sad_button.on_click(lambda b: mock_show_status("Finding songs to cheer you up", 'info'))
            happy_button.on_click(lambda b: mock_show_status("Thanks for sharing! We're glad you're feeling happy today. 😊", 'success'))
            fav_edit_button.on_click(lambda b: mock_show_status("Would navigate to Edit Favorites page", 'info'))
            exit_button.on_click(lambda b: mock_show_status("Would log out and return to start page", 'info'))
            mode_selector.observe(lambda change: mock_show_status(f"Mode changed to: {change.new}", 'info'), names='value')

            # Dashboard layout
            dashboard = widgets.VBox([
                header,
                mode_card,
                cards_row1,
                prefs_card,
                widgets.HTML("<div style='height: 15px;'></div>"),  # Space for status messages
                widgets.HTML("<div style='text-align:center; margin-top:10px; padding:5px; background-color:#f0f7ff; border-radius:5px;'><b>Dashboard Layout</b> - Card-based design with grouped features</div>")
            ], layout=widgets.Layout(
                display="flex",
                flex_flow="column",
                align_items="center",
                width="100%",
                max_width="550px",
                margin="0 auto",
                padding="10px",
                border="1px solid #e0e0e0",
                border_radius="8px"
            ))

            display(dashboard)

    # Selector for the layouts
    layout_selector = widgets.RadioButtons(
        options=[
            ('Grid Layout (Two-Column Design)', 'grid'),
            ('Tab Layout (Tabbed Interface)', 'tabs'),
            ('Dashboard Layout (Card-Based Design)', 'dashboard')
        ],
        value='grid',
        layout=widgets.Layout(width='auto', margin='0 auto 15px auto')
    )

    # Function to switch between layouts
    def switch_layout(change):
        if change['new'] == 'grid':
            render_grid_layout()
        elif change['new'] == 'tabs':
            render_tabs_layout()
        elif change['new'] == 'dashboard':
            render_dashboard_layout()

    # Connect the layout switcher
    layout_selector.observe(switch_layout, names='value')

    # Info text about the preview
    preview_info = widgets.HTML(
        """<div style='text-align:center; margin-bottom:15px;'>
            <h2 style='color:teal; margin-bottom:5px;'>Music Recommender - Layout Preview</h2>
            <p>Select a layout option below to preview different interface designs.</p>
            <p style='font-size:12px; color:#666;'>All layouts maintain the same functionality but organize elements differently.</p>
        </div>"""
    )

    # Load the initial preview (grid layout)
    render_grid_layout()

    # Display the preview interface
    main_preview = widgets.VBox([
        preview_info,
        layout_selector,
        preview_output
    ], layout=widgets.Layout(
        width='100%',
        max_width='600px',
        margin='0 auto',
        padding='20px'
    ))

    display(main_preview)

# To use this preview function, simply call:
# preview_session_layouts()

In [ ]:
preview_session_layouts()

In [ ]:
def show_session_page():
    """
    The reimagined Music Recommender interface.
    A single, opinionated design focused on simplicity, beauty, and joy of use.
    """
    clear_output()

    # --- DESIGN CONSTANTS ---
    # Beautiful minimalist color palette
    COLORS = {
        'primary': '#2d7bff',        # Vibrant blue for primary actions
        'secondary': '#30c1b8',      # Teal for secondary elements
        'background': '#ffffff',     # Clean white background
        'surface': '#f7f9fc',        # Subtle light blue-gray for cards
        'text': '#333333',           # Nearly black, but softer for readability
        'text_secondary': '#575f6e'  # Subdued gray for secondary text
    }

    # --- CUSTOM CSS FOR A PREMIUM FEEL ---
    display(HTML("""
    <style>
    /* Refined animation for all elements */
    .widget-button, .widget-toggle-buttons, .widget-dropdown, .widget-radio {
        transition: all 0.2s cubic-bezier(0.25, 0.8, 0.25, 1);
    }

    /* Elegant button hover effect */
    .widget-button:hover {
        transform: translateY(-1px);
        box-shadow: 0 2px 6px rgba(0, 0, 0, 0.1);
    }

    /* Custom toggle button styling */
    .jupyter-widgets.widget-toggle-buttons {
        border-radius: 8px;
        overflow: hidden;
    }

    /* Sophisticated scrollbar */
    ::-webkit-scrollbar {
        width: 6px;
        height: 6px;
    }

    ::-webkit-scrollbar-track {
        background: rgba(0,0,0,0.02);
        border-radius: 10px;
    }

    ::-webkit-scrollbar-thumb {
        background: rgba(0,0,0,0.1);
        border-radius: 10px;
    }

    ::-webkit-scrollbar-thumb:hover {
        background: rgba(0,0,0,0.2);
    }

    /* Status message animation */
    @keyframes statusAppear {
        from { opacity: 0; transform: translateY(5px); }
        to { opacity: 1; transform: translateY(0); }
    }

    .status-message {
        animation: statusAppear 0.3s ease-out;
    }
    </style>
    """))

    # --- HEADER: BOLD, ELEGANT, CONFIDENT ---
    header = widgets.HTML(
        f"""<div style='display:flex; justify-content:space-between; align-items:center;
              padding-bottom:12px; border-bottom:1px solid #eeeeee; margin-bottom:15px; width:100%;'>
            <span style='font-size:20px; font-weight:600; letter-spacing:-0.3px; color:{COLORS['text']};'>
                <span style='color:{COLORS['primary']}'>♫</span> Music Experience
            </span>
            <div style='display:flex; align-items:center;'>
                <span style='font-size:14px; font-weight:500; color:{COLORS['text_secondary']}; margin-right:8px;'>
                    {session['username']}
                </span>
                <div style='width:28px; height:28px; border-radius:50%; background-color:{COLORS['secondary']};
                      display:flex; align-items:center; justify-content:center; color:white; font-weight:bold;'>
                    {session['username'][0].upper()}
                </div>
            </div>
        </div>"""
    )

    # --- MODE SELECTOR: SIMPLIFIED, OBVIOUS CHOICES ---
    mode_selector = widgets.ToggleButtons(
        options=[
            ('Private', 'none'),
            ('Log', 'log'),
            ('Discover', 'recommend')
        ],
        value=session['mode'],
        button_style='',  # Custom styling below
        tooltips=[
            'Browse privately without tracking',
            'Track your listening patterns',
            'Get personalized music recommendations'
        ]
    )

    # Apply custom styling to make the toggle buttons beautiful
    mode_selector_styled = widgets.HBox([
        widgets.HTML(f"<span style='font-weight:500; color:{COLORS['text_secondary']}; font-size:14px; margin-right:10px;'>Mode:</span>"),
        mode_selector
    ], layout=widgets.Layout(align_items='center', margin='0 0 15px 0'))

    # --- MAIN ACTIONS: FOCUSED, OBVIOUS, DELIGHTFUL ---
    # The play button is the hero - large, prominent, inviting
    play_button = widgets.Button(
        description='Play Something',
        icon='music',
        button_style='',  # Custom styling
        style=dict(button_color=COLORS['primary'], text_color='white'),
        layout=widgets.Layout(
            height='50px',
            width='100%',
            margin='0 0 12px 0',
            border='none',
            border_radius='8px'
        )
    )

    # Mood detection - emphasize the sad detection as the key feature
    mood_detection = widgets.VBox([
        widgets.HTML(f"<p style='font-weight:500; color:{COLORS['text_secondary']}; margin:0 0 8px 0; font-size:14px;'>Feeling down?</p>"),
        widgets.Button(
            description='Find Music to Lift My Mood',
            icon='heartbeat',
            button_style='',
            style=dict(button_color=COLORS['secondary'], text_color='white'),
            layout=widgets.Layout(
                height='40px',
                width='100%',
                border='none',
                border_radius='8px',
                margin='0 0 20px 0'
            )
        )
    ], layout=widgets.Layout(width='100%'))

    # --- SECONDARY ACTIONS: ACCESSIBLE BUT NOT DISTRACTING ---
    secondary_actions = widgets.HBox([
        widgets.Button(
            description='Edit Favorites',
            icon='heart-o',
            button_style='',
            style=dict(button_color='transparent', text_color=COLORS['text_secondary']),
            layout=widgets.Layout(border='1px solid #eeeeee', border_radius='6px')
        ),
        widgets.Button(
            description='Sign Out',
            icon='sign-out',
            button_style='',
            style=dict(button_color='transparent', text_color=COLORS['text_secondary']),
            layout=widgets.Layout(border='1px solid #eeeeee', border_radius='6px')
        )
    ], layout=widgets.Layout(justify_content='center', width='100%', margin='5px 0'))

    # --- MODE LABELS ---
    mode_labels = {
        'none': 'Private Mode',
        'log': 'Log Mode',
        'recommend': 'Discover Mode'
    }

    # --- EVENT HANDLERS: MAINTAIN ORIGINAL FUNCTIONALITY ---
    def change_mode(change):
        session['mode'] = change['new']
        show_status(f"Switched to {mode_labels[session['mode']]}", 'info')

    def play_song(b):
        show_status("")
        show_loading("Finding the perfect song for you...")

        if session['mode'] == 'none':
            show_status("Private mode active. Your activity isn't being logged.", 'info')
            return

        songs_df = pd.read_csv(songs_csv)
        song = songs_df.sample(1).iloc[0]
        mood_before = emotion_service.detect()
        mood_after = emotion_service.detect()
        logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)

        time.sleep(1)  # Reduced for better UX
        switch_to_page('session')
        show_status(f"Now playing: {song['artist']} - {song['song']}", 'success')

    def feel_sad(b):
        show_status("")

        if session['mode'] != 'recommend':
            show_status("Please switch to Discover mode first to get personalized recommendations.", 'warning')
            return

        show_loading("Finding music to lift your mood...")
        recommended = recommender.recommend_songs(session['user_id'])

        if recommended.empty:
            show_status("No songs found to recommend. Try adding more favorite artists.", 'error')
            return

        options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]

        # Beautiful song selection UI
        song_header = widgets.HTML(
            f"<p style='font-weight:500; color:{COLORS['text']}; margin:0 0 10px 0;'>These songs might cheer you up:</p>"
        )

        # Elegant radio buttons for selection
        song_options = widgets.RadioButtons(
            options=options,
            layout=widgets.Layout(width='100%', margin='0 0 15px 0')
        )

        # Play button with perfect proportions
        confirm_button = widgets.Button(
            description='Play Selected',
            icon='play',
            button_style='',
            style=dict(button_color=COLORS['primary'], text_color='white'),
            layout=widgets.Layout(width='150px', height='36px', border='none', border_radius='6px')
        )

        def on_confirm_click(_):
            if not song_options.value:
                show_status("Please select a song first", 'warning')
                return

            show_loading("Preparing your selection...")
            selected_label = song_options.value
            selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
            mood_before = 'sad'
            mood_after = emotion_service.detect()
            logger.log(
                session['user_id'],
                selected_row['song_id'],
                mood_before,
                mood_after,
                is_recommended=1
            )
            time.sleep(1)  # Reduced for better UX
            switch_to_page('session')
            show_status(f"Now playing: {selected_row['artist']} - {selected_row['song']}", 'success')

        confirm_button.on_click(on_confirm_click)

        with music_recommender_output:
            clear_output()
            display(widgets.VBox([
                song_header,
                widgets.VBox([song_options],
                    layout=widgets.Layout(
                        padding='15px',
                        border=f'1px solid #eeeeee',
                        border_radius='8px',
                        background_color=COLORS['surface'],
                        width='100%',
                        margin='0 0 15px 0'
                    )
                ),
                widgets.HBox([confirm_button], layout=widgets.Layout(justify_content='center'))
            ], layout=widgets.Layout(align_items='center', width='100%')))

    def feel_happy(b):
        show_status("")
        show_status("Glad you're feeling good today! Enjoy your music experience.", 'success')

    def exit_session(b):
        session['user_id'] = None
        session['username'] = ''
        session['mode'] = 'none'
        show_status("")
        switch_to_page('start')

    # Button references
    sad_button = mood_detection.children[1]
    fav_edit_button = secondary_actions.children[0]
    exit_button = secondary_actions.children[1]

    # Connect event handlers
    mode_selector.observe(change_mode, names='value')
    play_button.on_click(play_song)
    sad_button.on_click(feel_sad)
    fav_edit_button.on_click(lambda b: switch_to_page('edit_favorites'))
    exit_button.on_click(exit_session)

    # Custom status message rendering
    def show_custom_status(message, status='info'):
        colors = {
            'info': '#3498db',
            'success': '#2ecc71',
            'error': '#e74c3c',
            'warning': '#f39c12'
        }
        icons = {
            'info': '●',
            'success': '✓',
            'error': '✕',
            'warning': '!'
        }
        with output:
            clear_output()
            if message:
                display(HTML(f"""
                <div class="status-message" style='
                     padding:10px 15px;
                     border-radius:6px;
                     background-color:{colors[status]}10;
                     color:{colors[status]};
                     display:flex;
                     align-items:center;
                     margin:10px 0;
                     font-size:14px;'>
                    <span style='
                         width:20px;
                         height:20px;
                         border-radius:50%;
                         background-color:{colors[status]};
                         color:white;
                         display:flex;
                         align-items:center;
                         justify-content:center;
                         margin-right:10px;
                         font-size:11px;'>{icons[status]}</span>
                    <span style='flex:1;'>{message}</span>
                </div>
                """))

    # Override the regular status display
    global show_status
    original_show_status = show_status
    show_status = show_custom_status

    # Main container with perfect spacing
    main_container = widgets.VBox([
        header,
        mode_selector_styled,
        play_button,
        mood_detection,
        secondary_actions,
        output
    ], layout=widgets.Layout(
        width="100%",
        max_width="500px",
        margin="0 auto",
        padding="20px",
        background_color=COLORS['background'],
        border_radius="10px"
    ))

    display(main_container)

    # Restore original function when done
    show_status = original_show_status

In [ ]:
show_session_page()

NameError: name 'session' is not defined

In [ ]:
def show_session_page():
    """
    The reimagined Music Recommender interface.
    A single, opinionated design focused on simplicity, beauty, and joy of use.
    """
    clear_output()

    # --- DESIGN CONSTANTS ---
    # Beautiful minimalist color palette
    COLORS = {
        'primary': '#2d7bff',        # Vibrant blue for primary actions
        'secondary': '#30c1b8',      # Teal for secondary elements
        'background': '#ffffff',     # Clean white background
        'surface': '#f7f9fc',        # Subtle light blue-gray for cards
        'text': '#333333',           # Nearly black, but softer for readability
        'text_secondary': '#575f6e'  # Subdued gray for secondary text
    }

    # --- CUSTOM CSS FOR A PREMIUM FEEL ---
    display(HTML("""
    <style>
    /* Refined animation for all elements */
    .widget-button, .widget-toggle-buttons, .widget-dropdown, .widget-radio {
        transition: all 0.2s cubic-bezier(0.25, 0.8, 0.25, 1);
    }

    /* Elegant button hover effect */
    .widget-button:hover {
        transform: translateY(-1px);
        box-shadow: 0 2px 6px rgba(0, 0, 0, 0.1);
    }

    /* Custom toggle button styling */
    .jupyter-widgets.widget-toggle-buttons {
        border-radius: 8px;
        overflow: hidden;
    }

    /* Sophisticated scrollbar */
    ::-webkit-scrollbar {
        width: 6px;
        height: 6px;
    }

    ::-webkit-scrollbar-track {
        background: rgba(0,0,0,0.02);
        border-radius: 10px;
    }

    ::-webkit-scrollbar-thumb {
        background: rgba(0,0,0,0.1);
        border-radius: 10px;
    }

    ::-webkit-scrollbar-thumb:hover {
        background: rgba(0,0,0,0.2);
    }

    /* Status message animation */
    @keyframes statusAppear {
        from { opacity: 0; transform: translateY(5px); }
        to { opacity: 1; transform: translateY(0); }
    }

    .status-message {
        animation: statusAppear 0.3s ease-out;
    }
    </style>
    """))

    # --- HEADER: BOLD, ELEGANT, CONFIDENT ---
    header = widgets.HTML(
        f"""<div style='display:flex; justify-content:space-between; align-items:center;
              padding-bottom:12px; border-bottom:1px solid #eeeeee; margin-bottom:15px; width:100%;'>
            <span style='font-size:20px; font-weight:600; letter-spacing:-0.3px; color:{COLORS['text']};'>
                <span style='color:{COLORS['primary']}'>♫</span> Music Experience
            </span>
            <div style='display:flex; align-items:center;'>
                <span style='font-size:14px; font-weight:500; color:{COLORS['text_secondary']}; margin-right:8px;'>
                    {session['username']}
                </span>
                <div style='width:28px; height:28px; border-radius:50%; background-color:{COLORS['secondary']};
                      display:flex; align-items:center; justify-content:center; color:white; font-weight:bold;'>
                    {session['username'][0].upper()}
                </div>
            </div>
        </div>"""
    )

    # --- MODE SELECTOR: SIMPLIFIED, OBVIOUS CHOICES ---
    mode_selector = widgets.ToggleButtons(
        options=[
            ('Private', 'none'),
            ('Log', 'log'),
            ('Discover', 'recommend')
        ],
        value=session['mode'],
        button_style='',  # Custom styling below
        tooltips=[
            'Browse privately without tracking',
            'Track your listening patterns',
            'Get personalized music recommendations'
        ]
    )

    # Apply custom styling to make the toggle buttons beautiful
    mode_selector_styled = widgets.HBox([
        widgets.HTML(f"<span style='font-weight:500; color:{COLORS['text_secondary']}; font-size:14px; margin-right:10px;'>Mode:</span>"),
        mode_selector
    ], layout=widgets.Layout(align_items='center', margin='0 0 15px 0'))

    # --- MAIN ACTIONS: FOCUSED, OBVIOUS, DELIGHTFUL ---
    # The play button is the hero - large, prominent, inviting
    play_button = widgets.Button(
        description='Play Something',
        icon='music',
        button_style='',  # Custom styling
        style=dict(button_color=COLORS['primary'], text_color='white'),
        layout=widgets.Layout(
            height='50px',
            width='100%',
            margin='0 0 12px 0',
            border='none',
            border_radius='8px'
        )
    )

    # Mood detection - emphasize the sad detection as the key feature
    mood_detection = widgets.VBox([
        widgets.HTML(f"<p style='font-weight:500; color:{COLORS['text_secondary']}; margin:0 0 8px 0; font-size:14px;'>Feeling down?</p>"),
        widgets.Button(
            description='Find Music to Lift My Mood',
            icon='heartbeat',
            button_style='',
            style=dict(button_color=COLORS['secondary'], text_color='white'),
            layout=widgets.Layout(
                height='40px',
                width='100%',
                border='none',
                border_radius='8px',
                margin='0 0 20px 0'
            )
        )
    ], layout=widgets.Layout(width='100%'))

    # --- SECONDARY ACTIONS: ACCESSIBLE BUT NOT DISTRACTING ---
    secondary_actions = widgets.HBox([
        widgets.Button(
            description='Edit Favorites',
            icon='heart-o',
            button_style='',
            style=dict(button_color='transparent', text_color=COLORS['text_secondary']),
            layout=widgets.Layout(border='1px solid #eeeeee', border_radius='6px')
        ),
        widgets.Button(
            description='Sign Out',
            icon='sign-out',
            button_style='',
            style=dict(button_color='transparent', text_color=COLORS['text_secondary']),
            layout=widgets.Layout(border='1px solid #eeeeee', border_radius='6px')
        )
    ], layout=widgets.Layout(justify_content='center', width='100%', margin='5px 0'))

    # --- MODE LABELS ---
    mode_labels = {
        'none': 'Private Mode',
        'log': 'Log Mode',
        'recommend': 'Discover Mode'
    }

    # --- EVENT HANDLERS: MAINTAIN ORIGINAL FUNCTIONALITY ---
    def change_mode(change):
        session['mode'] = change['new']
        show_status(f"Switched to {mode_labels[session['mode']]}", 'info')

    def play_song(b):
        show_status("")
        show_loading("Finding the perfect song for you...")

        if session['mode'] == 'none':
            show_status("Private mode active. Your activity isn't being logged.", 'info')
            return

        songs_df = pd.read_csv(songs_csv)
        song = songs_df.sample(1).iloc[0]
        mood_before = emotion_service.detect()
        mood_after = emotion_service.detect()
        logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)

        time.sleep(1)  # Reduced for better UX
        switch_to_page('session')
        show_status(f"Now playing: {song['artist']} - {song['song']}", 'success')

    def feel_sad(b):
        show_status("")

        if session['mode'] != 'recommend':
            show_status("Please switch to Discover mode first to get personalized recommendations.", 'warning')
            return

        show_loading("Finding music to lift your mood...")
        recommended = recommender.recommend_songs(session['user_id'])

        if recommended.empty:
            show_status("No songs found to recommend. Try adding more favorite artists.", 'error')
            return

        options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]

        # Beautiful song selection UI
        song_header = widgets.HTML(
            f"<p style='font-weight:500; color:{COLORS['text']}; margin:0 0 10px 0;'>These songs might cheer you up:</p>"
        )

        # Elegant radio buttons for selection
        song_options = widgets.RadioButtons(
            options=options,
            layout=widgets.Layout(width='100%', margin='0 0 15px 0')
        )

        # Play button with perfect proportions
        confirm_button = widgets.Button(
            description='Play Selected',
            icon='play',
            button_style='',
            style=dict(button_color=COLORS['primary'], text_color='white'),
            layout=widgets.Layout(width='150px', height='36px', border='none', border_radius='6px')
        )

        def on_confirm_click(_):
            if not song_options.value:
                show_status("Please select a song first", 'warning')
                return

            show_loading("Preparing your selection...")
            selected_label = song_options.value
            selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
            mood_before = 'sad'
            mood_after = emotion_service.detect()
            logger.log(
                session['user_id'],
                selected_row['song_id'],
                mood_before,
                mood_after,
                is_recommended=1
            )
            time.sleep(1)  # Reduced for better UX
            switch_to_page('session')
            show_status(f"Now playing: {selected_row['artist']} - {selected_row['song']}", 'success')

        confirm_button.on_click(on_confirm_click)

        with music_recommender_output:
            clear_output()
            display(widgets.VBox([
                song_header,
                widgets.VBox([song_options],
                    layout=widgets.Layout(
                        padding='15px',
                        border=f'1px solid #eeeeee',
                        border_radius='8px',
                        background_color=COLORS['surface'],
                        width='100%',
                        margin='0 0 15px 0'
                    )
                ),
                widgets.HBox([confirm_button], layout=widgets.Layout(justify_content='center'))
            ], layout=widgets.Layout(align_items='center', width='100%')))

    def feel_happy(b):
        show_status("")
        show_status("Glad you're feeling good today! Enjoy your music experience.", 'success')

    def exit_session(b):
        session['user_id'] = None
        session['username'] = ''
        session['mode'] = 'none'
        show_status("")
        switch_to_page('start')

    # Button references
    sad_button = mood_detection.children[1]
    fav_edit_button = secondary_actions.children[0]
    exit_button = secondary_actions.children[1]

    # Connect event handlers
    mode_selector.observe(change_mode, names='value')
    play_button.on_click(play_song)
    sad_button.on_click(feel_sad)
    fav_edit_button.on_click(lambda b: switch_to_page('edit_favorites'))
    exit_button.on_click(exit_session)

    # Custom status message rendering
    def show_custom_status(message, status='info'):
        colors = {
            'info': '#3498db',
            'success': '#2ecc71',
            'error': '#e74c3c',
            'warning': '#f39c12'
        }
        icons = {
            'info': '●',
            'success': '✓',
            'error': '✕',
            'warning': '!'
        }
        with output:
            clear_output()
            if message:
                display(HTML(f"""
                <div class="status-message" style='
                     padding:10px 15px;
                     border-radius:6px;
                     background-color:{colors[status]}10;
                     color:{colors[status]};
                     display:flex;
                     align-items:center;
                     margin:10px 0;
                     font-size:14px;'>
                    <span style='
                         width:20px;
                         height:20px;
                         border-radius:50%;
                         background-color:{colors[status]};
                         color:white;
                         display:flex;
                         align-items:center;
                         justify-content:center;
                         margin-right:10px;
                         font-size:11px;'>{icons[status]}</span>
                    <span style='flex:1;'>{message}</span>
                </div>
                """))

    # Override the regular status display
    global show_status
    original_show_status = show_status
    show_status = show_custom_status

    # Main container with perfect spacing
    main_container = widgets.VBox([
        header,
        mode_selector_styled,
        play_button,
        mood_detection,
        secondary_actions,
        output
    ], layout=widgets.Layout(
        width="100%",
        max_width="500px",
        margin="0 auto",
        padding="20px",
        background_color=COLORS['background'],
        border_radius="10px"
    ))

    display(main_container)

    # Restore original function when done
    show_status = original_show_status

In [ ]:
show_session_page()

NameError: name 'session' is not defined

# ddd

In [ ]:
#%%writefile "/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/main.py"
import ipywidgets as widgets
from google.colab import drive
drive.mount('/content/drive')
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
import time
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
project_dir = '/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/'
data_dir = project_dir + 'data/'
emotion_dir=f"{data_dir}emotion_results.csv"
emotion_csv = pd.read_csv(emotion_dir)
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################
def start_music_recommender():
    print("DEBUG: Starting enhanced multi-page recommender with modern UI/UX")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService(emotion_csv)
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start',
        'theme': 'light',  # New theme setting for the user
        'recent_plays': []  # Track recent plays
    }

    output = music_recommender_output

    # Define color schemes
    colors = {
        'light': {
            'primary': '#6200EA',  # Deep purple
            'secondary': '#03DAC6',  # Teal
            'accent': '#FF4081',  # Pink
            'success': '#4CAF50',  # Green
            'warning': '#FFC107',  # Amber
            'error': '#F44336',  # Red
            'info': '#2196F3',  # Blue
            'background': '#FFFFFF',
            'card': '#F5F5F5',
            'text': '#212121',
            'border': '#E0E0E0'
        },
        'dark': {
            'primary': '#BB86FC',  # Light purple
            'secondary': '#03DAC6',  # Teal
            'accent': '#CF6679',  # Pink
            'success': '#81C784',  # Green
            'warning': '#FFD54F',  # Amber
            'error': '#E57373',  # Red
            'info': '#64B5F6',  # Blue
            'background': '#121212',
            'card': '#1E1E1E',
            'text': '#E0E0E0',
            'border': '#424242'
        }
    }

    def get_color(name):
        """Get a color from the current theme"""
        return colors[session['theme']][name]

    # Define common styles with improved typography and spacing
    def get_main_title_style():
        return f"font-size:28px; font-weight:700; margin-bottom:20px; text-align:center; color:{get_color('primary')}; font-family:Arial,sans-serif;"

    def get_subtitle_style():
        return f"font-size:20px; font-weight:600; margin-top:20px; margin-bottom:12px; text-align:center; color:{get_color('primary')}; font-family:Arial,sans-serif;"

    # Improved button styles with proper spacing
    button_layout = widgets.Layout(
        margin="10px",
        width="auto",
        min_width="140px"
    )

    # Define common layout for all containers - consistent width and centered
    container_layout = widgets.Layout(
        display="flex",
        flex_flow="column",
        align_items="center",
        width="100%",
        max_width="700px",  # Wider maximum for better spacing
        margin="0 auto"
    )

    # Form fields layout
    form_layout = widgets.Layout(
        display="flex",
        flex_flow="column",
        align_items="flex-start",
        width="100%",
        max_width="500px",
        margin="0 auto"
    )

    # Apply an enhanced stylesheet that creates a modern, clean interface
    display(HTML(f"""
    <style>
    /* Base styling for widgets */
    .widget-button, .widget-text, .widget-password,
    .widget-dropdown, .widget-label, .widget-html,
    .widget-radio, .widget-toggle-buttons {{
        transition: all 0.3s ease;
        font-family: 'Arial', sans-serif;
    }}

    /* Button styling */
    .widget-button {{
        border-radius: 8px;
        font-weight: 600;
        box-shadow: 0 2px 4px rgba(0,0,0,0.12);
        text-transform: uppercase;
        letter-spacing: 0.5px;
        font-size: 14px;
    }}

    .widget-button:hover {{
        transform: translateY(-2px);
        box-shadow: 0 4px 8px rgba(0,0,0,0.15);
        filter: brightness(105%);
    }}

    .widget-button:active {{
        transform: translateY(1px);
        box-shadow: 0 1px 2px rgba(0,0,0,0.1);
    }}

    /* Input field styling */
    .widget-text input,
    .widget-textarea textarea,
    .widget-password input {{
        border-radius: 8px;
        border: 2px solid {get_color('border')};
        padding: 8px 12px;
        width: 100%;
        background-color: {get_color('background')};
        color: {get_color('text')};
        transition: border-color 0.3s;
    }}

    .widget-text input:focus,
    .widget-textarea textarea:focus,
    .widget-password input:focus {{
        border-color: {get_color('primary')};
        box-shadow: 0 0 0 2px {get_color('primary')}40;
    }}

    /* Label styling */
    .widget-label {{
        font-weight: 500;
        color: {get_color('text')};
    }}

    /* Remove focus outlines */
    .widget-box:focus,
    .jupyter-widgets:focus {{
        outline: none !important;
    }}

    /* Page transition animation */
    @keyframes fadeIn {{
        from {{ opacity: 0; transform: translateY(10px); }}
        to {{ opacity: 1; transform: translateY(0); }}
    }}

    .fade-in {{
        animation: fadeIn 0.4s ease-in-out;
    }}

    /* Theme-compatible widgets */
    .jupyter-widgets {{
        color: {get_color('text')};
    }}

    /* Fix for HTML content */
    .widget-html-content {{
        width: 100%;
        text-align: center;
        color: {get_color('text')};
    }}

    /* Status message styling */
    .status-message {{
        padding: 12px 16px;
        border-radius: 10px;
        margin: 12px 0;
        text-align: center;
        width: 100%;
        max-width: 550px;
        font-weight: 500;
        box-shadow: 0 2px 10px rgba(0,0,0,0.08);
    }}

    .status-info {{
        background-color: {get_color('info')}20;
        border-left: 5px solid {get_color('info')};
    }}

    .status-success {{
        background-color: {get_color('success')}20;
        border-left: 5px solid {get_color('success')};
    }}

    .status-warning {{
        background-color: {get_color('warning')}20;
        border-left: 5px solid {get_color('warning')};
    }}

    .status-error {{
        background-color: {get_color('error')}20;
        border-left: 5px solid {get_color('error')};
    }}

    /* Enhanced dropdown button */
    .widget-dropdown > select {{
        height: 36px;
        border-radius: 8px;
        padding: 0 12px;
        background-color: {get_color('background')};
        color: {get_color('text')};
        border: 2px solid {get_color('border')};
    }}

    /* Fix dark mode compatibility */
    .dark-mode .widget-label,
    .dark-mode .widget-html-content {{
        color: {get_color('text')} !important;
    }}

    .dark-mode .widget-text input,
    .dark-mode .widget-password input,
    .dark-mode .widget-dropdown > select {{
        background: {get_color('background')} !important;
        color: {get_color('text')} !important;
        border-color: {get_color('border')} !important;
    }}

    /* Card styling */
    .card {{
        background-color: {get_color('card')};
        border-radius: 12px;
        padding: 16px;
        box-shadow: 0 4px 15px rgba(0,0,0,0.08);
        transition: all 0.3s ease;
        margin: 10px 0;
        width: 100%;
        max-width: 600px;
    }}

    .card:hover {{
        transform: translateY(-2px);
        box-shadow: 0 6px 20px rgba(0,0,0,0.12);
    }}

    /* Toggle button styling */
    .widget-toggle-buttons button {{
        border-radius: 8px !important;
        font-weight: 600 !important;
        transition: all 0.3s ease !important;
    }}

    .widget-toggle-buttons button.mod-active {{
        background-color: {get_color('primary')} !important;
        color: white !important;
    }}

    /* Radio button styling */
    .widget-radio-box {{
        margin: 5px 0 !important;
    }}

    /* Profile badge */
    .profile-badge {{
        background-color: {get_color('primary')}20;
        color: {get_color('primary')};
        padding: 5px 10px;
        border-radius: 30px;
        font-weight: 600;
        display: inline-block;
        margin: 5px 0;
    }}

    /* Song list styling */
    .song-item {{
        padding: 10px 12px;
        border-radius: 8px;
        margin: 5px 0;
        background-color: {get_color('card')};
        transition: all 0.3s ease;
        border-left: 3px solid {get_color('secondary')};
    }}

    .song-item:hover {{
        background-color: {get_color('primary')}10;
        border-left-color: {get_color('primary')};
    }}

    /* Loading animation */
    @keyframes pulse {{
        0% {{ opacity: 0.6; }}
        50% {{ opacity: 1; }}
        100% {{ opacity: 0.6; }}
    }}

    .pulse {{
        animation: pulse 1.5s infinite ease-in-out;
    }}

    /* Toggle switch for theme */
    .theme-switch {{
        position: relative;
        display: inline-block;
        width: 50px;
        height: 24px;
    }}

    .theme-switch input {{
        opacity: 0;
        width: 0;
        height: 0;
    }}

    .slider {{
        position: absolute;
        cursor: pointer;
        top: 0;
        left: 0;
        right: 0;
        bottom: 0;
        background-color: #ccc;
        transition: .4s;
        border-radius: 34px;
    }}

    .slider:before {{
        position: absolute;
        content: "";
        height: 16px;
        width: 16px;
        left: 4px;
        bottom: 4px;
        background-color: white;
        transition: .4s;
        border-radius: 50%;
    }}

    input:checked + .slider {{
        background-color: {get_color('primary')};
    }}

    input:checked + .slider:before {{
        transform: translateX(26px);
    }}

    /* Mood emoji buttons */
    .mood-button {{
        padding: 12px 15px;
        border-radius: 50%;
        width: 60px;
        height: 60px;
        background-color: {get_color('card')};
        display: flex;
        align-items: center;
        justify-content: center;
        font-size: 28px;
        border: none;
        cursor: pointer;
        box-shadow: 0 2px 5px rgba(0,0,0,0.1);
        transition: all 0.3s ease;
        margin: 0 8px;
    }}

    .mood-button:hover {{
        transform: scale(1.1);
        box-shadow: 0 4px 10px rgba(0,0,0,0.15);
    }}

    .stats-container {{
        display: flex;
        flex-wrap: wrap;
        justify-content: center;
        gap: 10px;
        margin: 15px 0;
    }}

    .stat-card {{
        background-color: {get_color('card')};
        border-radius: 10px;
        padding: 12px;
        min-width: 120px;
        text-align: center;
        box-shadow: 0 2px 8px rgba(0,0,0,0.06);
    }}

    .stat-value {{
        font-size: 24px;
        font-weight: bold;
        color: {get_color('primary')};
    }}

    .stat-label {{
        font-size: 14px;
        color: {get_color('text')}90;
    }}
    </style>
    """))

    # Navigation history
    nav_history = []

    def show_status(message, status='info'):
        """Display a styled status message"""
        if not message:
            with output:
                clear_output()
            return

        status_classes = {
            'info': 'status-info',
            'success': 'status-success',
            'error': 'status-error',
            'warning': 'status-warning'
        }

        icons = {
            'info': 'ℹ️',
            'success': '✅',
            'error': '❌',
            'warning': '⚠️'
        }

        with output:
            clear_output()
            display(HTML(f"""
            <div class="status-message {status_classes[status]}">
                <span style="font-weight:bold">{icons[status]} {message}</span>
            </div>
            """))

    def show_loading(message="Loading..."):
        """Display an enhanced loading animation with message"""
        with output:
            clear_output()
            display(HTML(f"""
            <div style="text-align:center; padding:20px;">
                <div style="font-weight:bold; margin-bottom:15px; color:{get_color('primary')};">{message}</div>
                <div style="display:flex; justify-content:center; align-items:center; gap:6px;">
                    <div class="pulse" style="width:10px; height:10px; border-radius:50%; background-color:{get_color('primary')};"></div>
                    <div class="pulse" style="width:10px; height:10px; border-radius:50%; background-color:{get_color('primary')}; animation-delay: 0.2s;"></div>
                    <div class="pulse" style="width:10px; height:10px; border-radius:50%; background-color:{get_color('primary')}; animation-delay: 0.4s;"></div>
                </div>
            </div>
            """))

    def toggle_theme():
        """Toggle between light and dark theme"""
        session['theme'] = 'dark' if session['theme'] == 'light' else 'light'
        # Re-render the current page to apply the new theme
        switch_to_page(session['current_page'], save_history=False)

    def theme_toggle_button():
        """Create a theme toggle button widget"""
        return widgets.HTML(f"""
        <div style="display:flex; justify-content:center; align-items:center; margin:10px 0;">
            <span style="margin-right:8px; color:{get_color('text')};">☀️</span>
            <label class="theme-switch">
                <input type="checkbox" id="theme-toggle" {"checked" if session['theme'] == 'dark' else ""}>
                <span class="slider"></span>
            </label>
            <span style="margin-left:8px; color:{get_color('text')};">🌙</span>
            <script>
                document.getElementById('theme-toggle').addEventListener('change', function() {{
                    IPython.notebook.kernel.execute('toggle_theme()');
                }});
            </script>
        </div>
        """)

    def switch_to_page(page, save_history=True):
        """Switch to a different page in the application"""
        if save_history and session['current_page'] != page:
            nav_history.append(session['current_page'])
        session['current_page'] = page
        show_status("")

        # Add fade transition between pages
        display(HTML("<div class='fade-in'>"))

        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        elif page == 'history':
            show_history_page()
        elif page == 'discover':
            show_discover_page()
        elif page == 'profile':
            show_profile_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

        display(HTML("</div>"))

    def go_back():
        """Navigate to the previous page"""
        if nav_history:
            previous_page = nav_history.pop()
            switch_to_page(previous_page, save_history=False)
        else:
            switch_to_page('start', save_history=False)

    def app_header(title, subtitle=None, show_back=False):
        """Create a consistent header for all pages"""
        back_button = f"""
        <button
            id="back-btn"
            style="position:absolute; left:10px; top:10px; background:none; border:none;
                  color:{get_color('primary')}; font-size:24px; cursor:pointer;"
        >
            ←
        </button>
        <script>
            document.getElementById('back-btn').addEventListener('click', function() {{
                IPython.notebook.kernel.execute('go_back()');
            }});
        </script>
        """ if show_back else ""

        subtitle_html = f"<p style='margin-top:-10px; margin-bottom:20px; text-align:center; color:{get_color('text')}80;'>{subtitle}</p>" if subtitle else ""

        return widgets.HTML(f"""
        <div style="position:relative; width:100%; padding:10px 0;">
            {back_button}
            <h1 style="{get_main_title_style()}">{title}</h1>
            {subtitle_html}
        </div>
        """)

    def show_start_page():
        """Display the enhanced start/welcome page"""
        clear_output()

        # Create elements
        header = app_header("🎵 Harmony Hub", "Your Personal Music Companion")

        # Theme toggle
        theme_button = theme_toggle_button()

        # Welcome content with animation
        welcome_content = widgets.HTML(f"""
        <div class="card" style="text-align:center; padding:25px;">
            <div style="font-size:48px; margin-bottom:15px;">🎧</div>
            <p style="margin:15px 0; color:{get_color('text')};">Discover music that matches your mood and preferences.</p>
            <p style="margin:15px 0; color:{get_color('text')}; font-style:italic;">Sign in to access your personalized recommendations.</p>
        </div>
        """)

        # Buttons
        signin_button = widgets.Button(
            description='Sign In',
            button_style='primary',
            layout=button_layout,
            icon='sign-in'
        )
        signup_button = widgets.Button(
            description='Sign Up',
            layout=button_layout,
            icon='user-plus'
        )

        # Button handlers
        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        # Button container
        buttons = widgets.HBox(
            [signin_button, signup_button],
            layout=widgets.Layout(justify_content='center', margin='20px 0')
        )

        # Footer
        footer = widgets.HTML(f"""
        <p style="text-align:center; margin-top:30px; font-size:12px; color:{get_color('text')}60;">
            © 2025 Harmony Hub • Your Personal Music Recommender
        </p>
        """)

        # Main container with simplified structure
        main_container = widgets.VBox([
            header,
            theme_button,
            welcome_content,
            buttons,
            footer,
            output
        ], layout=container_layout)

        display(main_container)

    def show_signin_page():
        """Display the enhanced sign in page"""
        clear_output()

        # Create elements
        header = app_header("🔑 Sign In", "Welcome back!", show_back=True)

        # Form fields with consistent styling
        form_style = {'description_width': '100px'}

        si_username = widgets.Text(
            description='Username',
            style=form_style,
            placeholder='Enter your username'
        )
        si_password = widgets.Password(
            description='Password',
            style=form_style,
            placeholder='Enter your password'
        )

        # Remember me checkbox
        remember_me = widgets.Checkbox(
            description='Remember me',
            indent=False,
            layout=widgets.Layout(margin='10px 0 15px 100px')
        )

        # Buttons
        si_button = widgets.Button(
            description='Sign In',
            button_style='primary',
            layout=button_layout,
            icon='sign-in'
        )
        back_button = widgets.Button(
            description='Back',
            layout=button_layout,
            icon='arrow-left'
        )

        def sign_in_action(b):
            """Handle sign in button click"""
            show_loading("Signing in...")

            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]

            if user_row.empty:
                show_status("Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: go_back())

        # Form container with card styling
        form_card = widgets.HTML(f"""
        <div class="card">
            <h3 style="margin-top:0; margin-bottom:20px; color:{get_color('primary')}; text-align:center;">Account Login</h3>
        </div>
        """)

        signin_form = widgets.VBox([
            form_card,
            si_username,
            si_password,
            remember_me
        ], layout=form_layout)

        # Button container
        buttons = widgets.HBox([
            si_button,
            back_button
        ], layout=widgets.Layout(justify_content='center', margin='15px 0'))

        # Forgot password link
        forgot_password = widgets.HTML(f"""
        <p style="text-align:center; margin-top:15px;">
            <a href="#" style="color:{get_color('primary')}; text-decoration:none;">
                Forgot your password?
            </a>
        </p>
        """)

        # Main container with simplified structure
        main_container = widgets.VBox([
            header,
            signin_form,
            buttons,
            forgot_password,
            output
        ], layout=container_layout)

        display(main_container)

    def show_signup_page():
        """Display the enhanced sign up page"""
        clear_output()

        # Create elements
        header = app_header("🔐 Create Account", "Join our music community", show_back=True)

        # Form fields with consistent styling
        form_style = {'description_width': '100px'}

        su_username = widgets.Text(
            description='Username',
            style=form_style,
            placeholder='Choose a username'
        )
        su_password = widgets.Password(
            description='Password',
            style=form_style,
            placeholder='Create a password'
        )
        su_firstname = widgets.Text(
            description='First Name',
            style=form_style,
            placeholder='Your first name'
        )
        su_lastname = widgets.Text(
            description='Last Name',
            style=form_style,
            placeholder='Your last name'
        )
        su_age = widgets.IntText(
            description='Age',
            style=form_style,
            placeholder='Your age'
        )
        su_gender = widgets.Dropdown(
            options=['M', 'F', 'Other'],
            description='Gender',
            style=form_style
        )

        selected_artists = []

        # Artist selection fields
        artist_header = widgets.HTML(f"<h2 style='{get_subtitle_style()}'>🎵 Select Favorite Artists</h2>")
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(
            description='Add',
            button_style='info',
            icon='plus',
            layout=widgets.Layout(width='80px')
        )

        # Artist list container
        artist_box = widgets.VBox(layout=widgets.Layout(
            width='100%',
            max_width='450px',
            margin='15px auto',
            border=f'1px solid {get_color("border")}',
            padding='10px',
            border_radius='8px',
            max_height='200px',
            overflow_y='auto'
        ))

        def refresh_artist_table():
            """Refresh the list of selected artists"""
            rows = []
            for a in selected_artists:
                row = widgets.HTML(f"""
                <div class="song-item" style="display:flex; justify-content:space-between; align-items:center;">
                    <span style="color:{get_color('text')};">{a}</span>
                    <button
                        class="remove-artist"
                        data-artist="{a}"
                        style="background:none; border:none; color:{get_color('error')}; cursor:pointer;"
                    >
                        ✕
                    </button>
                </div>
                <script>
                    document.querySelectorAll('.remove-artist').forEach(btn => {{
                        btn.addEventListener('click', function() {{
                            const artist = this.getAttribute('data-artist');
                            IPython.notebook.kernel.execute(`selected_artists.remove('${{artist}}'); refresh_artist_table()`);
                        }});
                    }});
                </script>
                """)
                rows.append(row)

            if not selected_artists:
                rows = [widgets.HTML(
                    f'<div style="text-align:center; padding:10px; color:{get_color("text")}80;">No artists selected yet</div>'
                )]

            artist_box.children = rows

        def on_add(b):
            """Handle add artist button click"""
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)

        # Create account button and back button
        su_button = widgets.Button(
            description='Create Account',
            button_style='primary',
            icon='user-plus',
            layout=button_layout
        )
        back_button = widgets.Button(
            description='Back',
            icon='arrow-left',
            layout=button_layout
        )

        def sign_up_action(b):
            """Handle create account button click"""
            show_loading("Creating your account...")

            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            if len(su_password.value) < 4:
                show_status("Password should be at least 4 characters long.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: go_back())

        # Artist input container
        artist_input_container = widgets.HBox(
            [artist_input, add_button],
            layout=widgets.Layout(width='100%', max_width='450px', margin='0 auto', justify_content='space-between')
        )

        # Form container
        form_card = widgets.HTML(f"""
        <div class="card" style="margin-bottom:20px;">
            <h3 style="margin-top:0; color:{get_color('primary')}; text-align:center;">Personal Information</h3>
        </div>
        """)

        user_form = widgets.VBox([
            form_card,
            su_username,
            su_password,
            su_firstname,
            su_lastname,
            su_age,
            su_gender
        ], layout=form_layout)

        # Button container
        buttons = widgets.HBox([
            su_button,
            back_button
        ], layout=widgets.Layout(justify_content='center', margin='20px 0'))

        # Terms agreement
        terms_agreement = widgets.Checkbox(
            value=False,
            description='I agree to the Terms of Service',
            indent=False,
            layout=widgets.Layout(margin='0 auto', width='auto')
        )

        # Main container with simplified structure and better spacing
        main_container = widgets.VBox([
            header,
            user_form,
            artist_header,
            artist_input_container,
            artist_box,
            terms_agreement,
            buttons,
            output
        ], layout=container_layout)

        refresh_artist_table()
        display(main_container)

    def show_edit_favorites_page():
        """Display page for editing favorite artists"""
        clear_output()

        # Create elements
        header = app_header("🎨 Edit Favorite Artists", "Personalize your music experience", show_back=True)

        # Get current favorites
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        # Artist selection fields
        form_style = {'description_width': '100px'}
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(
            description='Add',
            icon='plus',
            button_style='info',
            layout=widgets.Layout(width='80px')
        )

        # Artist list container
        artist_box = widgets.VBox(layout=widgets.Layout(
            width='100%',
            max_width='450px',
            margin='15px auto',
            border=f'1px solid {get_color("border")}',
            padding='10px',
            border_radius='8px',
            max_height='300px',
            overflow_y='auto'
        ))

        def refresh_artist_table():
            """Refresh the list of selected artists"""
            rows = []
            for a in selected_artists:
                row = widgets.HTML(f"""
                <div class="song-item" style="display:flex; justify-content:space-between; align-items:center;">
                    <span style="color:{get_color('text')};">{a}</span>
                    <button
                        class="remove-artist"
                        data-artist="{a}"
                        style="background:none; border:none; color:{get_color('error')}; cursor:pointer;"
                    >
                        ✕
                    </button>
                </div>
                <script>
                    document.querySelectorAll('.remove-artist').forEach(btn => {{
                        btn.addEventListener('click', function() {{
                            const artist = this.getAttribute('data-artist');
                            IPython.notebook.kernel.execute(`selected_artists.remove('${{artist}}'); refresh_artist_table()`);
                        }});
                    }});
                </script>
                """)
                rows.append(row)

            if not selected_artists:
                rows = [widgets.HTML(
                    f'<div style="text-align:center; padding:10px; color:{get_color("text")}80;">No artists selected yet</div>'
                )]

            artist_box.children = rows

        def on_add(b):
            """Handle add artist button click"""
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)
        refresh_artist_table()

        # Suggestions section
        suggestions_title = widgets.HTML(f"<h3 style='{get_subtitle_style()}'>Suggested Artists For You</h3>")

        # Get recommended artists
        rec_artists_str = user_row.iloc[0].get('recommended_artists', '')
        if pd.isna(rec_artists_str):
            rec_artists_str = ''

        rec_artists = [a.strip() for a in rec_artists_str.split(',') if a.strip()]

        # Create suggestion chips
        suggestions_html = ""
        for artist in rec_artists[:5]:  # Show top 5 suggestions
            if artist and artist not in selected_artists:
                suggestions_html += f"""
                <div class="artist-chip"
                     data-artist="{artist}"
                     style="display:inline-block; margin:5px; padding:8px 15px; background-color:{get_color('primary')}20;
                            color:{get_color('primary')}; border-radius:20px; cursor:pointer; transition:all 0.3s ease;">
                    {artist} <span style="font-size:15px;">+</span>
                </div>
                """

        if not suggestions_html:
            suggestions_html = f"""
            <div style="text-align:center; padding:10px; color:{get_color('text')}80;">
                No suggestions available yet
            </div>
            """

        suggestions_display = widgets.HTML(f"""
        <div style="text-align:center; margin:15px 0;">
            {suggestions_html}
        </div>
        <script>
            document.querySelectorAll('.artist-chip').forEach(chip => {{
                chip.addEventListener('click', function() {{
                    const artist = this.getAttribute('data-artist');
                    if (!selected_artists.includes(artist)) {{
                        IPython.notebook.kernel.execute(`selected_artists.append('${{artist}}'); refresh_artist_table()`);
                    }}
                }});
            }});
        </script>
        """)

        # Buttons
        confirm_button = widgets.Button(
            description='Save Changes',
            button_style='primary',
            icon='check',
            layout=button_layout
        )
        back_button = widgets.Button(
            description='Cancel',
            icon='times',
            layout=button_layout
        )

        def save_favorites(b):
            """Handle save changes button click"""
            show_loading("Saving your preferences...")

            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists)
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(session['user_id'], selected_artists)
            show_status("Your favorite artists have been updated successfully!", 'success')
            switch_to_page('session')

        confirm_button.on_click(save_favorites)
        back_button.on_click(lambda b: switch_to_page('session'))

        # Artist input container
        artist_input_container = widgets.HBox(
            [artist_input, add_button],
            layout=widgets.Layout(width='100%', max_width='450px', margin='0 auto', justify_content='space-between')
        )

        # Header
        artist_header = widgets.HTML(f"<h3 style='{get_subtitle_style()}'>Your Current Favorite Artists</h3>")

        # Divider
        divider = widgets.HTML(f"""<div style="height:1px; background-color:{get_color('border')}; margin:20px 0;"></div>""")

        # Button container
        buttons = widgets.HBox(
            [confirm_button, back_button],
            layout=widgets.Layout(justify_content='center', margin='20px 0')
        )

        # Main container with simplified structure
        main_container = widgets.VBox([
            header,
            artist_input_container,
            artist_header,
            artist_box,
            divider,
            suggestions_title,
            suggestions_display,
            buttons,
            output
        ], layout=container_layout)

        display(main_container)
    def show_history_page():
        """Display user listening history page"""
        clear_output()

        # Create elements
        header = app_header("📊 Listening History", "Track your music journey", show_back=True)

        # Get user history
        history_df = pd.read_csv(history_csv)
        user_history = history_df[history_df['user_id'] == session['user_id']]

        # Join with songs data to get song names
        songs_df = pd.read_csv(songs_csv)

        if not user_history.empty:
            user_history = user_history.merge(songs_df, on='song_id')
            user_history = user_history.sort_values('timestamp', ascending=False)

            # Create history list
            history_items = ""
            for _, row in user_history.head(10).iterrows():  # Show last 10 songs
                song_info = f"{row['artist']} - {row['song']}"
                mood_before = row['mood_before'] if not pd.isna(row['mood_before']) else 'Unknown'
                mood_after = row['mood_after'] if not pd.isna(row['mood_after']) else 'Unknown'
                timestamp = pd.to_datetime(row['timestamp']).strftime('%Y-%m-%d %H:%M')
                rec_badge = f'<span style="background-color:{get_color("success")}; color:white; padding:3px 8px; border-radius:10px; font-size:12px; margin-left:5px;">Recommended</span>' if row['is_recommended'] == 1 else ''

                history_items += f"""
                <div class="card" style="margin:10px 0; padding:12px;">
                    <div style="display:flex; justify-content:space-between; align-items:center;">
                        <div>
                            <div style="font-weight:bold; color:{get_color('primary')};">{song_info} {rec_badge}</div>
                            <div style="font-size:12px; color:{get_color('text')}80; margin-top:5px;">{timestamp}</div>
                        </div>
                        <div style="text-align:right;">
                            <div style="font-size:13px; color:{get_color('text')};">Mood before: {mood_before}</div>
                            <div style="font-size:13px; color:{get_color('text')};">Mood after: {mood_after}</div>
                        </div>
                    </div>
                </div>
                """

            # Statistics cards
            total_songs = len(user_history)
            unique_artists = user_history['artist'].nunique()
            rec_songs = user_history[user_history['is_recommended'] == 1].shape[0]

            stats_html = f"""
            <div class="stats-container">
                <div class="stat-card">
                    <div class="stat-value">{total_songs}</div>
                    <div class="stat-label">Songs Played</div>
                </div>
                <div class="stat-card">
                    <div class="stat-value">{unique_artists}</div>
                    <div class="stat-label">Artists</div>
                </div>
                <div class="stat-card">
                    <div class="stat-value">{rec_songs}</div>
                    <div class="stat-label">Recommendations</div>
                </div>
            </div>
            """

            history_display = widgets.HTML(f"""
            <div style="margin-bottom:20px;">
                {stats_html}
                <h3 style="{get_subtitle_style()}">Recent Plays</h3>
                <div style="max-height:400px; overflow-y:auto; padding-right:5px;">
                    {history_items}
                </div>
            </div>
            """)

            # Export button
            export_button = widgets.Button(
                description='Export History',
                button_style='info',
                icon='download',
                layout=button_layout
            )

            export_button.on_click(lambda b: show_status("History export feature coming soon!", 'info'))

        else:
            history_display = widgets.HTML(f"""
            <div class="card" style="text-align:center; padding:30px; margin:20px 0;">
                <div style="font-size:48px; margin-bottom:15px;">📊</div>
                <h3 style="color:{get_color('primary')};">No Listening History Yet</h3>
                <p style="color:{get_color('text')};">Start playing songs to build your music journey.</p>
            </div>
            """)
            export_button = None

        # Back button
        back_button = widgets.Button(
            description='Back to Dashboard',
            icon='arrow-left',
            layout=button_layout
        )

        back_button.on_click(lambda b: switch_to_page('session'))

        # Button container
        buttons = widgets.HBox(
            [export_button, back_button] if export_button else [back_button],
            layout=widgets.Layout(justify_content='center', margin='20px 0')
        )

        # Main container
        main_container = widgets.VBox([
            header,
            history_display,
            buttons,
            output
        ], layout=container_layout)

        display(main_container)

    def show_discover_page():
        """Display discover new music page"""
        clear_output()

        # Create elements
        header = app_header("🔍 Discover", "Find new music based on your taste", show_back=True)

        # Get songs data
        songs_df = pd.read_csv(songs_csv)

        # Create filter controls
        genre_options = ['All Genres'] + sorted(songs_df['genre'].dropna().unique().tolist())
        year_options = ['All Years'] + sorted(songs_df['year'].dropna().unique().astype(str).tolist())

        genre_filter = widgets.Dropdown(
            options=genre_options,
            value='All Genres',
            description='Genre:',
            style={'description_width': '80px'}
        )

        year_filter = widgets.Dropdown(
            options=year_options,
            value='All Years',
            description='Year:',
            style={'description_width': '80px'}
        )

        search_input = widgets.Text(
            placeholder='Search songs or artists...',
            description='Search:',
            style={'description_width': '80px'}
        )

        # Results container
        results_container = widgets.Output()

        def update_results(*args):
            """Update the search results based on filters"""
            filtered_df = songs_df.copy()

            # Apply genre filter
            if genre_filter.value != 'All Genres':
                filtered_df = filtered_df[filtered_df['genre'] == genre_filter.value]

            # Apply year filter
            if year_filter.value != 'All Years':
                filtered_df = filtered_df[filtered_df['year'].astype(str) == year_filter.value]

            # Apply search filter
            if search_input.value:
                search_term = search_input.value.lower()
                filtered_df = filtered_df[
                    filtered_df['song'].str.lower().str.contains(search_term, na=False) |
                    filtered_df['artist'].str.lower().str.contains(search_term, na=False)
                ]

            # Display results
            with results_container:
                clear_output()

                if filtered_df.empty:
                    display(widgets.HTML(f"""
                    <div style="text-align:center; padding:20px; color:{get_color('text')};">
                        No songs found matching your criteria.
                    </div>
                    """))
                else:
                    # Limit to 15 songs for performance
                    results = filtered_df.head(15)
                    result_html = ""

                    for _, row in results.iterrows():
                        artist = row['artist']
                        song = row['song']
                        genre = row['genre'] if not pd.isna(row['genre']) else 'Unknown'
                        year = int(row['year']) if not pd.isna(row['year']) else 'Unknown'
                        song_id = row['song_id']

                        result_html += f"""
                        <div class="song-item" style="margin:10px 0; padding:12px;">
                            <div style="display:flex; justify-content:space-between; align-items:center;">
                                <div>
                                    <div style="font-weight:bold; color:{get_color('primary')};">{song}</div>
                                    <div style="font-size:14px; color:{get_color('text')};">{artist}</div>
                                    <div style="font-size:12px; color:{get_color('text')}80; margin-top:3px;">
                                        {genre} • {year}
                                    </div>
                                </div>
                                <button
                                    class="play-song-btn"
                                    data-song-id="{song_id}"
                                    style="background-color:{get_color('success')}; color:white; border:none;
                                           border-radius:50%; width:36px; height:36px; display:flex;
                                           justify-content:center; align-items:center; cursor:pointer;">
                                    ▶
                                </button>
                            </div>
                        </div>
                        """

                    display(widgets.HTML(f"""
                    <div style="margin-top:20px;">
                        <div style="display:flex; justify-content:space-between; align-items:center; margin-bottom:10px;">
                            <h3 style="margin:0; color:{get_color('primary')};">Results ({len(filtered_df)} songs)</h3>
                            <div style="font-size:14px; color:{get_color('text')}80;">
                                Showing {min(15, len(filtered_df))} of {len(filtered_df)}
                            </div>
                        </div>
                        <div style="max-height:400px; overflow-y:auto; padding-right:5px;">
                            {result_html}
                        </div>
                    </div>
                    <script>
                        document.querySelectorAll('.play-song-btn').forEach(btn => {{
                            btn.addEventListener('click', function() {{
                                const songId = this.getAttribute('data-song-id');
                                IPython.notebook.kernel.execute(`play_song_from_discover(${{songId}})`);
                            }});
                        }});
                    </script>
                    """))

        # Initial update
        update_results()

        # Connect filter changes to update function
        genre_filter.observe(update_results, names='value')
        year_filter.observe(update_results, names='value')
        search_input.observe(update_results, names='value')

        def play_song_from_discover(song_id):
            """Play a song from the discover page"""
            if session['mode'] == 'none':
                show_status("Privacy mode → not logging song play.", 'info')
                return

            song = songs_df[songs_df['song_id'] == int(song_id)].iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)

            # Add to recent plays
            if len(session['recent_plays']) >= 10:
                session['recent_plays'].pop()
            session['recent_plays'].insert(0, {
                'song_id': song['song_id'],
                'artist': song['artist'],
                'song': song['song']
            })

            show_status(f"Now playing: {song['artist']} - {song['song']}", 'success')

        # Setup global function for play button
        globals()['play_song_from_discover'] = play_song_from_discover

        # Filter container
        filter_container = widgets.HBox([
            genre_filter,
            year_filter
        ], layout=widgets.Layout(justify_content='space-between', width='100%', margin='10px 0'))

        # Back button
        back_button = widgets.Button(
            description='Back to Dashboard',
            icon='arrow-left',
            layout=button_layout
        )

        back_button.on_click(lambda b: switch_to_page('session'))

        # Main container
        main_container = widgets.VBox([
            header,
            search_input,
            filter_container,
            results_container,
            widgets.HBox([back_button], layout=widgets.Layout(justify_content='center', margin='20px 0')),
            output
        ], layout=container_layout)

        display(main_container)
    def show_profile_page():
        """Display user profile page"""
        clear_output()

        # Create elements
        header = app_header("👤 My Profile", "View and edit your account details", show_back=True)

        # Get user data
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']].iloc[0]

        # Create profile card
        username = user_row['username']
        first_name = user_row['first_name'] if not pd.isna(user_row['first_name']) else ''
        last_name = user_row['last_name'] if not pd.isna(user_row['last_name']) else ''
        full_name = f"{first_name} {last_name}".strip()
        age = int(user_row['age']) if not pd.isna(user_row['age']) else ''
        gender = user_row['gender'] if not pd.isna(user_row['gender']) else ''

        # Get listening statistics
        history_df = pd.read_csv(history_csv)
        user_history = history_df[history_df['user_id'] == session['user_id']]
        total_plays = len(user_history)

        # Get user favorite artists
        fav_str = user_row.get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        fav_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        favorite_artists_html = ""
        for artist in fav_artists:
            favorite_artists_html += f"""
            <span style="display:inline-block; margin:3px; padding:5px 10px; background-color:{get_color('primary')}20;
                         color:{get_color('primary')}; border-radius:15px; font-size:14px;">
                {artist}
            </span>
            """

        if not favorite_artists_html:
            favorite_artists_html = f"""
            <div style="color:{get_color('text')}80; font-style:italic;">
                No favorite artists added yet
            </div>
            """

        # Create profile display
        profile_display = widgets.HTML(f"""
        <div class="card" style="margin:10px 0; padding:20px;">
            <div style="display:flex; align-items:center; margin-bottom:20px;">
                <div style="width:60px; height:60px; border-radius:50%; background-color:{get_color('primary')};
                            display:flex; justify-content:center; align-items:center; margin-right:15px;">
                    <span style="color:white; font-size:24px; font-weight:bold;">{username[0].upper()}</span>
                </div>
                <div>
                    <h3 style="margin:0; color:{get_color('primary')};">{full_name}</h3>
                    <div style="color:{get_color('text')}80;">@{username}</div>
                </div>
            </div>

            <div style="margin:15px 0;">
                <div style="display:flex; margin-bottom:10px;">
                    <div style="width:120px; color:{get_color('text')}80;">Age:</div>
                    <div style="color:{get_color('text')};">{age}</div>
                </div>
                <div style="display:flex; margin-bottom:10px;">
                    <div style="width:120px; color:{get_color('text')}80;">Gender:</div>
                    <div style="color:{get_color('text')};">{gender}</div>
                </div>
                <div style="display:flex; margin-bottom:10px;">
                    <div style="width:120px; color:{get_color('text')}80;">Total Plays:</div>
                    <div style="color:{get_color('text')};">{total_plays}</div>
                </div>
            </div>

            <div style="margin-top:20px;">
                <h4 style="color:{get_color('primary')};">Favorite Artists</h4>
                <div style="margin-top:10px;">
                    {favorite_artists_html}
                </div>
            </div>
        </div>
        """)

        # Action buttons
        edit_profile_button = widgets.Button(
            description='Edit Profile',
            button_style='primary',
            icon='edit',
            layout=button_layout
        )

        edit_favorites_button = widgets.Button(
            description='Edit Favorites',
            button_style='info',
            icon='music',
            layout=button_layout
        )

        change_password_button = widgets.Button(
            description='Change Password',
            icon='lock',
            layout=button_layout
        )

        back_button = widgets.Button(
            description='Back',
            icon='arrow-left',
            layout=button_layout
        )

        # Button handlers
        edit_profile_button.on_click(lambda b: show_status("Profile editing feature coming soon!", 'info'))
        edit_favorites_button.on_click(lambda b: switch_to_page('edit_favorites'))
        change_password_button.on_click(lambda b: show_status("Password change feature coming soon!", 'info'))
        back_button.on_click(lambda b: switch_to_page('session'))

        # Theme toggle
        theme_section = widgets.VBox([
            widgets.HTML(f"<h4 style='color:{get_color('primary')}; text-align:center;'>App Theme</h4>"),
            theme_toggle_button()
        ], layout=widgets.Layout(margin='20px 0', align_items='center'))

        # Button containers
        top_buttons = widgets.HBox(
            [edit_profile_button, edit_favorites_button],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )

        bottom_buttons = widgets.HBox(
            [change_password_button, back_button],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )

        # Main container
        main_container = widgets.VBox([
            header,
            profile_display,
            top_buttons,
            bottom_buttons,
            theme_section,
            output
        ], layout=container_layout)

        display(main_container)

    def show_session_page():
        """Display the enhanced main dashboard page after login"""
        clear_output()

        # Create elements
        header = app_header("🎵 Harmony Hub", "Your personal music dashboard")
        user_info = widgets.HTML(f"""
        <div style="text-align:center; margin-bottom:15px;">
            <span class="profile-badge">👤 {session['username']}</span>
        </div>
        """)

        # Theme toggle
        theme_button = theme_toggle_button()

        # Mode selector with improved styling
        mode_selector = widgets.ToggleButtons(
            options=[
                ('🔒 Private', 'none'),
                ('📝 Log Activity', 'log'),
                ('✨ Recommend', 'recommend')
            ],
            value=session['mode'],
            description='',
            button_style='info',
            tooltips=[
                'Private Mode: No logging or recommendations',
                'Log Mode: Track your listening activity',
                'Recommend Mode: Get personalized recommendations'
            ],
            layout=widgets.Layout(width='auto', margin='0 auto')
        )

        # Mode labels
        mode_labels = {
            'none': '🔒 Private Mode',
            'log': '📝 Log Activity',
            'recommend': '✨ Recommend'
        }

        # Mode description box with enhanced styling
        mode_description = widgets.HTML(
            f"""<div id="mode-desc-box" style='margin:15px auto; padding:15px; background:{get_color('card')};
                border-radius:10px; width:100%; max-width:550px; text-align:center; box-shadow:0 2px 8px rgba(0,0,0,0.06);'>
                <p id='mode-desc' style='margin:0; color:{get_color('text')};'>Select a mode to continue.</p>
            </div>"""
        )

        # Quick action cards
        quick_actions = widgets.HTML(f"""
        <div style="display:flex; flex-wrap:wrap; justify-content:center; gap:15px; margin:20px 0;">
            <div class="card" style="width:150px; height:100px; text-align:center; cursor:pointer;" id="play-random">
                <div style="font-size:32px; margin-bottom:10px;">🎲</div>
                <div style="font-weight:bold; color:{get_color('primary')};">Random Song</div>
            </div>

            <div class="card" style="width:150px; height:100px; text-align:center; cursor:pointer;" id="discover-btn">
                <div style="font-size:32px; margin-bottom:10px;">🔍</div>
                <div style="font-weight:bold; color:{get_color('primary')};">Discover</div>
            </div>

            <div class="card" style="width:150px; height:100px; text-align:center; cursor:pointer;" id="history-btn">
                <div style="font-size:32px; margin-bottom:10px;">📊</div>
                <div style="font-weight:bold; color:{get_color('primary')};">History</div>
            </div>

            <div class="card" style="width:150px; height:100px; text-align:center; cursor:pointer;" id="profile-btn">
                <div style="font-size:32px; margin-bottom:10px;">👤</div>
                <div style="font-weight:bold; color:{get_color('primary')};">Profile</div>
            </div>
        </div>

        <script>
            document.getElementById('play-random').addEventListener('click', function() {{
                IPython.notebook.kernel.execute('play_random_song()');
            }});

            document.getElementById('discover-btn').addEventListener('click', function() {{
                IPython.notebook.kernel.execute('switch_to_page("discover")');
            }});

            document.getElementById('history-btn').addEventListener('click', function() {{
                IPython.notebook.kernel.execute('switch_to_page("history")');
            }});

            document.getElementById('profile-btn').addEventListener('click', function() {{
                IPython.notebook.kernel.execute('switch_to_page("profile")');
            }});
        </script>
        """)
        # Mood section with emoji buttons
        mood_section = widgets.HTML(f"""
        <div style="margin:25px 0; text-align:center;">
            <h3 style="{get_subtitle_style()}">How are you feeling today?</h3>

            <div style="display:flex; justify-content:center; margin:15px 0;">
                <button class="mood-button" id="mood-sad" style="background-color:{get_color('info')};">😢</button>
                <button class="mood-button" id="mood-neutral" style="background-color:{get_color('warning')};">😐</button>
                <button class="mood-button" id="mood-happy" style="background-color:{get_color('success')};">😊</button>
            </div>
        </div>

        <script>
            document.getElementById('mood-sad').addEventListener('click', function() {{
                IPython.notebook.kernel.execute('feel_sad()');
            }});

            document.getElementById('mood-neutral').addEventListener('click', function() {{
                IPython.notebook.kernel.execute('feel_neutral()');
            }});

            document.getElementById('mood-happy').addEventListener('click', function() {{
                IPython.notebook.kernel.execute('feel_happy()');
            }});
        </script>
        """)

        # Recent plays section
        recent_plays_html = ""
        if session['recent_plays']:
            for play in session['recent_plays'][:3]:  # Show only 3 recent plays
                recent_plays_html += f"""
                <div class="song-item" style="cursor:pointer;" data-song-id="{play['song_id']}">
                    <div style="font-weight:bold; color:{get_color('primary')};">{play['song']}</div>
                    <div style="font-size:14px; color:{get_color('text')};">{play['artist']}</div>
                </div>
                """
        else:
            recent_plays_html = f"""
            <div style="text-align:center; padding:15px; color:{get_color('text')}80;">
                No recent plays
            </div>
            """

        recent_plays_section = widgets.HTML(f"""
        <div style="margin:20px 0;">
            <h3 style="{get_subtitle_style()}">Recently Played</h3>
            <div style="max-height:200px; overflow-y:auto;">
                {recent_plays_html}
            </div>

            <div style="text-align:center; margin-top:10px;">
                <button
                    id="see-all-history"
                    style="background:none; border:none; color:{get_color('primary')};
                           text-decoration:underline; cursor:pointer; font-weight:bold;"
                >
                    See Full History
                </button>
            </div>
        </div>

        <script>
            document.getElementById('see-all-history').addEventListener('click', function() {{
                IPython.notebook.kernel.execute('switch_to_page("history")');
            }});

            document.querySelectorAll('.song-item').forEach(item => {{
                item.addEventListener('click', function() {{
                    const songId = this.getAttribute('data-song-id');
                    IPython.notebook.kernel.execute(`replay_song(${{songId}})`);
                }});
            }});
        </script>
        """)

        # Log out button
        exit_button = widgets.Button(
            description='Log Out',
            icon='sign-out',
            button_style='danger',
            layout=button_layout
        )

        def change_mode(change):
            """Handle mode selection change"""
            session['mode'] = change['new']

            # Mode descriptions
            mode_descriptions = {
                'none': "🔒 Private Mode: Your activity won't be logged or used for recommendations.",
                'log': "📝 Log Mode: Your listening activity will be tracked to improve recommendations.",
                'recommend': "✨ Recommend Mode: Tell us how you're feeling and get personalized recommendations!"
            }

            # Update description box
            mode_description.value = f"""<div id="mode-desc-box" style='margin:15px auto; padding:15px; background:{get_color('card')};
                border-radius:10px; width:100%; max-width:550px; text-align:center; box-shadow:0 2px 8px rgba(0,0,0,0.06);'>
                <p id='mode-desc' style='margin:0; color:{get_color('text')};'>{mode_descriptions[session['mode']]}</p>
            </div>"""

            show_status(f"Mode changed to: {mode_labels[session['mode']]}", 'info')

        def play_random_song():
            """Handle play random song button click"""
            show_status("")
            show_loading("Finding a song...")

            if session['mode'] == 'none':
                show_status("Privacy mode → not logging song play.", 'info')
                return

            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)

            # Add to recent plays
            if len(session['recent_plays']) >= 10:
                session['recent_plays'].pop()
            session['recent_plays'].insert(0, {
                'song_id': song['song_id'],
                'artist': song['artist'],
                'song': song['song']
            })

            time.sleep(0.8)  # Reduced delay for better responsiveness
            switch_to_page('session', save_history=False)  # Stay on the same page
            show_status(f"Now playing: {song['artist']} - {song['song']}", 'success')

        def replay_song(song_id):
            """Replay a previously played song"""
            if session['mode'] == 'none':
                show_status("Privacy mode → not logging song play.", 'info')
                return

            songs_df = pd.read_csv(songs_csv)
            song = songs_df[songs_df['song_id'] == int(song_id)].iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)

            # Update recent plays order
            for i, play in enumerate(session['recent_plays']):
                if play['song_id'] == int(song_id):
                    play_data = session['recent_plays'].pop(i)
                    session['recent_plays'].insert(0, play_data)
                    break

            show_status(f"Now playing: {song['artist']} - {song['song']}", 'success')

        def feel_sad():
            """Handle I feel sad button click"""
            show_status("")

            if session['mode'] != 'recommend':
                show_status("Not in recommend mode — please switch to 'Recommend Me Music' mode first.", 'warning')
                return

            show_loading("Finding songs to lift your mood...")
            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("No songs found to recommend. Try adding more favorite artists.", 'error')
                return

            # Create recommendations display
            recommendations_html = ""
            for _, row in recommended.iterrows():
                song_info = f"{row['artist']} - {row['song']}"
                song_id = row['song_id']
                genre = row['genre'] if not pd.isna(row['genre']) else ''
                year = int(row['year']) if not pd.isna(row['year']) else ''
                info_text = f"{genre} • {year}" if genre and year else (genre or year or '')

                recommendations_html += f"""
                <div class="card" style="margin:10px 0; padding:12px; cursor:pointer;" data-song-id="{song_id}">
                    <div style="display:flex; justify-content:space-between; align-items:center;">
                        <div>
                            <div style="font-weight:bold; color:{get_color('primary')};">{row['song']}</div>
                            <div style="font-size:14px; color:{get_color('text')};">{row['artist']}</div>
                            <div style="font-size:12px; color:{get_color('text')}80; margin-top:3px;">
                                {info_text}
                            </div>
                        </div>
                        <div style="background-color:{get_color('success')}; color:white; border-radius:50%;
                                  width:36px; height:36px; display:flex; justify-content:center;
                                  align-items:center;">
                            ▶
                        </div>
                    </div>
                </div>
                """

            with music_recommender_output:
                clear_output()
                display(widgets.HTML(f"""
                <div style="width:100%; max-width:600px; margin:0 auto;">
                    <h3 style="{get_subtitle_style()}">🎵 Songs to Cheer You Up</h3>
                    <p style="text-align:center; margin-bottom:20px; color:{get_color('text')};">
                        Here are some personalized recommendations to boost your mood
                    </p>

                    <div style="max-height:400px; overflow-y:auto; padding-right:5px;">
                        {recommendations_html}
                    </div>

                    <div style="text-align:center; margin-top:20px;">
                        <button
                            id="back-to-dashboard"
                            style="background-color:{get_color('primary')}; color:white; border:none;
                                   border-radius:8px; padding:8px 16px; cursor:pointer; font-weight:bold;"
                        >
                            Back to Dashboard
                        </button>
                    </div>
                </div>

                <script>
                    document.querySelectorAll('.card[data-song-id]').forEach(card => {{
                        card.addEventListener('click', function() {{
                            const songId = this.getAttribute('data-song-id');
                            IPython.notebook.kernel.execute(`play_recommended_song(${{songId}})`);
                        }});
                    }});

                    document.getElementById('back-to-dashboard').addEventListener('click', function() {{
                        IPython.notebook.kernel.execute('switch_to_page("session")');
                    }});
                </script>
                """))

        def play_recommended_song(song_id):
            """Play a recommended song"""
            songs_df = pd.read_csv(songs_csv)
            selected_row = songs_df[songs_df['song_id'] == int(song_id)].iloc[0]
            mood_before = 'sad'
            mood_after = emotion_service.detect()
            logger.log(
                session['user_id'],
                selected_row['song_id'],
                mood_before,
                mood_after,
                is_recommended=1
            )

            # Add to recent plays
            if len(session['recent_plays']) >= 10:
                session['recent_plays'].pop()
            session['recent_plays'].insert(0, {
                'song_id': selected_row['song_id'],
                'artist': selected_row['artist'],
                'song': selected_row['song']
            })

            time.sleep(0.8)  # Reduced delay for better responsiveness
            switch_to_page('session')
            show_status(f"Now playing: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

        def feel_neutral(b=None):
            """Handle neutral mood button click"""
            show_status("")
            show_status("Thanks for sharing your mood! Would you like to discover new music?", 'info')

        def feel_happy(b=None):
            """Handle happy mood button click"""
            show_status("")
            show_status("We're glad you're feeling happy today! 😊 Enjoy your music experience!", 'success')

        def exit_session(b):
            """Handle log out button click"""
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            session['recent_plays'] = []
            show_status("")
            switch_to_page('start')

        # Set up event handlers
        mode_selector.observe(change_mode, names='value')
        exit_button.on_click(exit_session)

        # Make functions available globally
        globals()['play_random_song'] = play_random_song
        globals()['replay_song'] = replay_song
        globals()['feel_sad'] = feel_sad
        globals()['feel_neutral'] = feel_neutral
        globals()['feel_happy'] = feel_happy
        globals()['play_recommended_song'] = play_recommended_song

        # Initialize mode description
        change_mode({'new': session['mode']})

        # Section titles
        mode_title = widgets.HTML(f"<h3 style='{get_subtitle_style()}'>Experience Mode</h3>")

        # Divider
        divider = widgets.HTML(f"""<div style="height:1px; background-color:{get_color('border')}; margin:25px 0 15px;"></div>""")

        # Footer
        footer = widgets.HTML(f"""
        <div style="text-align:center; margin-top:25px; padding-top:20px; border-top:1px solid {get_color('border')};">
            <button
                id="logout-btn"
                style="background:none; border:none; color:{get_color('error')}; cursor:pointer;"
            >
                Log Out
            </button>

            <div style="font-size:12px; color:{get_color('text')}60; margin-top:10px;">
                © 2025 Harmony Hub • Version 2.0
            </div>
        </div>

        <script>
            document.getElementById('logout-btn').addEventListener('click', function() {{
                IPython.notebook.kernel.execute('exit_session(None)');
            }});
        </script>
        """)

        # Main container with simplified structure
        main_container = widgets.VBox([
            header,
            user_info,
            theme_button,
            mode_title,
            mode_selector,
            mode_description,
            divider,
            quick_actions,
            mood_section,
            recent_plays_section,
            footer,
            output
        ], layout=container_layout)

        display(main_container)

    # Start the application at the start page
    switch_to_page('start')

#if __name__ == "__main__":
#    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
start_music_recommender()

In [ ]:
#%%writefile "/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/main.py"
import ipywidgets as widgets
from google.colab import drive
drive.mount('/content/drive')
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
import time
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
project_dir = '/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/'
data_dir = project_dir + 'data/'
emotion_dir=f"{data_dir}emotion_results.csv"
emotion_csv = pd.read_csv(emotion_dir)
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################
def start_music_recommender():
    print("DEBUG: Starting modern minimalist music recommender")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService(emotion_csv)
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start',
        'theme': 'light',  # New theme setting for the user
        'recent_plays': []  # Track recent plays
    }

    output = music_recommender_output

    # Define color schemes - simplified minimalist palette
    colors = {
        'light': {
            'primary': '#4F46E5',  # Indigo
            'secondary': '#0EA5E9',  # Sky blue
            'accent': '#F43F5E',  # Rose
            'success': '#10B981',  # Emerald
            'warning': '#F59E0B',  # Amber
            'error': '#EF4444',  # Red
            'info': '#3B82F6',  # Blue
            'background': '#FFFFFF',
            'card': '#F9FAFB',
            'text': '#1F2937',
            'border': '#E5E7EB'
        },
        'dark': {
            'primary': '#818CF8',  # Indigo lighter
            'secondary': '#38BDF8',  # Sky blue lighter
            'accent': '#FB7185',  # Rose lighter
            'success': '#34D399',  # Emerald lighter
            'warning': '#FBBF24',  # Amber lighter
            'error': '#F87171',  # Red lighter
            'info': '#60A5FA',  # Blue lighter
            'background': '#111827',
            'card': '#1F2937',
            'text': '#F9FAFB',
            'border': '#374151'
        }
    }

    def get_color(name):
        """Get a color from the current theme"""
        return colors[session['theme']][name]

    # Button layout with more subtle styling
    button_layout = widgets.Layout(
        margin="8px",
        width="auto",
        min_width="100px"
    )

    # Container layout - focused width
    container_layout = widgets.Layout(
        display="flex",
        flex_flow="column",
        align_items="center",
        width="100%",
        max_width="600px",
        margin="0 auto"
    )

    # Form layout
    form_layout = widgets.Layout(
        display="flex",
        flex_flow="column",
        align_items="flex-start",
        width="100%",
        max_width="400px",
        margin="0 auto"
    )

    # Apply modern minimalist stylesheet
    display(HTML(f"""
    <style>
    /* Modern minimalist color palette */
    :root {{
        --primary: {get_color('primary')};
        --text: {get_color('text')};
        --background: {get_color('background')};
        --card: {get_color('card')};
        --border: {get_color('border')};
    }}

    /* Base styling reset */
    .widget-button, .widget-text, .widget-password,
    .widget-dropdown, .widget-label, .widget-html,
    .widget-radio, .widget-toggle-buttons {{
        transition: all 0.2s ease;
        font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Helvetica, Arial, sans-serif;
    }}

    /* Typography */
    h1, h2, h3, h4, h5, h6 {{
        font-weight: 500;
        letter-spacing: -0.02em;
    }}

    /* Button styling - minimalist pill shape */
    .widget-button {{
        border-radius: 50px;
        font-weight: 500;
        box-shadow: none;
        text-transform: none;
        letter-spacing: 0;
        font-size: 14px;
        padding: 0 16px;
        height: 36px;
    }}

    .widget-button:hover {{
        transform: translateY(-1px);
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    }}

    /* Input field styling - clean lines */
    .widget-text input,
    .widget-textarea textarea,
    .widget-password input {{
        border-radius: 4px;
        border: 1px solid {get_color('border')};
        padding: 8px 12px;
        width: 100%;
        background-color: {get_color('background')};
        color: {get_color('text')};
    }}

    .widget-text input:focus,
    .widget-textarea textarea:focus,
    .widget-password input:focus {{
        border-color: {get_color('primary')};
        box-shadow: none;
    }}

    /* Label styling - subtle */
    .widget-label {{
        font-weight: 400;
        color: {get_color('text')}90;
    }}

    /* Remove focus outlines */
    .widget-box:focus,
    .jupyter-widgets:focus {{
        outline: none !important;
    }}

    /* Page transition - subtle fade */
    @keyframes fadeIn {{
        from {{ opacity: 0; }}
        to {{ opacity: 1; }}
    }}

    .fade-in {{
        animation: fadeIn 0.3s ease-in-out;
    }}

    /* Content containers */
    .content-card {{
        background-color: {get_color('background')};
        border: 1px solid {get_color('border')};
        border-radius: 8px;
        padding: 16px;
        margin: 8px 0;
        width: 100%;
    }}

    /* Status message styling - minimalist toast */
    .status-message {{
        padding: 10px 16px;
        border-radius: 4px;
        margin: 12px 0;
        text-align: center;
        width: auto;
        display: inline-block;
        font-weight: 400;
        font-size: 14px;
    }}

    .status-info {{
        background-color: {get_color('info')}10;
        color: {get_color('info')};
    }}

    .status-success {{
        background-color: {get_color('success')}10;
        color: {get_color('success')};
    }}

    .status-warning {{
        background-color: {get_color('warning')}10;
        color: {get_color('warning')};
    }}

    .status-error {{
        background-color: {get_color('error')}10;
        color: {get_color('error')};
    }}

    /* Toggle button styling - pill shape */
    .widget-toggle-buttons button {{
        border-radius: 50px !important;
        font-weight: 400 !important;
        padding: 2px 12px !important;
        font-size: 13px !important;
        border: 1px solid {get_color('border')} !important;
        background: transparent !important;
        color: {get_color('text')} !important;
    }}

    .widget-toggle-buttons button.mod-active {{
        background-color: {get_color('primary')} !important;
        color: white !important;
        border-color: {get_color('primary')} !important;
    }}

    /* Song list styling - subtle indication */
    .song-item {{
        padding: 10px;
        border-radius: 4px;
        margin: 4px 0;
        transition: all 0.2s ease;
        border-bottom: 1px solid {get_color('border')}20;
    }}

    .song-item:hover {{
        background-color: {get_color('background')}80;
    }}

    /* Icons and indicators */
    .icon-btn {{
        width: 36px;
        height: 36px;
        border-radius: 50%;
        display: flex;
        align-items: center;
        justify-content: center;
        cursor: pointer;
        transition: all 0.2s ease;
        background: transparent;
        color: {get_color('text')};
    }}

    .icon-btn:hover {{
        background-color: {get_color('background')}80;
    }}

    /* Theme toggle - minimal switch */
    .theme-switch {{
        position: relative;
        display: inline-block;
        width: 34px;
        height: 18px;
    }}

    .slider {{
        position: absolute;
        cursor: pointer;
        top: 0;
        left: 0;
        right: 0;
        bottom: 0;
        background-color: {get_color('border')};
        transition: .3s;
        border-radius: 34px;
    }}

    .slider:before {{
        position: absolute;
        content: "";
        height: 14px;
        width: 14px;
        left: 2px;
        bottom: 2px;
        background-color: white;
        transition: .3s;
        border-radius: 50%;
    }}

    input:checked + .slider {{
        background-color: {get_color('primary')};
    }}

    input:checked + .slider:before {{
        transform: translateX(16px);
    }}

    /* Modern avatars */
    .avatar {{
        width: 36px;
        height: 36px;
        border-radius: 50%;
        display: flex;
        align-items: center;
        justify-content: center;
        background-color: {get_color('primary')}20;
        color: {get_color('primary')};
        font-weight: 500;
        font-size: 14px;
    }}

    /* Clean badge style */
    .badge {{
        display: inline-block;
        padding: 2px 8px;
        border-radius: 50px;
        font-size: 12px;
        font-weight: 500;
    }}

    /* Headers for main app */
    .app-header {{
        margin-bottom: 24px;
        text-align: center;
    }}

    .app-header h1 {{
        font-size: 24px;
        font-weight: 600;
        margin-bottom: 4px;
        color: {get_color('text')};
    }}

    .app-header p {{
        font-size: 14px;
        color: {get_color('text')}80;
        margin: 0;
    }}

    /* Subtle separators */
    .divider {{
        height: 1px;
        background-color: {get_color('border')};
        margin: 16px 0;
        opacity: 0.5;
    }}
    </style>
    """))

    # Navigation history
    nav_history = []

    def show_status(message, status='info'):
        """Display a styled status message"""
        if not message:
            with output:
                clear_output()
            return

        status_classes = {
            'info': 'status-info',
            'success': 'status-success',
            'error': 'status-error',
            'warning': 'status-warning'
        }

        icons = {
            'info': 'ℹ️',
            'success': '✓',
            'error': '✕',
            'warning': '⚠️'
        }

        with output:
            clear_output()
            display(HTML(f"""
            <div class="status-message {status_classes[status]}">
                {icons[status]} {message}
            </div>
            """))

    def show_loading(message="Loading..."):
        """Display a minimal loading indicator"""
        with output:
            clear_output()
            display(HTML(f"""
            <div style="text-align:center; padding:20px;">
                <div style="color:{get_color('text')}80; margin-bottom:12px; font-size:14px;">{message}</div>
                <div style="width:24px; height:24px; border:2px solid {get_color('border')};
                      border-top:2px solid {get_color('primary')}; border-radius:50%;
                      margin:0 auto; animation:spin 1s linear infinite;"></div>
                <style>@keyframes spin {{ 0% {{ transform: rotate(0deg); }} 100% {{ transform: rotate(360deg); }} }}</style>
            </div>
            """))

    def toggle_theme():
        """Toggle between light and dark theme"""
        session['theme'] = 'dark' if session['theme'] == 'light' else 'light'
        # Re-render the current page to apply the new theme
        switch_to_page(session['current_page'], save_history=False)

    def theme_toggle_button():
        """Create a minimal theme toggle button"""
        return widgets.HTML(f"""
        <div style="display:flex; justify-content:center; align-items:center; margin:10px 0;">
            <span style="margin-right:8px; color:{get_color('text')}80; font-size:12px;">Light</span>
            <label class="theme-switch">
                <input type="checkbox" id="theme-toggle" {"checked" if session['theme'] == 'dark' else ""}>
                <span class="slider"></span>
            </label>
            <span style="margin-left:8px; color:{get_color('text')}80; font-size:12px;">Dark</span>
            <script>
                document.getElementById('theme-toggle').addEventListener('change', function() {{
                    IPython.notebook.kernel.execute('toggle_theme()');
                }});
            </script>
        </div>
        """)

    def switch_to_page(page, save_history=True):
        """Switch to a different page in the application"""
        if save_history and session['current_page'] != page:
            nav_history.append(session['current_page'])
        session['current_page'] = page
        show_status("")

        # Add fade transition between pages
        display(HTML("<div class='fade-in'>"))

        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        elif page == 'history':
            show_history_page()
        elif page == 'discover':
            show_discover_page()
        elif page == 'profile':
            show_profile_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

        display(HTML("</div>"))

    def go_back():
        """Navigate to the previous page"""
        if nav_history:
            previous_page = nav_history.pop()
            switch_to_page(previous_page, save_history=False)
        else:
            switch_to_page('start', save_history=False)

    def app_header(title, subtitle=None, show_back=False):
        """Create a minimal header for all pages"""
        back_button = f"""
        <button
            id="back-btn"
            style="position:absolute; left:10px; top:20px; background:none; border:none;
                  color:{get_color('text')}; font-size:18px; cursor:pointer; padding:4px 8px;
                  border-radius:4px;"
        >
            ←
        </button>
        <script>
            document.getElementById('back-btn').addEventListener('click', function() {{
                IPython.notebook.kernel.execute('go_back()');
            }});
        </script>
        """ if show_back else ""

        subtitle_html = f"<p>{subtitle}</p>" if subtitle else ""

        return widgets.HTML(f"""
        <div class="app-header" style="position:relative; width:100%;">
            {back_button}
            <h1>{title}</h1>
            {subtitle_html}
        </div>
        """)

    def show_start_page():
        """Display minimal start page"""
        clear_output()

        # Create elements
        header = app_header("Harmony", "Music for your mood")

        # Theme toggle
        theme_button = theme_toggle_button()

        # Welcome content - minimalist
        welcome_content = widgets.HTML(f"""
        <div style="text-align:center; padding:40px 0;">
            <div style="font-size:36px; margin-bottom:20px;">🎧</div>
            <p style="margin:0 0 24px; color:{get_color('text')};">Simple. Personal. Music.</p>
        </div>
        """)

        # Buttons
        signin_button = widgets.Button(
            description='Sign In',
            button_style='primary',
            layout=button_layout
        )
        signup_button = widgets.Button(
            description='Sign Up',
            layout=button_layout
        )

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        # Button container
        buttons = widgets.HBox(
            [signin_button, signup_button],
            layout=widgets.Layout(justify_content='center', margin='0')
        )

        # Main container with minimal structure
        main_container = widgets.VBox([
            header,
            welcome_content,
            buttons,
            theme_button,
            output
        ], layout=container_layout)

        display(main_container)

    def show_signin_page():
        """Display minimal sign in page"""
        clear_output()

        # Create elements
        header = app_header("Sign In", show_back=True)

        # Form fields with minimal styling
        form_style = {'description_width': '80px'}

        si_username = widgets.Text(
            description='Username',
            style=form_style,
            placeholder='Enter username'
        )
        si_password = widgets.Password(
            description='Password',
            style=form_style,
            placeholder='Enter password'
        )

        # Remember me checkbox - minimal
        remember_me = widgets.Checkbox(
            description='Remember me',
            indent=False,
            layout=widgets.Layout(margin='8px 0 16px 80px')
        )

        # Buttons
        si_button = widgets.Button(
            description='Sign In',
            button_style='primary',
            layout=button_layout
        )
        back_button = widgets.Button(
            description='Back',
            layout=button_layout
        )

        def sign_in_action(b):
            """Handle sign in button click"""
            show_loading("Signing in")

            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("Please enter both username and password", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]

            if user_row.empty:
                show_status("Username not found", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("Incorrect password", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: go_back())

        # Form container - clean layout
        signin_form = widgets.VBox([
            si_username,
            si_password,
            remember_me
        ], layout=form_layout)

        # Button container
        buttons = widgets.HBox([
            si_button,
            back_button
        ], layout=widgets.Layout(justify_content='center', margin='16px 0'))

        # Main container with minimal structure
        main_container = widgets.VBox([
            header,
            signin_form,
            buttons,
            output
        ], layout=container_layout)

        display(main_container)

    def show_signup_page():
        """Display minimal sign up page"""
        clear_output()

        # Create elements
        header = app_header("Create Account", show_back=True)

        # Form fields with minimal styling
        form_style = {'description_width': '100px'}

        su_username = widgets.Text(
            description='Username',
            style=form_style,
            placeholder='Choose a username'
        )
        su_password = widgets.Password(
            description='Password',
            style=form_style,
            placeholder='Create a password'
        )
        su_firstname = widgets.Text(
            description='First Name',
            style=form_style,
            placeholder='Your first name'
        )
        su_lastname = widgets.Text(
            description='Last Name',
            style=form_style,
            placeholder='Your last name'
        )
        su_age = widgets.IntText(
            description='Age',
            style=form_style,
            placeholder='Your age'
        )
        su_gender = widgets.Dropdown(
            options=['M', 'F', 'Other'],
            description='Gender',
            style=form_style
        )

        selected_artists = []

        # Artist selection - minimal
        artist_label = widgets.HTML(f"""<div style="margin:24px 0 8px; color:{get_color('text')}; font-weight:500;">Select Favorite Artists</div>""")
        artist_input = widgets.Combobox(
            placeholder='Search artists',
            options=artist_list,
            description='',  # No label for cleaner look
            ensure_option=True
        )
        add_button = widgets.Button(
            description='Add',
            button_style='info',
            layout=widgets.Layout(width='60px', margin='0 0 0 8px')
        )

        # Artist list container - minimal
        artist_box = widgets.VBox(layout=widgets.Layout(
            width='100%',
            max_width='400px',
            margin='8px auto 16px',
            border=f'1px solid {get_color("border")}',
            padding='8px',
            border_radius='4px',
            max_height='150px',
            overflow_y='auto'
        ))

        def refresh_artist_table():
            """Refresh the list of selected artists"""
            rows = []
            for a in selected_artists:
                row = widgets.HTML(f"""
                <div class="song-item" style="display:flex; justify-content:space-between; align-items:center; padding:6px 0;">
                    <span style="color:{get_color('text')};">{a}</span>
                    <button
                        class="remove-artist"
                        data-artist="{a}"
                        style="background:none; border:none; color:{get_color('text')}60; cursor:pointer; font-size:18px;"
                    >
                        ×
                    </button>
                </div>
                <script>
                    document.querySelectorAll('.remove-artist').forEach(btn => {{
                        btn.addEventListener('click', function() {{
                            const artist = this.getAttribute('data-artist');
                            IPython.notebook.kernel.execute(`selected_artists.remove('${{artist}}'); refresh_artist_table()`);
                        }});
                    }});
                </script>
                """)
                rows.append(row)

            if not selected_artists:
                rows = [widgets.HTML(
                    f'<div style="text-align:center; padding:8px; color:{get_color("text")}60; font-size:14px;">No artists selected</div>'
                )]

            artist_box.children = rows

        def on_add(b):
            """Handle add artist button click"""
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("Already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found", 'warning')

        add_button.on_click(on_add)

        # Create account button and back button
        su_button = widgets.Button(
            description='Create Account',
            button_style='primary',
            layout=button_layout
        )
        back_button = widgets.Button(
            description='Back',
            layout=button_layout
        )

        def sign_up_action(b):
            """Handle create account button click"""
            show_loading("Creating account")

            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("Please enter username and password", 'error')
                return

            if len(su_password.value) < 4:
                show_status("Password must be at least 4 characters", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("Username already taken", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: go_back())

        # Artist input container - clean look
        artist_input_container = widgets.HBox(
            [artist_input, add_button],
            layout=widgets.Layout(width='100%', margin='0')
        )

        # Form container
        user_form = widgets.VBox([
            su_username,
            su_password,
            su_firstname,
            su_lastname,
            su_age,
            su_gender
        ], layout=form_layout)

        # Button container
        buttons = widgets.HBox([
            su_button,
            back_button
        ], layout=widgets.Layout(justify_content='center', margin='16px 0'))

        # Main container with minimal structure
        main_container = widgets.VBox([
            header,
            user_form,
            artist_label,
            artist_input_container,
            artist_box,
            buttons,
            output
        ], layout=container_layout)

        refresh_artist_table()
        display(main_container)

    def show_edit_favorites_page():
        """Display minimal edit favorites page"""
        clear_output()

        # Create elements
        header = app_header("Favorite Artists", show_back=True)

        # Get current favorites
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        # Artist selection - minimal
        artist_input = widgets.Combobox(
            placeholder='Search artists',
            options=artist_list,
            description='',
            ensure_option=True
        )
        add_button = widgets.Button(
            description='Add',
            button_style='info',
            layout=widgets.Layout(width='60px', margin='0 0 0 8px')
        )

        # Artist list container - minimal
        artist_box = widgets.VBox(layout=widgets.Layout(
            width='100%',
            max_width='400px',
            margin='16px auto',
            border=f'1px solid {get_color("border")}',
            padding='8px',
            border_radius='4px',
            max_height='240px',
            overflow_y='auto'
        ))

        def refresh_artist_table():
            """Refresh the list of selected artists"""
            rows = []
            for a in selected_artists:
                row = widgets.HTML(f"""
                <div class="song-item" style="display:flex; justify-content:space-between; align-items:center; padding:6px 0;">
                    <span style="color:{get_color('text')};">{a}</span>
                    <button
                        class="remove-artist"
                        data-artist="{a}"
                        style="background:none; border:none; color:{get_color('text')}60; cursor:pointer; font-size:18px;"
                    >
                        ×
                    </button>
                </div>
                <script>
                    document.querySelectorAll('.remove-artist').forEach(btn => {{
                        btn.addEventListener('click', function() {{
                            const artist = this.getAttribute('data-artist');
                            IPython.notebook.kernel.execute(`selected_artists.remove('${{artist}}'); refresh_artist_table()`);
                        }});
                    }});
                </script>
                """)
                rows.append(row)

            if not selected_artists:
                rows = [widgets.HTML(
                    f'<div style="text-align:center; padding:8px; color:{get_color("text")}60; font-size:14px;">No artists selected</div>'
                )]

            artist_box.children = rows

        def on_add(b):
            """Handle add artist button click"""
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("Already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found", 'warning')

        add_button.on_click(on_add)
        refresh_artist_table()

        # Suggestions section - clean minimal style
        rec_artists_str = user_row.iloc[0].get('recommended_artists', '')
        if pd.isna(rec_artists_str):
            rec_artists_str = ''

        rec_artists = [a.strip() for a in rec_artists_str.split(',') if a.strip()]

        # Only show suggestions section if there are recommendations
        suggestions_display = None
        if rec_artists:
            suggestions_title = widgets.HTML(f"""<div style="margin:16px 0 8px; color:{get_color('text')}; font-weight:500;">Suggestions</div>""")

            suggestions_html = ""
            for artist in rec_artists[:5]:  # Show top 5 suggestions
                if artist and artist not in selected_artists:
                    suggestions_html += f"""
                    <span class="artist-chip"
                         data-artist="{artist}"
                         style="display:inline-block; margin:4px; padding:4px 10px; background-color:{get_color('primary')}10;
                                color:{get_color('primary')}; border-radius:50px; cursor:pointer; font-size:13px;">
                        {artist} +
                    </span>
                    """

            if not suggestions_html:
                suggestions_html = f"""
                <div style="text-align:center; padding:8px; color:{get_color('text')}60; font-size:14px;">
                    No suggestions available
                </div>
                """

            suggestions_display = widgets.HTML(f"""
            <div style="margin:8px 0;">
                {suggestions_html}
            </div>
            <script>
                document.querySelectorAll('.artist-chip').forEach(chip => {{
                    chip.addEventListener('click', function() {{
                        const artist = this.getAttribute('data-artist');
                        if (!selected_artists.includes(artist)) {{
                            IPython.notebook.kernel.execute(`selected_artists.append('${{artist}}'); refresh_artist_table()`);
                        }}
                    }});
                }});
            </script>
            """)

        # Buttons
        save_button = widgets.Button(
            description='Save',
            button_style='primary',
            layout=button_layout
        )
        back_button = widgets.Button(
            description='Cancel',
            layout=button_layout
        )

        def save_favorites(b):
            """Handle save changes button click"""
            show_loading("Saving preferences")

            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists)
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(session['user_id'], selected_artists)
            show_status("Favorites updated", 'success')
            switch_to_page('session')

        save_button.on_click(save_favorites)
        back_button.on_click(lambda b: switch_to_page('session'))

        # Artist input container - minimal
        artist_input_container = widgets.HBox(
            [artist_input, add_button],
            layout=widgets.Layout(width='100%', margin='0', max_width='400px')
        )

        # Button container
        buttons = widgets.HBox(
            [save_button, back_button],
            layout=widgets.Layout(justify_content='center', margin='16px 0')
        )

        # Main container with minimal structure
        components = [header, artist_input_container, artist_box]

        if suggestions_display:
            components.extend([suggestions_title, suggestions_display])

        components.extend([buttons, output])

        main_container = widgets.VBox(components, layout=container_layout)

        display(main_container)

    def show_history_page():
        """Display minimal history page"""
        clear_output()

        # Create elements
        header = app_header("History", show_back=True)

        # Get user history
        history_df = pd.read_csv(history_csv)
        user_history = history_df[history_df['user_id'] == session['user_id']]

        # Join with songs data to get song names
        songs_df = pd.read_csv(songs_csv)

        if not user_history.empty:
            user_history = user_history.merge(songs_df, on='song_id')
            user_history = user_history.sort_values('timestamp', ascending=False)

            # Create history list - clean minimal style
            history_items = ""
            for _, row in user_history.head(10).iterrows():  # Show last 10 songs
                song_info = f"{row['artist']} - {row['song']}"
                timestamp = pd.to_datetime(row['timestamp']).strftime('%m/%d/%y')
                rec_badge = f'<span style="background-color:{get_color("success")}; color:white; padding:2px 6px; border-radius:50px; font-size:11px; margin-left:5px;">Rec</span>' if row['is_recommended'] == 1 else ''

                history_items += f"""
                <div class="song-item" style="padding:8px 0;">
                    <div style="display:flex; justify-content:space-between; align-items:center;">
                        <div>
                            <div style="font-weight:500; color:{get_color('text')};">{song_info} {rec_badge}</div>
                            <div style="font-size:12px; color:{get_color('text')}60;">{timestamp}</div>
                        </div>
                    </div>
                </div>
                """

            # Minimal stats
            total_songs = len(user_history)

            history_display = widgets.HTML(f"""
            <div style="margin:8px 0 24px;">
                <div style="color:{get_color('text')}60; font-size:14px; margin-bottom:16px; text-align:center;">{total_songs} songs played</div>
                <div style="max-height:400px; overflow-y:auto; padding-right:5px;">
                    {history_items}
                </div>
            </div>
            """)

        else:
            history_display = widgets.HTML(f"""
            <div style="text-align:center; padding:32px 0; color:{get_color('text')}60;">
                No listening history yet
            </div>
            """)

        # Back button
        back_button = widgets.Button(
            description='Back',
            layout=button_layout
        )

        back_button.on_click(lambda b: switch_to_page('session'))

        # Button container
        buttons = widgets.HBox(
            [back_button],
            layout=widgets.Layout(justify_content='center')
        )

        # Main container
        main_container = widgets.VBox([
            header,
            history_display,
            buttons,
            output
        ], layout=container_layout)

        display(main_container)

    def show_discover_page():
        """Display minimal discover page"""
        clear_output()

        # Create elements
        header = app_header("Discover", show_back=True)

        # Get songs data
        songs_df = pd.read_csv(songs_csv)

        # Create filter controls - minimal style
        search_input = widgets.Text(
            placeholder='Search songs or artists',
            description='',
            style={'description_width': '0px'}
        )

        # Results container
        results_container = widgets.Output()

        def update_results(*args):
            """Update the search results based on filters"""
            filtered_df = songs_df.copy()

            # Apply search filter
            if search_input.value:
                search_term = search_input.value.lower()
                filtered_df = filtered_df[
                    filtered_df['song'].str.lower().str.contains(search_term, na=False) |
                    filtered_df['artist'].str.lower().str.contains(search_term, na=False)
                ]

            # Display results
            with results_container:
                clear_output()

                if filtered_df.empty:
                    display(widgets.HTML(f"""
                    <div style="text-align:center; padding:16px; color:{get_color('text')}60;">
                        No matching songs found
                    </div>
                    """))
                else:
                    # Limit to 10 songs for cleaner display
                    results = filtered_df.head(10)
                    result_html = ""

                    for _, row in results.iterrows():
                        artist = row['artist']
                        song = row['song']
                        song_id = row['song_id']

                        result_html += f"""
                        <div class="song-item" style="padding:8px 0;">
                            <div style="display:flex; justify-content:space-between; align-items:center;">
                                <div>
                                    <div style="font-weight:500; color:{get_color('text')};">{song}</div>
                                    <div style="font-size:14px; color:{get_color('text')}60;">{artist}</div>
                                </div>
                                <button
                                    class="play-song-btn"
                                    data-song-id="{song_id}"
                                    style="background-color:{get_color('primary')}; color:white; border:none;
                                           border-radius:50%; width:28px; height:28px; display:flex;
                                           justify-content:center; align-items:center; cursor:pointer;"
                                >
                                    ▶
                                </button>
                            </div>
                        </div>
                        """

                    display(widgets.HTML(f"""
                    <div style="margin-top:16px;">
                        <div style="color:{get_color('text')}60; font-size:14px; margin-bottom:16px;">
                            Showing {min(10, len(filtered_df))} of {len(filtered_df)} songs
                        </div>
                        <div style="max-height:400px; overflow-y:auto; padding-right:5px;">
                            {result_html}
                        </div>
                    </div>
                    <script>
                        document.querySelectorAll('.play-song-btn').forEach(btn => {{
                            btn.addEventListener('click', function() {{
                                const songId = this.getAttribute('data-song-id');
                                IPython.notebook.kernel.execute(`play_song_from_discover(${{songId}})`);
                            }});
                        }});
                    </script>
                    """))

        # Initial update
        update_results()

        # Connect search change to update function
        search_input.observe(update_results, names='value')

        def play_song_from_discover(song_id):
            """Play a song from the discover page"""
            if session['mode'] == 'none':
                show_status("Privacy mode enabled", 'info')
                return

            song = songs_df[songs_df['song_id'] == int(song_id)].iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)

            # Add to recent plays
            if len(session['recent_plays']) >= 10:
                session['recent_plays'].pop()
            session['recent_plays'].insert(0, {
                'song_id': song['song_id'],
                'artist': song['artist'],
                'song': song['song']
            })

            show_status(f"Playing: {song['song']}", 'success')

        # Setup global function for play button
        globals()['play_song_from_discover'] = play_song_from_discover

        # Back button
        back_button = widgets.Button(
            description='Back',
            layout=button_layout
        )

        back_button.on_click(lambda b: switch_to_page('session'))

        # Main container
        main_container = widgets.VBox([
            header,
            search_input,
            results_container,
            widgets.HBox([back_button], layout=widgets.Layout(justify_content='center', margin='16px 0')),
            output
        ], layout=container_layout)

        display(main_container)

    def show_profile_page():
        """Display minimal profile page"""
        clear_output()

        # Create elements
        header = app_header("Profile", show_back=True)

        # Get user data
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']].iloc[0]

        # Create profile display - minimal style
        username = user_row['username']
        first_name = user_row['first_name'] if not pd.isna(user_row['first_name']) else ''
        last_name = user_row['last_name'] if not pd.isna(user_row['last_name']) else ''
        full_name = f"{first_name} {last_name}".strip()

        # Profile display
        profile_display = widgets.HTML(f"""
        <div style="text-align:center; margin:24px 0;">
            <div class="avatar" style="margin:0 auto 16px; width:64px; height:64px; font-size:24px;">{username[0].upper()}</div>
            <div style="font-weight:500; font-size:18px; margin-bottom:4px; color:{get_color('text')};">{full_name}</div>
            <div style="color:{get_color('text')}60;">@{username}</div>
        </div>
        """)

        # Action buttons - horizontal layout for minimal design
        edit_favorites_button = widgets.Button(
            description='Edit Favorites',
            button_style='primary',
            layout=button_layout
        )

        theme_toggle = theme_toggle_button()

        logout_button = widgets.Button(
            description='Log Out',
            button_style='danger',
            layout=button_layout
        )

        # Handlers
        edit_favorites_button.on_click(lambda b: switch_to_page('edit_favorites'))
        logout_button.on_click(lambda b: exit_session(None))

        # Buttons in horizontal layout
        buttons = widgets.HBox(
            [edit_favorites_button, logout_button],
            layout=widgets.Layout(justify_content='center', margin='16px 0')
        )

        # Main container
        main_container = widgets.VBox([
            header,
            profile_display,
            buttons,
            theme_toggle,
            output
        ], layout=container_layout)

        display(main_container)

    def show_session_page():
        """Display a minimalist main dashboard page after login"""
        clear_output()

        # Create elements - minimalist header
        header = app_header("Harmony", session['username'])

        # Mode selector - minimal pill buttons
        mode_selector = widgets.ToggleButtons(
            options=[
                ('Private', 'none'),
                ('Log', 'log'),
                ('Recommend', 'recommend')
            ],
            value=session['mode'],
            description='',
            button_style='info',
            layout=widgets.Layout(width='auto', margin='0 auto 16px')
        )

        # Minimal action buttons - use widgets for reliability
        play_button = widgets.Button(
            description='Play',
            icon='play',
            button_style='success',
            layout=button_layout
        )

        discover_button = widgets.Button(
            description='Discover',
            icon='search',
            layout=button_layout
        )

        history_button = widgets.Button(
            description='History',
            icon='list',
            layout=button_layout
        )

        profile_button = widgets.Button(
            description='Profile',
            icon='user',
            layout=button_layout
        )

        # Button handlers
        play_button.on_click(lambda b: play_random_song())
        discover_button.on_click(lambda b: switch_to_page('discover'))
        history_button.on_click(lambda b: switch_to_page('history'))
        profile_button.on_click(lambda b: switch_to_page('profile'))

        # Action buttons in horizontal layout
        action_buttons = widgets.HBox(
            [play_button, discover_button, history_button, profile_button],
            layout=widgets.Layout(justify_content='center', margin='16px 0')
        )

        # Minimalist mood selector
        mood_buttons = widgets.HTML(f"""
        <div style="text-align:center; margin:24px 0;">
            <div style="color:{get_color('text')}60; font-size:14px; margin-bottom:12px;">How are you feeling?</div>
            <div style="display:flex; justify-content:center; gap:24px;">
                <button class="icon-btn" id="mood-sad">😢</button>
                <button class="icon-btn" id="mood-happy">😊</button>
            </div>
        </div>
        <script>
            document.getElementById('mood-sad').addEventListener('click', function() {{
                IPython.notebook.kernel.execute('feel_sad()');
            }});
            document.getElementById('mood-happy').addEventListener('click', function() {{
                IPython.notebook.kernel.execute('feel_happy()');
            }});
        </script>
        """)

        # Now playing - only show if there's something recent
        now_playing = None
        if session['recent_plays']:
            latest = session['recent_plays'][0]
            now_playing = widgets.HTML(f"""
            <div style="text-align:center; margin:16px 0; padding:16px; border:1px solid {get_color('border')}; border-radius:4px;">
                <div style="color:{get_color('text')}60; font-size:12px; margin-bottom:8px;">Now Playing</div>
                <div style="font-weight:500; color:{get_color('text')};">{latest['song']}</div>
                <div style="color:{get_color('text')}80; font-size:14px;">{latest['artist']}</div>
            </div>
            """)

        # Minimal footer with logout
        footer = widgets.HTML(f"""
        <div style="text-align:center; margin-top:24px;">
            <button
                id="logout-btn"
                style="background:none; border:none; color:{get_color('text')}60;
                       font-size:14px; cursor:pointer; text-decoration:underline;"
            >
                Log out
            </button>
        </div>
        <script>
            document.getElementById('logout-btn').addEventListener('click', function() {{
                IPython.notebook.kernel.execute('exit_session(None)');
            }});
        </script>
        """)

        # Mode change handler
        def change_mode(change):
            """Handle mode selection change"""
            session['mode'] = change['new']
            show_status(f"Mode: {change['new']}", 'info')

        mode_selector.observe(change_mode, names='value')

        def play_random_song():
            """Play a random song"""
            if session['mode'] == 'none':
                show_status("Privacy mode enabled", 'info')
                return

            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)

            # Add to recent plays
            if len(session['recent_plays']) >= 10:
                session['recent_plays'].pop()
            session['recent_plays'].insert(0, {
                'song_id': song['song_id'],
                'artist': song['artist'],
                'song': song['song']
            })

            switch_to_page('session', save_history=False)  # Stay on the same page
            show_status(f"Playing: {song['song']}", 'success')

        def feel_sad():
            """Handle sad mood"""
            if session['mode'] != 'recommend':
                show_status("Switch to Recommend mode", 'warning')
                return

            show_loading("Finding recommendations")
            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("No recommendations found", 'error')
                return

            # Clean minimal recommendations display
            recommendations_html = ""
            for _, row in recommended.iterrows():
                song_id = row['song_id']

                recommendations_html += f"""
                <div class="song-item" style="padding:8px 0;">
                    <div style="display:flex; justify-content:space-between; align-items:center;">
                        <div>
                            <div style="font-weight:500; color:{get_color('text')};">{row['song']}</div>
                            <div style="font-size:14px; color:{get_color('text')}60;">{row['artist']}</div>
                        </div>
                        <button
                            class="play-song-btn"
                            data-song-id="{song_id}"
                            style="background-color:{get_color('primary')}; color:white; border:none;
                                   border-radius:50%; width:28px; height:28px; display:flex;
                                   justify-content:center; align-items:center; cursor:pointer;"
                        >
                            ▶
                        </button>
                    </div>
                </div>
                """

            with music_recommender_output:
                clear_output()
                display(widgets.HTML(f"""
                <div style="padding:16px 0;">
                    <div style="color:{get_color('text')}; font-weight:500; text-align:center; margin-bottom:16px;">
                        Recommendations for you
                    </div>

                    <div style="max-height:300px; overflow-y:auto;">
                        {recommendations_html}
                    </div>

                    <div style="text-align:center; margin-top:16px;">
                        <button
                            id="back-to-dashboard"
                            style="background:none; border:none; color:{get_color('primary')};
                                   cursor:pointer; text-decoration:underline; font-size:14px;"
                        >
                            Back
                        </button>
                    </div>
                </div>

                <script>
                    document.querySelectorAll('.play-song-btn').forEach(btn => {{
                        btn.addEventListener('click', function() {{
                            const songId = this.getAttribute('data-song-id');
                            IPython.notebook.kernel.execute(`play_recommended_song(${{songId}})`);
                        }});
                    }});

                    document.getElementById('back-to-dashboard').addEventListener('click', function() {{
                        IPython.notebook.kernel.execute('switch_to_page("session")');
                    }});
                </script>
                """))

        def play_recommended_song(song_id):
            """Play a recommended song"""
            songs_df = pd.read_csv(songs_csv)
            selected_row = songs_df[songs_df['song_id'] == int(song_id)].iloc[0]
            mood_before = 'sad'
            mood_after = emotion_service.detect()
            logger.log(
                session['user_id'],
                selected_row['song_id'],
                mood_before,
                mood_after,
                is_recommended=1
            )

            # Add to recent plays
            if len(session['recent_plays']) >= 10:
                session['recent_plays'].pop()
            session['recent_plays'].insert(0, {
                'song_id': selected_row['song_id'],
                'artist': selected_row['artist'],
                'song': selected_row['song']
            })

            switch_to_page('session')
            show_status(f"Playing: {selected_row['song']}", 'success')

        def feel_happy():
            """Handle happy mood"""
            show_status("Enjoy your music", 'success')

        def exit_session(b):
            """Log out"""
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            session['recent_plays'] = []
            switch_to_page('start')

        # Make functions available globally
        globals()['play_random_song'] = play_random_song
        globals()['feel_sad'] = feel_sad
        globals()['feel_happy'] = feel_happy
        globals()['exit_session'] = exit_session
        globals()['play_recommended_song'] = play_recommended_song

        # Assemble components
        components = [header, mode_selector, action_buttons, mood_buttons]

        if now_playing:
            components.append(now_playing)

        components.extend([footer, output])

        # Main container with minimal structure
        main_container = widgets.VBox(
            components,
            layout=container_layout
        )

        display(main_container)

    # Start the application at the start page
    switch_to_page('start')

#if __name__ == "__main__":
#    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
start_music_recommender()

In [ ]:
#%%writefile "/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/main.py"
import ipywidgets as widgets
from google.colab import drive
drive.mount('/content/drive')
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
import time
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService  # ✅ import the new class

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager  # ✅ import the new class

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
project_dir = '/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/'
data_dir = project_dir + 'data/'
emotion_dir=f"{data_dir}emotion_results.csv"
emotion_csv = pd.read_csv(emotion_dir)
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################
def start_music_recommender():
    print("DEBUG: Starting enhanced multi-page recommender with improved UI/UX")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService(emotion_csv)
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    # Define common styles
    main_title_style = "font-size:24px; font-weight:bold; margin-bottom:15px; text-align:center; color:teal;"
    subtitle_style = "font-size:18px; font-weight:bold; margin-top:15px; margin-bottom:10px; text-align:center; color:teal;"

    # Improved button styles with transitions
    button_layout = widgets.Layout(
        margin="8px",
        width="auto",
        min_width="120px"
    )

    # Common container layout - consistently centered with fixed width
    container_layout = widgets.Layout(
        display="flex",
        flex_flow="column",
        align_items="center",
        width="100%",
        max_width="450px",
        margin="0 auto",
        overflow="hidden",  # Changed from hidden to visible
        padding="20px",
        border="none"
    )

    # Update the form_layout definition
    form_layout = widgets.Layout(
        align_items="center",
        width='100%',          # Changed from 300px to 100%
        max_width='450px',     # Adding this line to limit width
        margin='0 auto'        # Keep auto margins for centering
    )

    # Define CSS for transitions and smooth animations
    display(HTML("""
    <style>
    /* Smooth transitions for all elements */
    .widget-button, .widget-text, .widget-password,
    .widget-dropdown, .widget-label, .widget-html {
        transition: all 0.3s ease-in-out;
    }

    /* Improved Button Hover Effects */
    .widget-button:hover {{
        transform: translateY(-2px);
        box-shadow: 0 4px 8px rgba(0,0,0,0.1);
    }}

    /* Remove outline focus on widgets */
    .widget-box:focus {
        outline: none !important;
    }

    /* Comprehensive scrollbar hiding */
    ::-webkit-scrollbar {
        width: 0px !important;
        height: 0px !important;
        background: white !important;
        display: none !important;
    }

    /* Hide scrollbars everywhere - more aggressive approach */
    * {
        -ms-overflow-style: none !important;  /* IE and Edge */
        scrollbar-width: none !important;  /* Firefox */
    }

    /* Additional scrollbar hiding for specific containers */
    .widget-box, .widget-vbox, .widget-hbox,
    .jupyter-widgets, .widget-container,
    .p-Widget, .p-Panel, .lm-Widget, .lm-Panel {
        overflow: hidden !important;
        scrollbar-width: none !important;
    }

    /* Force all Jupyter output areas to hide scrollbars */
    .jp-OutputArea, .jp-OutputArea-output, .jp-OutputArea-child {
        overflow: hidden !important;
        max-height: none !important;
    }

    /* Force the main output widget to expand as needed without scrollbars */
    .output_scroll {
        height: auto !important;
        max-height: none !important;
        overflow-y: hidden !important;
    }

    /* Fade animation for page transitions */
    @keyframes fadeIn {{
        from {{ opacity: 0; transform: translateY(10px); }}
        to {{ opacity: 1; transform: translateY(0); }}
    }}

    .fade-in {
        animation: fadeIn 0.4s ease-in-out;
    }

    /* Ensure proper sizing for input elements */
    .widget-text, .widget-textarea, .widget-password {
        box-sizing: border-box !important;
        width: 100% !important;
    }

    /* Dark mode compatibility - ensure text is visible */
    .widget-text input, .widget-textarea textarea, .widget-password input,
    .widget-text, .widget-textarea, .widget-password,
    .widget-label, .widget-html, .widget-dropdown {
          color: #333 !important;
          background-color: #fff !important;
    }

    /* Ensure form labels are visible in dark mode */
    .widget-label {
      color: white !important;
      background-color: transparent !important;
    }

    /* Fix input backgrounds in dark mode */
    .widget-text .widget-text-input,
    .widget-textarea .widget-textarea-input,
    .widget-password .widget-password-input {
          background-color: #fff !important;
          color: #333 !important;
          border: 1px solid #ccc !important;
    }

    /* Make HTML element content visible in dark mode */
    .jupyter-widgets.widget-html-content {
          color: white !important;
    }

    /* Fix the title styling for dark mode */
    h1[style*='color:#3366cc'], h2[style*='color:#333'] {
          color: white !important;
    }

    /* Style the container backgrounds in dark mode */
    .widget-container {
          background-color: transparent !important;
    }

    /* Fix dropdown styling for dark mode */
    .widget-dropdown > select {
          background-color: white !important;
          color: #333 !important;
    }

    /* Fix for mode description box */
    div[style*='background:#f8f8f8'] {
          background: #444 !important;
          color: white !important;
    }
    div[style*='background:#f8f8f8'] p {
          color: white !important;
    }

    /* Status message styling for dark mode */
    div[style*='background-color:#ff990015'] {
          background-color: rgba(255, 153, 0, 0.2) !important;
    }
    div[style*='background-color:#00aa0015'] {
          background-color: rgba(0, 170, 0, 0.2) !important;
    }
    div[style*='background-color:#cc000015'] {
          background-color: rgba(204, 0, 0, 0.2) !important;
    }
    div[style*='background-color:#ff660015'] {
          background-color: rgba(255, 102, 0, 0.2) !important;
    }

    /* Make sure HTML-based content is visible */
    p[style*='text-align:center'], p[style*='margin-bottom'] {
          color: white !important;
    }
    </style>
    """))

    # Navigation history
    nav_history = []

    def show_status(message, status='info'):
        colors = {'info': '#ff9900', 'success': '#00aa00', 'error': '#cc0000', 'warning': '#ff6600'}
        icons = {'info': 'ℹ️', 'success': '✅', 'error': '❌', 'warning': '⚠️'}
        with output:
            clear_output()
            if message:
                display(HTML(f"""
                <div style='padding:10px; border-radius:8px; background-color:{colors[status]}15;
                     border-left:4px solid {colors[status]}; margin:15px auto; max-width:500px; text-align:center;'>
                    <span style='color:{colors[status]}; font-weight:bold'>{icons[status]} {message}</span>
                </div>
                """))

    def show_loading(message="Loading..."):
        with output:
            clear_output()
            display(HTML(f"""
            <div style='text-align:center; padding:15px;'>
                <div style='font-weight:bold; margin-bottom:15px;'>{message}</div>
                <div style='display:inline-block; width:40px; height:40px; border:5px solid #f3f3f3;
                     border-top:5px solid #3498db; border-radius:50%; animation:spin 1s linear infinite;'></div>
                <style>@keyframes spin {{0% {{ transform: rotate(0deg); }} 100% {{ transform: rotate(360deg); }}}}</style>
            </div>
            """))

    def switch_to_page(page, save_history=True):
        if save_history and session['current_page'] != page:
            nav_history.append(session['current_page'])
        session['current_page'] = page
        show_status("")

        # Add fade transition between pages
        display(HTML("<div class='fade-in'>"))

        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

        display(HTML("</div>"))

    def go_back():
        if nav_history:
            previous_page = nav_history.pop()
            switch_to_page(previous_page, save_history=False)
        else:
            switch_to_page('start', save_history=False)

    def show_start_page():
        clear_output()
        start_label = widgets.HTML(f"<h1 style='{main_title_style}'>🎵 Music Recommender</h1>")
        welcome_text = widgets.HTML("<p style='margin-bottom:20px; text-align:center;'>Welcome to your personal music recommendation system.</p>")
        signin_button = widgets.Button(description='Sign In', button_style='primary', layout=button_layout)
        signup_button = widgets.Button(description='Sign Up', layout=button_layout)

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            welcome_text,
            widgets.HBox([signin_button, signup_button], layout=widgets.Layout(justify_content='center')),
            output
        ], layout=container_layout))

    def show_signin_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🔑 Sign In</h1>")

        # Consistent form field styling
        form_style = {'description_width': '80px'}
        #form_layout = widgets.Layout(width='100%',max_width='450px', margin='0 auto')

        si_username = widgets.Text(description='Username', style=form_style)
        si_password = widgets.Password(description='Password', style=form_style)
        si_button = widgets.Button(description='Sign In', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Back', layout=button_layout)

        def sign_in_action(b):
            show_loading("Signing in...")

            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: go_back())

        buttons_layout = widgets.Layout(justify_content='center', margin='15px 0')

        # Create a simplified container without nested layouts
        signin_form = widgets.VBox([
            si_username,
            si_password
        ], layout=form_layout)

        buttons = widgets.HBox([
            si_button,
            back_button
        ], layout=buttons_layout)

        # Use minimal nesting to avoid scroll issues
        main_container = widgets.VBox([
            label,
            signin_form,
            buttons,
            output
        ], layout=container_layout)

        # Direct display without further nesting
        display(main_container)

    def show_signup_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🔐 Sign Up</h1>")

        # Create form fields with consistent styling
        form_style = {'description_width': '100px'}
        #form_layout = widgets.Layout(width='100%',max_width='450px',margin='0 auto')

        su_username = widgets.Text(description='Username', style=form_style)
        su_password = widgets.Password(description='Password', style=form_style)
        su_firstname = widgets.Text(description='First Name', style=form_style)
        su_lastname = widgets.Text(description='Last Name', style=form_style)
        su_age = widgets.IntText(description='Age', style=form_style)
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender', style=form_style)

        selected_artists = []

        artist_header = widgets.HTML(f"<h2 style='{subtitle_style}'>🎵 Select Favorite Artists</h2>")
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(description='Add', layout=widgets.Layout(width='80px'))

        # Use fixed height with visible overflow only for the artist box
        artist_box = widgets.VBox(layout=widgets.Layout(
            width='300px',
            margin='10px auto',
            # No max-height to prevent scroll issues
        ))

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(
                    description='Remove',
                    button_style='danger',
                    layout=widgets.Layout(width='80px')
                )
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn], layout=widgets.Layout(justify_content='space-between', width='100%')))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)

        su_button = widgets.Button(description='Create Account', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Back', layout=button_layout)

        def sign_up_action(b):
            show_loading("Creating account...")

            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            if len(su_password.value) < 4:
                show_status("Password should be at least 4 characters long.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: go_back())

        artist_input_container = widgets.HBox([artist_input, add_button],
                                             layout=widgets.Layout(width='400px', margin='0 auto', justify_content='space-between'))

        buttons_layout = widgets.Layout(justify_content='center', margin='15px 0')

        # Group form fields to reduce nesting
        user_form = widgets.VBox([
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender
        ], layout=form_layout)

        buttons = widgets.HBox([
            su_button, back_button
        ], layout=buttons_layout)

        # Simplified container structure
        main_container = widgets.VBox([
            label,
            user_form,
            artist_header,
            artist_input_container,
            artist_box,
            buttons,
            output
        ], layout=container_layout)

        display(main_container)

    def show_edit_favorites_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🎨 Edit Favorite Artists</h1>")
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        form_style = {'description_width': '100px'}
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(description='Add', layout=widgets.Layout(width='80px'))

        # Remove fixed height and scroll settings
        artist_box = widgets.VBox(layout=widgets.Layout(
            width='400px',
            margin='10px auto'
        ))

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(
                    description='Remove',
                    button_style='danger',
                    layout=widgets.Layout(width='80px')
                )
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn], layout=widgets.Layout(justify_content='space-between', width='100%')))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)
        refresh_artist_table()

        confirm_button = widgets.Button(description='Save Changes', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Cancel', layout=button_layout)

        def save_favorites(b):
            show_loading("Saving your preferences...")

            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists)
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(session['user_id'], selected_artists)
            show_status("Your favorite artists have been updated successfully!", 'success')
            switch_to_page('session')

        confirm_button.on_click(save_favorites)
        back_button.on_click(lambda b: switch_to_page('session'))

        artist_input_container = widgets.HBox([artist_input, add_button],
                                             layout=widgets.Layout(width='100%',max_width='400px',margin='0 auto', justify_content='space-between'))

        artist_header = widgets.HTML(f"<p style='{subtitle_style}'>Your Current Favorite Artists:</p>")

        buttons_layout = widgets.Layout(justify_content='center', margin='15px 0')
        buttons = widgets.HBox([confirm_button, back_button], layout=buttons_layout)

        # Simplified structure
        main_container = widgets.VBox([
            label,
            artist_input_container,
            artist_header,
            artist_box,
            buttons,
            output
        ], layout=container_layout)

        display(main_container)

    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<h1 style='{main_title_style}'>🎵 Music Recommender</h1>")
        user_info = widgets.HTML(f"<p style='text-align:center; margin-bottom:15px;'><b>👤 Signed in as:</b> {session['username']}</p>")

        # Improved mode selector with better spacing
        mode_selector = widgets.ToggleButtons(
            options=[
                ('Private Mode', 'none'),
                ('Log My Activity', 'log'),
                ('Recommend Me Music', 'recommend')
            ],
            value=session['mode'],
            description='Mode:',
            button_style='info',
            tooltips=[
                'Private Mode: No logging or recommendations',
                'Log Mode: Track your listening activity',
                'Recommend Mode: Get personalized music recommendations'
            ],
            layout=widgets.Layout(width='auto', margin='0 auto')
        )

        # Create a mapping for mode labels
        mode_labels = {
            'none': 'Private Mode',
            'log': 'Log My Activity',
            'recommend': 'Recommend Me Music'
        }

        mode_description = widgets.HTML(
            f"""<div style='margin:15px auto; padding:12px; background:#f8f8f8; border-radius:8px; width:500px; text-align:center;'>
                <p id='mode-desc' style='margin:0;'>Select a mode to continue.</p>
            </div>"""
        )

        # Action buttons with consistent styling and better layout
        play_button = widgets.Button(
            description='Play Random Song',
            icon='play',
            button_style='success',
            layout=button_layout
        )
        sad_button = widgets.Button(
            description='I Feel Sad 😢',
            button_style='info',
            layout=button_layout
        )
        happy_button = widgets.Button(
            description='I Feel Happy 😊',
            button_style='info',
            layout=button_layout
        )
        fav_edit_button = widgets.Button(
            description='Edit Favorites',
            icon='edit',
            layout=button_layout
        )
        exit_button = widgets.Button(
            description='Log Out',
            icon='sign-out',
            button_style='danger',
            layout=button_layout
        )

        def change_mode(change):
            session['mode'] = change['new']

            # Update mode description based on selection
            mode_descriptions = {
                'none': "Private Mode: Your activity won't be logged or used for recommendations.",
                'log': "Log Mode: Your listening activity will be tracked to improve recommendations.",
                'recommend': "Recommend Mode: Tell us how you're feeling and get personalized recommendations!"
            }
            mode_description.value = f"""<div style='margin:15px auto; padding:12px; background:#f8f8f8; border-radius:8px; width:500px; text-align:center;'>
                <p id='mode-desc' style='margin:0;'>{mode_descriptions[session['mode']]}</p>
            </div>"""

            show_status(f"Mode changed to: {mode_labels[session['mode']]}", 'info')

        def play_song(b):
            show_status("")
            show_loading("Finding a song...")

            if session['mode'] == 'none':
                show_status("Privacy mode → not logging song play.", 'info')
                return

            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)

            #show_status(f"Now playing: {song['artist']} - {song['song']}", 'success')
            time.sleep(2)
            switch_to_page('session')
            show_status(f"Now playing: {song['artist']} - {song['song']}", 'success')



        def feel_sad(b):
            show_status("")

            if session['mode'] != 'recommend':
                show_status("Not in recommend mode — please switch to 'Recommend Me Music' mode first.", 'warning')
                return

            show_loading("Finding songs to lift your mood...")
            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("No songs found to recommend. Try adding more favorite artists.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]

            # Create a nicer song selection widget
            song_header = widgets.HTML("<h3 style='margin-top:15px; text-align:center;'>🎵 Songs to cheer you up:</h3>")
            song_options = widgets.RadioButtons(
                options=options,
                layout=widgets.Layout(width='auto', margin='10px auto')
            )
            confirm_button = widgets.Button(
                description='Play Selected Song',
                button_style='success',
                layout=widgets.Layout(margin='10px auto', display='flex', justify_content='center')
            )

            def on_confirm_click(_):
                if not song_options.value:
                    show_status("Please select a song first", 'warning')
                    return

                show_loading("Playing your selection...")
                selected_label = song_options.value
                selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(
                    session['user_id'],
                    selected_row['song_id'],
                    mood_before,
                    mood_after,
                    is_recommended=1
                )
                time.sleep(2)
                switch_to_page('session')
                show_status(f"Now playing: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

            confirm_button.on_click(on_confirm_click)

            with music_recommender_output:
                clear_output()
                display(widgets.VBox([
                    song_header,
                    song_options,
                    confirm_button
                ], layout=widgets.Layout(align_items='center', width='500px', margin='0 auto')))

        def feel_happy(b):
            show_status("")
            show_status("Thanks for sharing! We're glad you're feeling happy today. 😊", 'success')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        fav_edit_button.on_click(lambda b: switch_to_page('edit_favorites'))
        exit_button.on_click(exit_session)

        # Initialize mode description
        change_mode({'new': session['mode']})

        # Action buttons container with centered layout
        action_buttons = widgets.HBox(
            [play_button, sad_button, happy_button],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )

        # Profile buttons container with centered layout
        profile_buttons = widgets.HBox(
            [fav_edit_button, exit_button],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )

        action_title = widgets.HTML(f"<h2 style='{subtitle_style}'>Actions:</h2>")
        profile_title = widgets.HTML(f"<h2 style='{subtitle_style}'>Profile Settings:</h2>")
        mode_title = widgets.HTML(f"<h2 style='{subtitle_style}'>Select Your Experience Mode:</h2>")

        display(widgets.VBox([
            profile_label,
            user_info,
            mode_title,
            mode_selector,
            mode_description,
            action_title,
            action_buttons,
            profile_title,
            profile_buttons,
            output
        ], layout=container_layout))

    switch_to_page('start')
    #print("✅ Enhanced musicalllllll recommender interface loaded successfully!")

#if __name__ == "__main__":
#    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
start_music_recommender()

In [ ]:
launch_admin_ui()

In [ ]:
start_music_recommender()

In [ ]:
# Updates to main.py for improved UI/UX
import ipywidgets as widgets
from google.colab import drive
drive.mount('/content/drive')
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
import time
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
project_dir = '/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/'
data_dir = project_dir + 'data/'
emotion_dir=f"{data_dir}emotion_results.csv"
emotion_csv = pd.read_csv(emotion_dir)
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################
def start_music_recommender():
    print("DEBUG: Starting enhanced multi-page recommender with improved UI/UX")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService(emotion_csv)
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    # Updated color scheme with a music-themed palette
    colors = {
        'primary': '#7851a9',  # Purple - main brand color
        'secondary': '#3d2c8d', # Darker purple for contrast
        'accent': '#f67280',    # Coral pink for accent elements
        'light': '#f9f7ff',     # Almost white with slight purple tint
        'dark': '#1e1e2f',      # Dark blue-gray for text
        'success': '#00c07f',   # Green for success messages
        'warning': '#ffb549',   # Amber for warnings
        'error': '#ff5252',     # Red for errors
        'info': '#4da6ff'       # Blue for info messages
    }

    # Define common styles with updated color scheme
    main_title_style = f"font-size:28px; font-weight:bold; margin-bottom:15px; text-align:center; color:{colors['primary']};"
    subtitle_style = f"font-size:20px; font-weight:bold; margin-top:15px; margin-bottom:10px; text-align:center; color:{colors['secondary']};"

    # Improved button styles with rounded corners and music theme
    button_layout = widgets.Layout(
        margin="12px",
        width="auto",
        min_width="130px"
    )

    # Common container layout with improved spacing
    container_layout = widgets.Layout(
        display="flex",
        flex_flow="column",
        align_items="center",
        width="100%",
        max_width="500px",
        margin="0 auto",
        overflow="hidden",
        padding="25px",
        border="none"
    )

    form_layout = widgets.Layout(
        align_items="center",
        width='100%',
        max_width='450px',
        margin='0 auto'
    )

    # Enhanced CSS with music-themed styling
    display(HTML(f"""
    <style>
    /* Color variables for consistency */
    :root {{
        --primary: {colors['primary']};
        --secondary: {colors['secondary']};
        --accent: {colors['accent']};
        --light: {colors['light']};
        --dark: {colors['dark']};
        --success: {colors['success']};
        --warning: {colors['warning']};
        --error: {colors['error']};
        --info: {colors['info']};
    }}

    /* Music-themed font (if Google Fonts are accessible) */
    @import url('https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;500;600&display=swap');

    /* Global styles */
    .widget-button, .widget-text, .widget-password,
    .widget-dropdown, .widget-label, .widget-html,
    .jupyter-widgets, .widget-box {{
        font-family: 'Poppins', sans-serif;
        transition: all 0.3s ease-in-out;
    }}

    /* Improved Button Styling */
    .widget-button {{
        border-radius: 8px !important;
        font-weight: 500 !important;
        box-shadow: 0 2px 4px rgba(0,0,0,0.1) !important;
        padding: 5px 15px !important;
    }}

    /* Primary button (purple) */
    .jupyter-button.mod-primary {{
        background-color: var(--primary) !important;
        border-color: var(--primary) !important;
    }}

    /* Success button (green) */
    .jupyter-button.mod-success {{
        background-color: var(--success) !important;
        border-color: var(--success) !important;
    }}

    /* Info button (blue) */
    .jupyter-button.mod-info {{
        background-color: var(--info) !important;
        border-color: var(--info) !important;
    }}

    /* Danger button (red) */
    .jupyter-button.mod-danger {{
        background-color: var(--error) !important;
        border-color: var(--error) !important;
    }}

    /* Button hover effects */
    .widget-button:hover {{
        transform: translateY(-2px);
        box-shadow: 0 4px 8px rgba(0,0,0,0.15) !important;
        filter: brightness(110%);
    }}

    /* Button active/pressed effects */
    .widget-button:active {{
        transform: translateY(0);
        box-shadow: 0 1px 2px rgba(0,0,0,0.1) !important;
    }}

    /* Improved form fields */
    .widget-text input, .widget-textarea textarea, .widget-password input {{
        border-radius: 6px !important;
        border: 1px solid #ddd !important;
        padding: 8px 12px !important;
        box-shadow: inset 0 1px 3px rgba(0,0,0,0.05) !important;
        transition: border-color 0.3s, box-shadow 0.3s;
    }}

    .widget-text input:focus, .widget-textarea textarea:focus, .widget-password input:focus {{
        border-color: var(--primary) !important;
        box-shadow: 0 0 0 3px rgba(120, 81, 169, 0.15) !important;
        outline: none !important;
    }}

    /* Label styling */
    .widget-label {{
        color: var(--secondary) !important;
        font-weight: 500 !important;
    }}

    /* Dropdown styling */
    .widget-dropdown > select {{
        border-radius: 6px !important;
        border: 1px solid #ddd !important;
        padding: 8px 12px !important;
        background-color: white !important;
        transition: border-color 0.3s;
    }}

    .widget-dropdown > select:focus {{
        border-color: var(--primary) !important;
        box-shadow: 0 0 0 3px rgba(120, 81, 169, 0.15) !important;
        outline: none !important;
    }}

    /* Toggle buttons */
    .widget-toggle-buttons {{
        border-radius: 8px;
        overflow: hidden;
        box-shadow: 0 2px 4px rgba(0,0,0,0.08);
    }}

    .widget-toggle-buttons button {{
        box-shadow: none !important;
        border-radius: 0 !important;
        font-weight: 500 !important;
        padding: 8px 16px !important;
    }}

    /* Mode description box */
    div[style*='background:#f8f8f8'] {{
        background: {colors['light']} !important;
        color: {colors['dark']} !important;
        border-radius: 10px !important;
        border-left: 4px solid var(--primary) !important;
        box-shadow: 0 2px 8px rgba(0,0,0,0.06) !important;
    }}

    /* Status message styling */
    div[style*='background-color:#ff990015'] {{
        background-color: rgba(255, 181, 73, 0.15) !important;
        border-radius: 8px !important;
    }}
    div[style*='background-color:#00aa0015'] {{
        background-color: rgba(0, 192, 127, 0.15) !important;
        border-radius: 8px !important;
    }}
    div[style*='background-color:#cc000015'] {{
        background-color: rgba(255, 82, 82, 0.15) !important;
        border-radius: 8px !important;
    }}
    div[style*='background-color:#ff660015'] {{
        background-color: rgba(255, 102, 0, 0.15) !important;
        border-radius: 8px !important;
    }}

    /* Music-themed icons using emoji as placeholders */
    .music-icon:before {{
        content: '🎵';
        font-size: 1.2em;
        margin-right: 5px;
    }}

    .user-icon:before {{
        content: '👤';
        font-size: 1.2em;
        margin-right: 5px;
    }}

    /* Card styling for recommendation items */
    .recommendation-card {{
        border-radius: 12px;
        border: 1px solid #eee;
        padding: 15px;
        margin: 10px 0;
        background: white;
        box-shadow: 0 3px 10px rgba(0,0,0,0.05);
        transition: all 0.3s ease;
    }}

    .recommendation-card:hover {{
        transform: translateY(-3px);
        box-shadow: 0 6px 15px rgba(0,0,0,0.1);
    }}

    /* Fade animation for page transitions with more smoothness */
    @keyframes fadeIn {{
        from {{ opacity: 0; transform: translateY(15px); }}
        to {{ opacity: 1; transform: translateY(0); }}
    }}

    .fade-in {{
        animation: fadeIn 0.5s ease-out;
    }}

    /* Remove outline focus on widgets */
    .widget-box:focus {{
        outline: none !important;
    }}

    /* Hide scrollbars everywhere - more aggressive approach */
    * {{
        -ms-overflow-style: none !important;
        scrollbar-width: none !important;
    }}

    /* Additional scrollbar hiding for specific containers */
    .widget-box, .widget-vbox, .widget-hbox,
    .jupyter-widgets, .widget-container,
    .p-Widget, .p-Panel, .lm-Widget, .lm-Panel {{
        overflow: hidden !important;
        scrollbar-width: none !important;
    }}

    /* Force all Jupyter output areas to hide scrollbars */
    .jp-OutputArea, .jp-OutputArea-output, .jp-OutputArea-child {{
        overflow: hidden !important;
        max-height: none !important;
    }}

    /* Force the main output widget to expand as needed without scrollbars */
    .output_scroll {{
        height: auto !important;
        max-height: none !important;
        overflow-y: hidden !important;
    }}

    /* App container with subtle gradient background */
    .app-container {{
        background: linear-gradient(135deg, {colors['light']} 0%, #ffffff 100%);
        padding: 20px;
        border-radius: 15px;
        box-shadow: 0 8px 30px rgba(0,0,0,0.08);
    }}
    </style>
    """))

    # Navigation history
    nav_history = []

    def show_status(message, status='info'):
        colors_map = {
            'info': colors['info'],
            'success': colors['success'],
            'error': colors['error'],
            'warning': colors['warning']
        }
        icons = {'info': 'ℹ️', 'success': '✅', 'error': '❌', 'warning': '⚠️'}
        with output:
            clear_output()
            if message:
                display(HTML(f"""
                <div style='padding:12px; border-radius:10px; background-color:{colors_map[status]}15;
                     border-left:4px solid {colors_map[status]}; margin:15px auto; max-width:500px; text-align:center;
                     box-shadow: 0 2px 8px rgba(0,0,0,0.05);'>
                    <span style='color:{colors_map[status]}; font-weight:500'>{icons[status]} {message}</span>
                </div>
                """))

    def show_loading(message="Loading..."):
        with output:
            clear_output()
            display(HTML(f"""
            <div style='text-align:center; padding:15px;'>
                <div style='font-weight:500; margin-bottom:15px; color:{colors['secondary']}'>{message}</div>
                <div style='display:inline-block; width:40px; height:40px; border:5px solid #f3f3f3;
                     border-top:5px solid {colors['primary']}; border-radius:50%; animation:spin 1s linear infinite;'></div>
                <style>@keyframes spin {{0% {{ transform: rotate(0deg); }} 100% {{ transform: rotate(360deg); }}}}</style>
            </div>
            """))

    def switch_to_page(page, save_history=True):
        if save_history and session['current_page'] != page:
            nav_history.append(session['current_page'])
        session['current_page'] = page
        show_status("")

        # Add fade transition between pages
        display(HTML("<div class='fade-in'>"))

        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

        display(HTML("</div>"))

    def go_back():
        if nav_history:
            previous_page = nav_history.pop()
            switch_to_page(previous_page, save_history=False)
        else:
            switch_to_page('start', save_history=False)

    def show_start_page():
        clear_output()
        # Add app icon and brand identity
        app_logo = widgets.HTML(f"""
            <div style='text-align:center; margin-bottom:15px;'>
                <div style='font-size:48px; line-height:1.2;'>🎵</div>
            </div>
        """)

        start_label = widgets.HTML(f"<h1 style='{main_title_style}'>Harmony Hub</h1>")
        tagline = widgets.HTML(
            f"<p style='margin-bottom:25px; text-align:center; color:{colors['secondary']}; font-size:16px;'>"
            "Your personal music companion</p>"
        )

        # Styled buttons with icons
        signin_button = widgets.Button(
            description='Sign In',
            button_style='primary',
            icon='sign-in',
            layout=button_layout
        )
        signup_button = widgets.Button(
            description='Create Account',
            icon='user-plus',
            layout=button_layout
        )

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        # Add a decorative element
        footer = widgets.HTML(f"""
            <div style='margin-top:40px; text-align:center; color:{colors['secondary']}; opacity:0.7; font-size:12px;'>
                <p>Discover your perfect soundtrack</p>
            </div>
        """)

        # Wrap everything in a container with background styling
        container = widgets.HTML("""<div class="app-container">""")
        end_container = widgets.HTML("""</div>""")

        display(widgets.VBox([
            container,
            app_logo,
            start_label,
            tagline,
            widgets.HBox([signin_button, signup_button], layout=widgets.Layout(justify_content='center')),
            footer,
            end_container,
            output
        ], layout=container_layout))

    def show_signin_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>Welcome Back</h1>")
        subtitle = widgets.HTML(f"<p style='text-align:center; color:{colors['secondary']}; margin-bottom:20px;'>Sign in to your Harmony Hub account</p>")

        # Consistent form field styling
        form_style = {'description_width': '80px'}

        si_username = widgets.Text(
            description='Username',
            style=form_style,
            placeholder='Enter your username'
        )
        si_password = widgets.Password(
            description='Password',
            style=form_style,
            placeholder='Enter your password'
        )

        # Enhanced buttons with icons
        si_button = widgets.Button(
            description='Sign In',
            button_style='primary',
            icon='sign-in',
            layout=button_layout
        )
        back_button = widgets.Button(
            description='Back',
            icon='arrow-left',
            layout=button_layout
        )

        def sign_in_action(b):
            show_loading("Signing in...")

            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: go_back())

        buttons_layout = widgets.Layout(justify_content='center', margin='20px 0')

        # Create a container with background styling
        container = widgets.HTML("""<div class="app-container">""")
        end_container = widgets.HTML("""</div>""")

        # Create a simplified container without nested layouts
        signin_form = widgets.VBox([
            si_username,
            si_password
        ], layout=form_layout)

        buttons = widgets.HBox([
            si_button,
            back_button
        ], layout=buttons_layout)

        # Use minimal nesting to avoid scroll issues
        main_container = widgets.VBox([
            container,
            label,
            subtitle,
            signin_form,
            buttons,
            end_container,
            output
        ], layout=container_layout)

        # Direct display without further nesting
        display(main_container)

    def show_signup_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>Join Harmony Hub</h1>")
        subtitle = widgets.HTML(f"<p style='text-align:center; color:{colors['secondary']}; margin-bottom:20px;'>Create your account to discover personalized music</p>")

        # Create form fields with consistent styling
        form_style = {'description_width': '100px'}

        su_username = widgets.Text(description='Username', style=form_style, placeholder='Choose a username')
        su_password = widgets.Password(description='Password', style=form_style, placeholder='Create a password')
        su_firstname = widgets.Text(description='First Name', style=form_style, placeholder='Your first name')
        su_lastname = widgets.Text(description='Last Name', style=form_style, placeholder='Your last name')
        su_age = widgets.IntText(description='Age', style=form_style, placeholder='Your age')
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender', style=form_style)

        selected_artists = []

        artist_header = widgets.HTML(f"<h2 style='{subtitle_style}'>Select Your Favorite Artists</h2>")
        artist_subheader = widgets.HTML(f"<p style='text-align:center; color:{colors['secondary']}; margin-bottom:15px;'>This helps us recommend music you'll love</p>")

        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(
            description='Add',
            icon='plus',
            button_style='info',
            layout=widgets.Layout(width='80px')
        )

        # Use fixed height with visible overflow only for the artist box
        artist_box = widgets.VBox(layout=widgets.Layout(
            width='300px',
            margin='10px auto',
            # No max-height to prevent scroll issues
        ))

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(
                    description='Remove',
                    icon='trash',
                    button_style='danger',
                    layout=widgets.Layout(width='80px')
                )
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn], layout=widgets.Layout(justify_content='space-between', width='100%')))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)

        su_button = widgets.Button(
            description='Create Account',
            button_style='primary',
            icon='check',
            layout=button_layout
        )
        back_button = widgets.Button(
            description='Back',
            icon='arrow-left',
            layout=button_layout
        )

        def sign_up_action(b):
            show_loading("Creating your account...")

            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            if len(su_password.value) < 4:
                show_status("Password should be at least 4 characters long.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: go_back())

        artist_input_container = widgets.HBox([artist_input, add_button],
                                             layout=widgets.Layout(width='400px', margin='0 auto', justify_content='space-between'))

        buttons_layout = widgets.Layout(justify_content='center', margin='20px 0')

        # Create a container with background styling
        container = widgets.HTML("""<div class="app-container">""")
        end_container = widgets.HTML("""</div>""")

        # Group form fields to reduce nesting
        user_form = widgets.VBox([
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender
        ], layout=form_layout)

        buttons = widgets.HBox([
            su_button, back_button
        ], layout=buttons_layout)

        # Simplified container structure
        main_container = widgets.VBox([
            container,
            label,
            subtitle,
            user_form,
            artist_header,
            artist_subheader,
            artist_input_container,
            artist_box,
            buttons,
            end_container,
            output
        ], layout=container_layout)

        display(main_container)

    def show_edit_favorites_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>Your Favorite Artists</h1>")
        subtitle = widgets.HTML(f"<p style='text-align:center; color:{colors['secondary']}; margin-bottom:20px;'>Edit your preferences to improve recommendations</p>")

        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        form_style = {'description_width': '100px'}
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(
            description='Add',
            icon='plus',
            button_style='info',
            layout=widgets.Layout(width='80px')
        )

        # Remove fixed height and scroll settings
        artist_box = widgets.VBox(layout=widgets.Layout(
            width='400px',
            margin='10px auto'
        ))

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                # Create artist card with rounded corners
                artist_card = widgets.HTML(f"""
                <div style="display:flex; justify-content:space-between; align-items:center;
                            padding:10px; margin:5px 0; background-color:{colors['light']};
                            border-radius:8px; border-left:3px solid {colors['primary']}">
                    <span style="font-weight:500">{a}</span>
                </div>
                """)

                remove_btn = widgets.Button(
                    description='',
                    icon='trash',
                    button_style='danger',
                    layout=widgets.Layout(width='40px', height='40px')
                )
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([artist_card, remove_btn], layout=widgets.Layout(justify_content='space-between', width='100%')))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)
        refresh_artist_table()

        confirm_button = widgets.Button(
            description='Save Changes',
            button_style='primary',
            icon='check',
            layout=button_layout
        )
        back_button = widgets.Button(
            description='Cancel',
            icon='times',
            layout=button_layout
        )

        #def save_
        def save_changes(b):
            show_loading("Saving your preferences...")
            users_df = pd.read_csv(users_csv)
            idx = users_df.index[users_df['user_id'] == session['user_id']].tolist()[0]
            users_df.at[idx, 'favorite_artists'] = ', '.join(selected_artists)
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(session['user_id'], selected_artists)
            show_status("Your favorite artists have been updated!", 'success')
            time.sleep(1)
            switch_to_page('session')

        confirm_button.on_click(save_changes)
        back_button.on_click(lambda b: switch_to_page('session'))

        artist_input_container = widgets.HBox([artist_input, add_button],
                                             layout=widgets.Layout(width='400px', margin='0 auto', justify_content='space-between'))

        buttons = widgets.HBox([
            confirm_button, back_button
        ], layout=widgets.Layout(justify_content='center', margin='20px 0'))

        # Create a container with background styling
        container = widgets.HTML("""<div class="app-container">""")
        end_container = widgets.HTML("""</div>""")

        main_container = widgets.VBox([
            container,
            label,
            subtitle,
            artist_input_container,
            artist_box,
            buttons,
            end_container,
            output
        ], layout=container_layout)

        display(main_container)

    def show_session_page():
        clear_output()

        # Fetch user data
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']].iloc[0]

        # Get user's favorite artists
        fav_str = user_row.get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        fav_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        # Welcome message with user's name
        first_name = user_row.get('first_name', '')
        if pd.isna(first_name) or first_name == '':
            welcome_message = f"Welcome, {session['username']}!"
        else:
            welcome_message = f"Welcome, {first_name}!"

        welcome = widgets.HTML(f"<h1 style='{main_title_style}'>{welcome_message}</h1>")

        # Add an avatar or decorative element
        avatar = widgets.HTML(f"""
            <div style='text-align:center; margin-bottom:10px;'>
                <div style='display:inline-block; background-color:{colors['secondary']}; color:{colors['light']};
                     width:70px; height:70px; line-height:70px; border-radius:50%; font-size:24px;'>
                    {session['username'][0].upper()}
                </div>
            </div>
        """)

        # Mode selection with toggle buttons
        mode_label = widgets.HTML(f"<h2 style='{subtitle_style}'>How would you like recommendations today?</h2>")

        mode_buttons = widgets.ToggleButtons(
            options=['Regular', 'Emotion-Based'],
            description='Mode:',
            style={'description_width': 'initial', 'button_width': '150px'},
            button_style='info'
        )

        # Mode description box with dynamic content
        mode_description = widgets.HTML(f"""
        <div id="mode-info" style='background:{colors['light']}; color:{colors['dark']}; padding:15px;
             margin:10px 0; border-radius:8px; text-align:center;'>
            <b>Regular Mode:</b> Get recommendations based on your favorite artists and listening history
        </div>
        """)

        def update_mode_description(change):
            if change['new'] == 'Regular':
                mode_description.value = f"""
                <div id="mode-info" style='background:{colors['light']}; color:{colors['dark']}; padding:15px;
                     margin:10px 0; border-radius:8px; text-align:center;'>
                    <b>Regular Mode:</b> Get recommendations based on your favorite artists and listening history
                </div>
                """
            else:
                mode_description.value = f"""
                <div id="mode-info" style='background:{colors['light']}; color:{colors['dark']}; padding:15px;
                     margin:10px 0; border-radius:8px; text-align:center;'>
                    <b>Emotion-Based Mode:</b> Get recommendations tailored to your current mood
                </div>
                """

        mode_buttons.observe(update_mode_description, names='value')

        # Regular mode container
        regular_container = widgets.VBox([
            widgets.HTML(f"<h3 style='{subtitle_style}'>Get New Recommendations</h3>"),
            widgets.Button(
                description='Generate Recommendations',
                button_style='primary',
                icon='music',
                layout=button_layout
            )
        ])

        # Emotion mode container
        mood_options = ['Happy', 'Sad', 'Energetic', 'Calm', 'Anxious', 'Nostalgic', 'Romantic', 'Focused']
        emotion_dropdown = widgets.Dropdown(
            options=mood_options,
            description='Your mood:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='250px', margin='0 auto')
        )
        emotion_container = widgets.VBox([
            widgets.HTML(f"<h3 style='{subtitle_style}'>How are you feeling today?</h3>"),
            emotion_dropdown,
            widgets.Button(
                description='Get Mood Music',
                button_style='primary',
                icon='smile-o',
                layout=button_layout
            )
        ])

        # Initially hide both containers
        regular_container.layout.display = 'none'
        emotion_container.layout.display = 'none'

        # Show the appropriate container when the mode changes
        def toggle_mode_containers(change):
            if change['new'] == 'Regular':
                regular_container.layout.display = 'flex'
                emotion_container.layout.display = 'none'
                session['mode'] = 'regular'
            else:
                regular_container.layout.display = 'none'
                emotion_container.layout.display = 'flex'
                session['mode'] = 'emotion'

        mode_buttons.observe(toggle_mode_containers, names='value')

        # Initially set the regular mode
        mode_buttons.value = 'Regular'
        session['mode'] = 'regular'

        # Results container
        results_container = widgets.Output()

        # Function to generate recommendations
        def get_regular_recommendations(b):
            with results_container:
                clear_output()
                show_loading("Finding perfect songs for you...")

                try:
                    recommendations = recommender.get_recommendations(session['user_id'])

                    if not recommendations.empty:
                        display(HTML(f"<h3 style='{subtitle_style}'>Your Recommendations</h3>"))

                        for _, row in recommendations.iterrows():
                            # Create a card for each recommendation
                            card_html = f"""
                            <div class="recommendation-card">
                                <div style="display:flex; justify-content:space-between; align-items:center;">
                                    <div>
                                        <div style="font-weight:bold; color:{colors['primary']}; font-size:18px;">
                                            {row['title']}
                                        </div>
                                        <div style="color:{colors['secondary']}; margin:5px 0;">
                                            by {row['artist']}
                                        </div>
                                        <div style="font-size:14px; color:{colors['dark']}; opacity:0.8;">
                                            {row['genre']} • {row['year']}
                                        </div>
                                    </div>
                                    <div>
                                        <button class="jupyter-button mod-info" style="border-radius:5px; margin:0;">
                                            <i class="fa fa-play"></i> Play
                                        </button>
                                    </div>
                                </div>
                            </div>
                            """
                            display(HTML(card_html))

                            # Log the recommendation
                            logger.log_recommendation(session['user_id'], row['song_id'])
                    else:
                        display(HTML(f"""
                        <div style="text-align:center; padding:20px; color:{colors['secondary']};">
                            <p>No recommendations found. Try updating your favorite artists first.</p>
                        </div>
                        """))

                except Exception as e:
                    display(HTML(f"""
                    <div style="text-align:center; padding:20px; color:{colors['error']};">
                        <p>Error generating recommendations: {str(e)}</p>
                    </div>
                    """))

        # Function to get emotion-based recommendations
        def get_emotion_recommendations(b):
            with results_container:
                clear_output()
                mood = emotion_dropdown.value
                show_loading(f"Finding {mood.lower()} music for you...")

                try:
                    recommendations = emotion_service.get_recommendations_by_emotion(
                        mood.lower(), session['user_id']
                    )

                    if not recommendations.empty:
                        display(HTML(f"<h3 style='{subtitle_style}'>Your {mood} Playlist</h3>"))

                        for _, row in recommendations.iterrows():
                            # Create a card for each recommendation
                            card_html = f"""
                            <div class="recommendation-card">
                                <div style="display:flex; justify-content:space-between; align-items:center;">
                                    <div>
                                        <div style="font-weight:bold; color:{colors['primary']}; font-size:18px;">
                                            {row['title']}
                                        </div>
                                        <div style="color:{colors['secondary']}; margin:5px 0;">
                                            by {row['artist']}
                                        </div>
                                        <div style="font-size:14px; color:{colors['dark']}; opacity:0.8;">
                                            {row['emotion_tags']}
                                        </div>
                                    </div>
                                    <div>
                                        <button class="jupyter-button mod-info" style="border-radius:5px; margin:0;">
                                            <i class="fa fa-play"></i> Play
                                        </button>
                                    </div>
                                </div>
                            </div>
                            """
                            display(HTML(card_html))

                            # Log the recommendation
                            logger.log_recommendation(session['user_id'], row['song_id'])
                    else:
                        display(HTML(f"""
                        <div style="text-align:center; padding:20px; color:{colors['secondary']};">
                            <p>No {mood.lower()} songs found. Try a different mood.</p>
                        </div>
                        """))

                except Exception as e:
                    display(HTML(f"""
                    <div style="text-align:center; padding:20px; color:{colors['error']};">
                        <p>Error generating recommendations: {str(e)}</p>
                    </div>
                    """))

        # Connect recommendation buttons to their functions
        regular_container.children[1].on_click(get_regular_recommendations)
        emotion_container.children[2].on_click(get_emotion_recommendations)

        # Buttons for user actions
        edit_favorites_button = widgets.Button(
            description='Edit Favorites',
            button_style='info',
            icon='edit',
            layout=button_layout
        )
        sign_out_button = widgets.Button(
            description='Sign Out',
            button_style='danger',
            icon='sign-out',
            layout=button_layout
        )

        edit_favorites_button.on_click(lambda b: switch_to_page('edit_favorites'))
        sign_out_button.on_click(lambda b: (session.update({'user_id': None, 'username': '', 'mode': 'none'}), switch_to_page('start')))

        user_actions = widgets.HBox([
            edit_favorites_button, sign_out_button
        ], layout=widgets.Layout(justify_content='center', margin='20px 0'))

        # Create a container with background styling
        container = widgets.HTML("""<div class="app-container">""")
        end_container = widgets.HTML("""</div>""")

        # Display user's favorite artists as badges
        fav_artists_html = '<div style="text-align:center; margin:10px 0;">'
        if fav_artists:
            fav_artists_html += '<div style="margin-bottom:10px; color:#666;">Your favorite artists:</div>'
            for artist in fav_artists:
                fav_artists_html += f"""
                <span style="display:inline-block; background:{colors['primary']}15; color:{colors['primary']};
                      padding:5px 12px; margin:5px; border-radius:20px; font-size:12px; font-weight:500;">
                    {artist}
                </span>
                """
        else:
            fav_artists_html += '<div style="margin-bottom:10px; color:#666;">No favorite artists yet. Add some!</div>'
        fav_artists_html += '</div>'

        favorite_artists_display = widgets.HTML(fav_artists_html)

        # Layout the session page
        main_container = widgets.VBox([
            container,
            avatar,
            welcome,
            favorite_artists_display,
            mode_label,
            mode_buttons,
            mode_description,
            regular_container,
            emotion_container,
            results_container,
            user_actions,
            end_container,
            output
        ], layout=container_layout)

        display(main_container)

    # Start the application with the entry page
    switch_to_page('start')

    return music_recommender_output

# Run the application
start_music_recommender()

# Drafts

In [ ]:
#%%writefile "/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/main.py"
import ipywidgets as widgets
from google.colab import drive
drive.mount('/content/drive')
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService  # ✅ import the new class

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager  # ✅ import the new class

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
project_dir = '/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/'
data_dir = project_dir + 'data/'
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################
def start_music_recommender():
    print("DEBUG: Starting enhanced multi-page recommender with improved UI/UX")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    # Define common styles
    main_title_style = "font-size:24px; font-weight:bold; margin-bottom:15px; text-align:center; color:black;"
    subtitle_style = "font-size:18px; font-weight:bold; margin-top:15px; margin-bottom:10px; text-align:center; color:black;"

    # Improved button styles with transitions
    button_layout = widgets.Layout(
        margin="8px",
        width="auto",
        min_width="120px"
    )

    # Common container layout - consistently centered with fixed width
    container_layout = widgets.Layout(
        display="flex",
        flex_flow="column",
        align_items="center",
        width="600px",
        max_width="100%",
        margin="0 auto",
        overflow="hidden",  # Changed from hidden to visible
        padding="20px",
        border="none"
    )

    # Define CSS for transitions and smooth animations
    display(HTML("""
    <style>
    /* Smooth transitions for all elements */
    .widget-button, .widget-text, .widget-password,
    .widget-dropdown, .widget-label, .widget-html {
        transition: all 0.3s ease-in-out;
    }

    /* Improved Button Hover Effects */
    .widget-button:hover {
        transform: translateY(-2px);
        box-shadow: 0 4px 8px rgba(0,0,0,0.1);
    }

    /* Remove outline focus on widgets */
    .widget-box:focus {
        outline: none !important;
    }

    /* Comprehensive scrollbar hiding */
    ::-webkit-scrollbar {
        width: 0px !important;
        height: 0px !important;
        background: white !important;
        display: none !important;
    }

    /* Hide scrollbars everywhere - more aggressive approach */
    * {
        -ms-overflow-style: none !important;  /* IE and Edge */
        scrollbar-width: none !important;  /* Firefox */
    }

    /* Additional scrollbar hiding for specific containers */
    .widget-box, .widget-vbox, .widget-hbox,
    .jupyter-widgets, .widget-container,
    .p-Widget, .p-Panel, .lm-Widget, .lm-Panel {
        overflow: hidden !important;
        scrollbar-width: none !important;
    }

    /* Force all Jupyter output areas to hide scrollbars */
    .jp-OutputArea, .jp-OutputArea-output, .jp-OutputArea-child {
        overflow: hidden !important;
        max-height: none !important;
    }

    /* Force the main output widget to expand as needed without scrollbars */
    .output_scroll {
        height: auto !important;
        max-height: none !important;
        overflow-y: hidden !important;
    }

    /* Fade animation for page transitions */
    @keyframes fadeIn {
        from { opacity: 0; transform: translateY(10px); }
        to { opacity: 1; transform: translateY(0); }
    }

    .fade-in {
        animation: fadeIn 0.4s ease-in-out;
    }

    /* Ensure proper sizing for input elements */
    .widget-text, .widget-textarea, .widget-password {
        box-sizing: border-box !important;
        width: 100% !important;
    }
    /* Dark mode compatibility - ensure text is visible */
    .widget-text input, .widget-textarea textarea, .widget-password input,
    .widget-text, .widget-textarea, .widget-password,
    .widget-label, .widget-html, .widget-dropdown {
          color: #333 !important;
          background-color: #fff !important;
    }
    /* Ensure form labels are visible in dark mode */
    .widget-label {
      color: white !important;
      background-color: transparent !important;
    }
    /* Fix input backgrounds in dark mode */
    .widget-text .widget-text-input,
    .widget-textarea .widget-textarea-input,
    .widget-password .widget-password-input {
          background-color: #fff !important;
          color: #333 !important;
          border: 1px solid #ccc !important;
    }
    /* Make HTML element content visible in dark mode */
    .jupyter-widgets.widget-html-content {
          color: white !important;
    }
    /* Fix the title styling for dark mode */
    h1[style*='color:#3366cc'], h2[style*='color:#333'] {
          color: white !important;
    }
    /* Style the container backgrounds in dark mode */
    .widget-container {
          background-color: transparent !important;
    }
    /* Fix dropdown styling for dark mode */
    .widget-dropdown > select {
          background-color: white !important;
          color: #333 !important;
    }
    /* Fix for mode description box */
    div[style*='background:#f8f8f8'] {
          background: #444 !important;
          color: white !important;
    }
    div[style*='background:#f8f8f8'] p {
          color: white !important;
    }
    /* Status message styling for dark mode */
    div[style*='background-color:#ff990015'] {
          background-color: rgba(255, 153, 0, 0.2) !important;
    }
    div[style*='background-color:#00aa0015'] {
          background-color: rgba(0, 170, 0, 0.2) !important;
    }
    div[style*='background-color:#cc000015'] {
          background-color: rgba(204, 0, 0, 0.2) !important;
    }
    div[style*='background-color:#ff660015'] {
          background-color: rgba(255, 102, 0, 0.2) !important;
    }
    /* Make sure HTML-based content is visible */
    p[style*='text-align:center'], p[style*='margin-bottom'] {
          color: white !important;
    }
    </style>
    """))

    # Navigation history
    nav_history = []

    def show_status(message, status='info'):
        colors = {'info': '#ff9900', 'success': '#00aa00', 'error': '#cc0000', 'warning': '#ff6600'}
        icons = {'info': 'ℹ️', 'success': '✅', 'error': '❌', 'warning': '⚠️'}
        with output:
            clear_output()
            if message:
                display(HTML(f"""
                <div style='padding:10px; border-radius:8px; background-color:{colors[status]}15;
                     border-left:4px solid {colors[status]}; margin:15px auto; max-width:500px; text-align:center;'>
                    <span style='color:{colors[status]}; font-weight:bold'>{icons[status]} {message}</span>
                </div>
                """))

    def show_loading(message="Loading..."):
        with output:
            clear_output()
            display(HTML(f"""
            <div style='text-align:center; padding:15px;'>
                <div style='font-weight:bold; margin-bottom:15px;'>{message}</div>
                <div style='display:inline-block; width:40px; height:40px; border:5px solid #f3f3f3;
                     border-top:5px solid #3498db; border-radius:50%; animation:spin 1s linear infinite;'></div>
                <style>@keyframes spin {{0% {{ transform: rotate(0deg); }} 100% {{ transform: rotate(360deg); }}}}</style>
            </div>
            """))

    def switch_to_page(page, save_history=True):
        if save_history and session['current_page'] != page:
            nav_history.append(session['current_page'])
        session['current_page'] = page
        show_status("")

        # Add fade transition between pages
        display(HTML("<div class='fade-in'>"))

        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

        display(HTML("</div>"))

    def go_back():
        if nav_history:
            previous_page = nav_history.pop()
            switch_to_page(previous_page, save_history=False)
        else:
            switch_to_page('start', save_history=False)

    def show_start_page():
        clear_output()
        start_label = widgets.HTML(f"<h1 style='{main_title_style}'>🎵 Music Recommender</h1>")
        welcome_text = widgets.HTML("<p style='margin-bottom:20px; text-align:center;'>Welcome to your personal music recommendation system.</p>")
        signin_button = widgets.Button(description='Sign In', button_style='primary', layout=button_layout)
        signup_button = widgets.Button(description='Sign Up', layout=button_layout)

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            welcome_text,
            widgets.HBox([signin_button, signup_button], layout=widgets.Layout(justify_content='center')),
            output
        ], layout=container_layout))

    def show_signin_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🔑 Sign In</h1>")

        # Consistent form field styling
        form_style = {'description_width': '80px'}
        form_layout = widgets.Layout(width='300px', margin='0 auto')

        si_username = widgets.Text(description='Username', style=form_style)
        si_password = widgets.Password(description='Password', style=form_style)
        si_button = widgets.Button(description='Sign In', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Back', layout=button_layout)

        def sign_in_action(b):
            show_loading("Signing in...")

            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: go_back())

        buttons_layout = widgets.Layout(justify_content='center', margin='15px 0')

        # Create a simplified container without nested layouts
        signin_form = widgets.VBox([
            si_username,
            si_password
        ], layout=form_layout)

        buttons = widgets.HBox([
            si_button,
            back_button
        ], layout=buttons_layout)

        # Use minimal nesting to avoid scroll issues
        main_container = widgets.VBox([
            label,
            signin_form,
            buttons,
            output
        ], layout=container_layout)

        # Direct display without further nesting
        display(main_container)

    def show_signup_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🔐 Sign Up</h1>")

        # Create form fields with consistent styling
        form_style = {'description_width': '100px'}
        form_layout = widgets.Layout(width='300px', margin='0 auto')

        su_username = widgets.Text(description='Username', style=form_style)
        su_password = widgets.Password(description='Password', style=form_style)
        su_firstname = widgets.Text(description='First Name', style=form_style)
        su_lastname = widgets.Text(description='Last Name', style=form_style)
        su_age = widgets.IntText(description='Age', style=form_style)
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender', style=form_style)

        selected_artists = []

        artist_header = widgets.HTML(f"<h2 style='{subtitle_style}'>🎵 Select Favorite Artists</h2>")
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(description='Add', layout=widgets.Layout(width='80px'))

        # Use fixed height with visible overflow only for the artist box
        artist_box = widgets.VBox(layout=widgets.Layout(
            width='400px',
            margin='10px auto',
            # No max-height to prevent scroll issues
        ))

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(
                    description='Remove',
                    button_style='danger',
                    layout=widgets.Layout(width='80px')
                )
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn], layout=widgets.Layout(justify_content='space-between', width='100%')))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)

        su_button = widgets.Button(description='Create Account', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Back', layout=button_layout)

        def sign_up_action(b):
            show_loading("Creating account...")

            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            if len(su_password.value) < 4:
                show_status("Password should be at least 4 characters long.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: go_back())

        artist_input_container = widgets.HBox([artist_input, add_button],
                                             layout=widgets.Layout(width='400px', margin='0 auto', justify_content='space-between'))

        buttons_layout = widgets.Layout(justify_content='center', margin='15px 0')

        # Group form fields to reduce nesting
        user_form = widgets.VBox([
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender
        ], layout=form_layout)

        buttons = widgets.HBox([
            su_button, back_button
        ], layout=buttons_layout)

        # Simplified container structure
        main_container = widgets.VBox([
            label,
            user_form,
            artist_header,
            artist_input_container,
            artist_box,
            buttons,
            output
        ], layout=container_layout)

        display(main_container)

    def show_edit_favorites_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🎨 Edit Favorite Artists</h1>")
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        form_style = {'description_width': '100px'}
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(description='Add', layout=widgets.Layout(width='80px'))

        # Remove fixed height and scroll settings
        artist_box = widgets.VBox(layout=widgets.Layout(
            width='400px',
            margin='10px auto'
        ))

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(
                    description='Remove',
                    button_style='danger',
                    layout=widgets.Layout(width='80px')
                )
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn], layout=widgets.Layout(justify_content='space-between', width='100%')))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)
        refresh_artist_table()

        confirm_button = widgets.Button(description='Save Changes', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Cancel', layout=button_layout)

        def save_favorites(b):
            show_loading("Saving your preferences...")

            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists)
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(session['user_id'], selected_artists)
            show_status("Your favorite artists have been updated successfully!", 'success')
            switch_to_page('session')

        confirm_button.on_click(save_favorites)
        back_button.on_click(lambda b: switch_to_page('session'))

        artist_input_container = widgets.HBox([artist_input, add_button],
                                             layout=widgets.Layout(width='400px', margin='0 auto', justify_content='space-between'))

        artist_header = widgets.HTML(f"<p style='{subtitle_style}'>Your Current Favorite Artists:</p>")

        buttons_layout = widgets.Layout(justify_content='center', margin='15px 0')
        buttons = widgets.HBox([confirm_button, back_button], layout=buttons_layout)

        # Simplified structure
        main_container = widgets.VBox([
            label,
            artist_input_container,
            artist_header,
            artist_box,
            buttons,
            output
        ], layout=container_layout)

        display(main_container)

    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<h1 style='{main_title_style}'>🎵 Music Recommender</h1>")
        user_info = widgets.HTML(f"<p style='text-align:center; margin-bottom:15px;'><b>👤 Signed in as:</b> {session['username']}</p>")

        # Improved mode selector with better spacing
        mode_selector = widgets.ToggleButtons(
            options=[
                ('Private Mode', 'none'),
                ('Log My Activity', 'log'),
                ('Recommend Me Music', 'recommend')
            ],
            value=session['mode'],
            description='Mode:',
            button_style='info',
            tooltips=[
                'Private Mode: No logging or recommendations',
                'Log Mode: Track your listening activity',
                'Recommend Mode: Get personalized music recommendations'
            ],
            layout=widgets.Layout(width='auto', margin='0 auto')
        )

        # Create a mapping for mode labels
        mode_labels = {
            'none': 'Private Mode',
            'log': 'Log My Activity',
            'recommend': 'Recommend Me Music'
        }

        mode_description = widgets.HTML(
            f"""<div style='margin:15px auto; padding:12px; background:#f8f8f8; border-radius:8px; width:500px; text-align:center;'>
                <p id='mode-desc' style='margin:0;'>Select a mode to continue.</p>
            </div>"""
        )

        # Action buttons with consistent styling and better layout
        play_button = widgets.Button(
            description='Play Random Song',
            icon='play',
            button_style='success',
            layout=button_layout
        )
        sad_button = widgets.Button(
            description='I Feel Sad 😢',
            button_style='info',
            layout=button_layout
        )
        happy_button = widgets.Button(
            description='I Feel Happy 😊',
            button_style='info',
            layout=button_layout
        )
        fav_edit_button = widgets.Button(
            description='Edit Favorites',
            icon='edit',
            layout=button_layout
        )
        exit_button = widgets.Button(
            description='Log Out',
            icon='sign-out',
            button_style='danger',
            layout=button_layout
        )

        def change_mode(change):
            session['mode'] = change['new']

            # Update mode description based on selection
            mode_descriptions = {
                'none': "Private Mode: Your activity won't be logged or used for recommendations.",
                'log': "Log Mode: Your listening activity will be tracked to improve recommendations.",
                'recommend': "Recommend Mode: Tell us how you're feeling and get personalized recommendations!"
            }
            mode_description.value = f"""<div style='margin:15px auto; padding:12px; background:#f8f8f8; border-radius:8px; width:500px; text-align:center;'>
                <p id='mode-desc' style='margin:0;'>{mode_descriptions[session['mode']]}</p>
            </div>"""

            show_status(f"Mode changed to: {mode_labels[session['mode']]}", 'info')

        def play_song(b):
            show_status("")
            show_loading("Finding a song...")

            if session['mode'] == 'none':
                show_status("Privacy mode → not logging song play.", 'info')
                return

            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)

            show_status(f"Now playing: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")

            if session['mode'] != 'recommend':
                show_status("Not in recommend mode — please switch to 'Recommend Me Music' mode first.", 'warning')
                return

            show_loading("Finding songs to lift your mood...")
            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("No songs found to recommend. Try adding more favorite artists.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]

            # Create a nicer song selection widget
            song_header = widgets.HTML("<h3 style='margin-top:15px; text-align:center;'>🎵 Songs to cheer you up:</h3>")
            song_options = widgets.RadioButtons(
                options=options,
                layout=widgets.Layout(width='auto', margin='10px auto')
            )
            confirm_button = widgets.Button(
                description='Play Selected Song',
                button_style='success',
                layout=widgets.Layout(margin='10px auto', display='flex', justify_content='center')
            )

            def on_confirm_click(_):
                if not song_options.value:
                    show_status("Please select a song first", 'warning')
                    return

                show_loading("Playing your selection...")

                selected_label = song_options.value
                selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(
                    session['user_id'],
                    selected_row['song_id'],
                    mood_before,
                    mood_after,
                    is_recommended=1
                )
                show_status(f"Now playing: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

            confirm_button.on_click(on_confirm_click)

            with music_recommender_output:
                clear_output()
                display(widgets.VBox([
                    song_header,
                    song_options,
                    confirm_button
                ], layout=widgets.Layout(align_items='center', width='500px', margin='0 auto')))

        def feel_happy(b):
            show_status("")
            show_status("Thanks for sharing! We're glad you're feeling happy today. 😊", 'success')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        fav_edit_button.on_click(lambda b: switch_to_page('edit_favorites'))
        exit_button.on_click(exit_session)

        # Initialize mode description
        change_mode({'new': session['mode']})

        # Action buttons container with centered layout
        action_buttons = widgets.HBox(
            [play_button, sad_button, happy_button],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )

        # Profile buttons container with centered layout
        profile_buttons = widgets.HBox(
            [fav_edit_button, exit_button],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )

        action_title = widgets.HTML(f"<h2 style='{subtitle_style}'>Actions:</h2>")
        profile_title = widgets.HTML(f"<h2 style='{subtitle_style}'>Profile Settings:</h2>")
        mode_title = widgets.HTML(f"<h2 style='{subtitle_style}'>Select Your Experience Mode:</h2>")

        display(widgets.VBox([
            profile_label,
            user_info,
            mode_title,
            mode_selector,
            mode_description,
            action_title,
            action_buttons,
            profile_title,
            profile_buttons,
            output
        ], layout=container_layout))

    switch_to_page('start')
    #print("✅ Enhanced musicalllllll recommender interface loaded successfully!")

#if __name__ == "__main__":
#    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
start_music_recommender()

DEBUG: Starting enhanced multi-page recommender with improved UI/UX


TypeError: EmotionService.__init__() missing 1 required positional argument: 'result_df'

# Main

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService  # ✅ import the new class

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager  # ✅ import the new class

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################
def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with full error clearing")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    def show_status(message, status='info'):
        colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
        with output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    def switch_to_page(page):
        session['current_page'] = page
        show_status("")
        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

    def show_start_page():
        clear_output()
        start_label = widgets.HTML("<h3>🎵 Welcome to the Music Recommender</h3>")
        signin_button = widgets.Button(description='Sign In')
        signup_button = widgets.Button(description='Sign Up')

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            widgets.HBox([signin_button, signup_button]),
            output
        ]))

    def show_signin_page():
        clear_output()
        label = widgets.HTML("<h3>🔑 Sign In</h3>")
        si_username = widgets.Text(description='Username')
        si_password = widgets.Password(description='Password')
        si_button = widgets.Button(description='Confirm')
        back_button = widgets.Button(description='Back')

        def sign_in_action(b):
            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("❌ Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("❌ Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            si_username, si_password, si_button,
            back_button,
            output
        ]))

    def show_signup_page():
        clear_output()
        label = widgets.HTML("<h3>🔐 Sign Up</h3>")
        su_username = widgets.Text(description='Username')
        su_password = widgets.Password(description='Password')
        su_firstname = widgets.Text(description='First Name')
        su_lastname = widgets.Text(description='Last Name')
        su_age = widgets.IntText(description='Age')
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender')

        selected_artists = []

        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Add artist:',
            ensure_option=True
        )
        add_button = widgets.Button(description='Add')
        artist_box = widgets.VBox()

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(description='Deselect', layout=widgets.Layout(width='80px'))
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn]))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
            artist_input.value = ''

        add_button.on_click(on_add)

        su_button = widgets.Button(description='Submit')
        back_button = widgets.Button(description='Back')

        def sign_up_action(b):
            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("❌ Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender,
            widgets.HTML("<b>🎵 Favorite Artists</b>"),
            widgets.HBox([artist_input, add_button]),
            artist_box,
            su_button,
            back_button,
            output
        ]))

    def show_edit_favorites_page():
        clear_output()
        label = widgets.HTML("<h3>🎨 Edit Favorite Artists</h3>")
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Add artist:',
            ensure_option=True
        )
        add_button = widgets.Button(description='Add')
        artist_box = widgets.VBox()

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(description='Deselect', layout=widgets.Layout(width='80px'))
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn]))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
            artist_input.value = ''

        add_button.on_click(on_add)
        refresh_artist_table()

        confirm_button = widgets.Button(description='Save')
        back_button = widgets.Button(description='Back')

        def save_favorites(b):
            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists)
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(session['user_id'], selected_artists)
            show_status("✅ Favorites updated.", 'success')
            switch_to_page('session')

        confirm_button.on_click(save_favorites)
        back_button.on_click(lambda b: switch_to_page('session'))

        display(widgets.VBox([
            label,
            widgets.HTML("<b>🎵 Favorite Artists</b>"),
            widgets.HBox([artist_input, add_button]),
            artist_box,
            confirm_button,
            back_button,
            output
        ]))

    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<b>👤 Signed in as: {session['username']}</b>")
        mode_selector = widgets.ToggleButtons(
            options=['none', 'log', 'recommend'],
            description='Mode'
        )
        play_button = widgets.Button(description='Play Song')
        sad_button = widgets.Button(description='Feel Sad')
        happy_button = widgets.Button(description='Feel Happy')
        fav_edit_button = widgets.Button(description='Edit Favorites')
        exit_button = widgets.Button(description='Log Out')

        def change_mode(change):
            session['mode'] = change['new']
            show_status(f"🔄 Mode changed to: {session['mode']}", 'info')

        def play_song(b):
            show_status("")
            if session['mode'] == 'none':
                show_status("🔒 Privacy mode → not logging song play.", 'info')
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")
            if session['mode'] != 'recommend':
                show_status("⚠ Not in recommend mode — no recommendations made.", 'info')
                return

            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("⚠ No songs found to recommend.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]
            song_selector = widgets.Dropdown(
                options=options,
                description='Pick one:',
                layout=widgets.Layout(width='auto')
            )
            confirm_button = widgets.Button(description='Confirm Choice')

            def on_confirm_click(_):
                selected_label = song_selector.value
                selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(
                    session['user_id'],
                    selected_row['song_id'],
                    mood_before,
                    mood_after,
                    is_recommended=1
                )
                show_status(f"✅ You picked: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

            confirm_button.on_click(on_confirm_click)

            with music_recommender_output:
                clear_output()
                display(widgets.VBox([song_selector, confirm_button]))

        def feel_happy(b):
            show_status("")
            show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        fav_edit_button.on_click(lambda b: switch_to_page('edit_favorites'))
        exit_button.on_click(exit_session)

        display(widgets.VBox([
            profile_label,
            mode_selector,
            widgets.HBox([play_button, sad_button, happy_button]),
            fav_edit_button,
            exit_button,
            output
        ]))

    switch_to_page('start')
    print("✅ Multi-page interface with updated favorites-recommendation link loaded.")

In [ ]:
start_music_recommender()

DEBUG: Starting full multi-page recommender with full error clearing


TypeError: EmotionService.__init__() missing 1 required positional argument: 'result_df'

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys

##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################
def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with full error clearing")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    def show_status(message, status='info'):
        colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
        with output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    def switch_to_page(page):
        session['current_page'] = page
        show_status("")
        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

    def show_start_page():
        clear_output()
        start_label = widgets.HTML("<h3 style='font-family: sans-serif; color: #ff6f61;'>🎶 Welcome to MusicLand!</h3>")
        signin_button = widgets.Button(description='Sign In', button_style='success', layout=widgets.Layout(width='200px'))
        signup_button = widgets.Button(description='Sign Up', button_style='info', layout=widgets.Layout(width='200px'))

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            widgets.HBox([signin_button, signup_button]),
            output
        ]))

    def show_signin_page():
        clear_output()
        label = widgets.HTML("<h3 style='color: #ff6f61;'>🔑 Sign In</h3>")
        si_username = widgets.Text(description='Username', layout=widgets.Layout(width='300px'))
        si_password = widgets.Password(description='Password', layout=widgets.Layout(width='300px'))
        si_button = widgets.Button(description='Confirm', button_style='primary')
        back_button = widgets.Button(description='Back', button_style='warning')

        def sign_in_action(b):
            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("❌ Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("❌ Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("✅ Signed in successfully!", 'success')
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            si_username, si_password, si_button,
            back_button,
            output
        ]))

    def show_signup_page():
        clear_output()
        label = widgets.HTML("<h3 style='color: #ff6f61;'>🔐 Sign Up</h3>")
        su_username = widgets.Text(description='Username', layout=widgets.Layout(width='300px'))
        su_password = widgets.Password(description='Password', layout=widgets.Layout(width='300px'))
        su_firstname = widgets.Text(description='First Name', layout=widgets.Layout(width='300px'))
        su_lastname = widgets.Text(description='Last Name', layout=widgets.Layout(width='300px'))
        su_age = widgets.IntText(description='Age', layout=widgets.Layout(width='300px'))
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender', layout=widgets.Layout(width='300px'))

        selected_artists = []

        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Add artist:',
            ensure_option=True,
            layout=widgets.Layout(width='300px')
        )
        add_button = widgets.Button(description='Add', button_style='primary', layout=widgets.Layout(width='100px'))
        artist_box = widgets.VBox()

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(description='Deselect', layout=widgets.Layout(width='80px'))
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn]))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
            artist_input.value = ''

        add_button.on_click(on_add)

        su_button = widgets.Button(description='Submit', button_style='primary')
        back_button = widgets.Button(description='Back', button_style='warning')

        def sign_up_action(b):
            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("❌ Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("✅ Successfully signed up!", 'success')
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender,
            widgets.HTML("<b>🎵 Favorite Artists</b>"),
            widgets.HBox([artist_input, add_button]),
            artist_box,
            su_button,
            back_button,
            output
        ]))

    def show_edit_favorites_page():
        clear_output()
        label = widgets.HTML("<h3 style='color: #ff6f61;'>🎨 Edit Favorite Artists</h3>")
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Add artist:',
            ensure_option=True,
            layout=widgets.Layout(width='300px')
        )
        add_button = widgets.Button(description='Add', button_style='primary', layout=widgets.Layout(width='100px'))
        artist_box = widgets.VBox()

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(description='Deselect', layout=widgets.Layout(width='80px'))
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn]))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
            artist_input.value = ''

        add_button.on_click(on_add)
        refresh_artist_table()

        confirm_button = widgets.Button(description='Save', button_style='success')
        back_button = widgets.Button(description='Back', button_style='warning')

        def save_favorites(b):
            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists)
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(session['user_id'], selected_artists)
            show_status("✅ Favorites updated.", 'success')
            switch_to_page('session')

        confirm_button.on_click(save_favorites)
        back_button.on_click(lambda b: switch_to_page('session'))

        display(widgets.VBox([
            label,
            widgets.HTML("<b>🎵 Favorite Artists</b>"),
            widgets.HBox([artist_input, add_button]),
            artist_box,
            confirm_button,
            back_button,
            output
        ]))

    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<b>👤 Signed in as: {session['username']}</b>")
        mode_selector = widgets.ToggleButtons(
            options=['none', 'log', 'recommend'],
            description='Mode',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        play_button = widgets.Button(description='Play Song', button_style='primary')
        sad_button = widgets.Button(description='Feel Sad 😢', button_style='danger')
        happy_button = widgets.Button(description='Feel Happy 😊', button_style='success')
        fav_edit_button = widgets.Button(description='Edit Favorites', button_style='info')
        exit_button = widgets.Button(description='Log Out', button_style='warning')

        def change_mode(change):
            session['mode'] = change['new']
            show_status(f"🔄 Mode changed to: {session['mode']}", 'info')

        def play_song(b):
            show_status("")
            if session['mode'] == 'none':
                show_status("🔒 Privacy mode → not logging song play.", 'info')
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")
            if session['mode'] != 'recommend':
                show_status("⚠ Not in recommend mode — no recommendations made.", 'info')
                return

            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("⚠ No songs found to recommend.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]
            song_selector = widgets.Dropdown(
                options=options,
                description='Pick one:',
                layout=widgets.Layout(width='300px')
            )
            confirm_button = widgets.Button(description='Confirm Choice', button_style='success')

            def on_confirm_click(_):
                selected_label = song_selector.value
                selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(
                    session['user_id'],
                    selected_row['song_id'],
                    mood_before,
                    mood_after,
                    is_recommended=1
                )
                show_status(f"✅ You picked: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

            confirm_button.on_click(on_confirm_click)

            with music_recommender_output:
                clear_output()
                display(widgets.VBox([song_selector, confirm_button]))

        def feel_happy(b):
            show_status("")
            show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        fav_edit_button.on_click(lambda b: switch_to_page('edit_favorites'))
        exit_button.on_click(exit_session)

        display(widgets.VBox([
            profile_label,
            mode_selector,
            widgets.HBox([play_button, sad_button, happy_button]),
            fav_edit_button,
            exit_button,
            output
        ]))

    switch_to_page('start')
    print("✅ Multi-page interface with updated favorites-recommendation link loaded.")

In [ ]:
start_music_recommender()

✅ Multi-page interface with updated favorites-recommendation link loaded.


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys

##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################
def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with full error clearing")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    def show_status(message, status='info'):
        colors = {'info': 'orange', 'success': 'green', 'error': 'red'}
        with output:
            clear_output()
            if message:
                display(HTML(f"<span style='color:{colors[status]}; font-weight:bold'>{message}</span>"))

    def switch_to_page(page):
        session['current_page'] = page
        show_status("")
        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

    def show_start_page():
        clear_output()
        start_label = widgets.HTML("<h3 style='font-family: sans-serif; color: #ff6f61;'>🎶 Welcome to MusicLand!</h3>")
        signin_button = widgets.Button(description='Sign In', button_style='success', layout=widgets.Layout(width='200px'))
        signup_button = widgets.Button(description='Sign Up', button_style='info', layout=widgets.Layout(width='200px'))

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            widgets.HBox([signin_button, signup_button]),
            output
        ]))

    def show_signin_page():
        clear_output()
        label = widgets.HTML("<h3 style='color: #ff6f61;'>🔑 Sign In</h3>")
        si_username = widgets.Text(description='Username', layout=widgets.Layout(width='300px'))
        si_password = widgets.Password(description='Password', layout=widgets.Layout(width='300px'))
        si_button = widgets.Button(description='Confirm', button_style='primary')
        back_button = widgets.Button(description='Back', button_style='warning')

        def sign_in_action(b):
            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("❌ Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("❌ Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("✅ Signed in successfully!", 'success')
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            si_username, si_password, si_button,
            back_button,
            output
        ]))

    def show_signup_page():
        clear_output()
        label = widgets.HTML("<h3 style='color: #ff6f61;'>🔐 Sign Up</h3>")
        su_username = widgets.Text(description='Username', layout=widgets.Layout(width='300px'))
        su_password = widgets.Password(description='Password', layout=widgets.Layout(width='300px'))
        su_firstname = widgets.Text(description='First Name', layout=widgets.Layout(width='300px'))
        su_lastname = widgets.Text(description='Last Name', layout=widgets.Layout(width='300px'))
        su_age = widgets.IntText(description='Age', layout=widgets.Layout(width='300px'))
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender', layout=widgets.Layout(width='300px'))

        selected_artists = []

        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Add artist:',
            ensure_option=True,
            layout=widgets.Layout(width='300px')
        )
        add_button = widgets.Button(description='Add', button_style='primary', layout=widgets.Layout(width='100px'))
        artist_box = widgets.VBox()

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(description='Deselect', layout=widgets.Layout(width='80px'))
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn]))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
            artist_input.value = ''

        add_button.on_click(on_add)

        su_button = widgets.Button(description='Submit', button_style='primary')
        back_button = widgets.Button(description='Back', button_style='warning')

        def sign_up_action(b):
            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("❌ Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("❌ Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("✅ Successfully signed up!", 'success')
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: switch_to_page('start'))

        display(widgets.VBox([
            label,
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender,
            widgets.HTML("<b>🎵 Favorite Artists</b>"),
            widgets.HBox([artist_input, add_button]),
            artist_box,
            su_button,
            back_button,
            output
        ]))

    def show_edit_favorites_page():
        clear_output()
        label = widgets.HTML("<h3 style='color: #ff6f61;'>🎨 Edit Favorite Artists</h3>")
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Add artist:',
            ensure_option=True,
            layout=widgets.Layout(width='300px')
        )
        add_button = widgets.Button(description='Add', button_style='primary', layout=widgets.Layout(width='100px'))
        artist_box = widgets.VBox()

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(description='Deselect', layout=widgets.Layout(width='80px'))
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn]))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
            artist_input.value = ''

        add_button.on_click(on_add)
        refresh_artist_table()

        confirm_button = widgets.Button(description='Save', button_style='success')
        back_button = widgets.Button(description='Back', button_style='warning')

        def save_favorites(b):
            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists)
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(session['user_id'], selected_artists)
            show_status("✅ Favorites updated.", 'success')
            switch_to_page('session')

        confirm_button.on_click(save_favorites)
        back_button.on_click(lambda b: switch_to_page('session'))

        display(widgets.VBox([
            label,
            widgets.HTML("<b>🎵 Favorite Artists</b>"),
            widgets.HBox([artist_input, add_button]),
            artist_box,
            confirm_button,
            back_button,
            output
        ]))

    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<b>👤 Signed in as: {session['username']}</b>")
        mode_selector = widgets.ToggleButtons(
            options=['none', 'log', 'recommend'],
            description='Mode',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        play_button = widgets.Button(description='Play Song', button_style='primary')
        sad_button = widgets.Button(description='Feel Sad 😢', button_style='danger')
        happy_button = widgets.Button(description='Feel Happy 😊', button_style='success')
        fav_edit_button = widgets.Button(description='Edit Favorites', button_style='info')
        exit_button = widgets.Button(description='Log Out', button_style='warning')

        def change_mode(change):
            session['mode'] = change['new']
            show_status(f"🔄 Mode changed to: {session['mode']}", 'info')

        def play_song(b):
            show_status("")
            if session['mode'] == 'none':
                show_status("🔒 Privacy mode → not logging song play.", 'info')
                return
            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)
            show_status(f"🎧 Played: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")
            if session['mode'] != 'recommend':
                show_status("⚠ Not in recommend mode — no recommendations made.", 'info')
                return

            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("⚠ No songs found to recommend.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]
            song_selector = widgets.Dropdown(
                options=options,
                description='Pick one:',
                layout=widgets.Layout(width='300px')
            )
            confirm_button = widgets.Button(description='Confirm Choice', button_style='success')

            def on_confirm_click(_):
                selected_label = song_selector.value
                selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(
                    session['user_id'],
                    selected_row['song_id'],
                    mood_before,
                    mood_after,
                    is_recommended=1
                )
                show_status(f"✅ You picked: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

            confirm_button.on_click(on_confirm_click)

            with music_recommender_output:
                clear_output()
                display(widgets.VBox([song_selector, confirm_button]))

        def feel_happy(b):
            show_status("")
            show_status("😊 Mood noted: happy (no recommendations triggered).", 'info')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        fav_edit_button.on_click(lambda b: switch_to_page('edit_favorites'))
        exit_button.on_click(exit_session)

        display(widgets.VBox([
            profile_label,
            mode_selector,
            widgets.HBox([play_button, sad_button, happy_button]),
            fav_edit_button,
            exit_button,
            output
        ]))

    switch_to_page('start')
    print("✅ Multi-page interface with updated favorites-recommendation link loaded.")


In [ ]:
start_music_recommender()

✅ Multi-page interface with updated favorites-recommendation link loaded.


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService  # ✅ import the new class

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager  # ✅ import the new class

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################
def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with full error clearing")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    # Define common styles
    main_title_style = "color:#3366cc; font-size:24px; font-weight:bold; margin-bottom:15px;"
    subtitle_style = "color:#333; font-size:18px; font-weight:bold; margin-top:15px; margin-bottom:10px;"
    button_layout = widgets.Layout(margin="5px", width="auto", min_width="120px")

    # Navigation history
    nav_history = []

    def show_status(message, status='info'):
        colors = {'info': '#ff9900', 'success': '#00aa00', 'error': '#cc0000', 'warning': '#ff6600'}
        icons = {'info': 'ℹ️', 'success': '✅', 'error': '❌', 'warning': '⚠️'}
        with output:
            clear_output()
            if message:
                display(HTML(f"""
                <div style='padding:8px; border-radius:5px; background-color:{colors[status]}20;
                     border-left:4px solid {colors[status]}; margin:10px 0;'>
                    <span style='color:{colors[status]}; font-weight:bold'>{icons[status]} {message}</span>
                </div>
                """))

    def show_loading(message="Loading..."):
        with output:
            clear_output()
            display(HTML(f"""
            <div style='text-align:center; padding:15px;'>
                <div style='font-weight:bold; margin-bottom:10px;'>{message}</div>
                <div style='display:inline-block; width:40px; height:40px; border:5px solid #f3f3f3;
                     border-top:5px solid #3498db; border-radius:50%; animation:spin 1s linear infinite;'></div>
                <style>@keyframes spin {{0% {{ transform: rotate(0deg); }} 100% {{ transform: rotate(360deg); }}}}</style>
            </div>
            """))

    def switch_to_page(page, save_history=True):
        if save_history and session['current_page'] != page:
            nav_history.append(session['current_page'])
        session['current_page'] = page
        show_status("")
        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

    def go_back():
        if nav_history:
            previous_page = nav_history.pop()
            switch_to_page(previous_page, save_history=False)
        else:
            switch_to_page('start', save_history=False)

    def show_start_page():
        clear_output()
        start_label = widgets.HTML(f"<h1 style='{main_title_style}'>🎵 Music Recommender</h1>")
        welcome_text = widgets.HTML("<p style='margin-bottom:20px;'>Welcome to your personal music recommendation system.</p>")
        signin_button = widgets.Button(description='Sign In', button_style='primary', layout=button_layout)
        signup_button = widgets.Button(description='Sign Up', layout=button_layout)

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            welcome_text,
            widgets.HBox([signin_button, signup_button], layout=widgets.Layout(justify_content='center')),
            output
        ], layout=widgets.Layout(align_items='center', padding='20px')))

    def show_signin_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🔑 Sign In</h1>")
        si_username = widgets.Text(description='Username', style={'description_width': '80px'})
        si_password = widgets.Password(description='Password', style={'description_width': '80px'})
        si_button = widgets.Button(description='Sign In', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Back', layout=button_layout)

        def sign_in_action(b):
            show_loading("Signing in...")

            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: go_back())

        display(widgets.VBox([
            label,
            widgets.VBox([si_username, si_password], layout=widgets.Layout(width='300px')),
            widgets.HBox([si_button, back_button], layout=widgets.Layout(justify_content='flex-start')),
            output
        ], layout=widgets.Layout(align_items='center',justify_content='center', padding='20px')))
        #], layout=widgets.Layout(align_items='center', justify_content='center', padding='20px', width='100%', height='auto', overflow='hidden')))
        #], layout=widgets.Layout(align_items='flex-start', padding='20px')))


    def show_signup_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🔐 Sign Up</h1>")

        # Create form fields with consistent styling
        form_style = {'description_width': '100px'}
        form_layout = widgets.Layout(width='300px')

        su_username = widgets.Text(description='Username', style=form_style)
        su_password = widgets.Password(description='Password', style=form_style)
        su_firstname = widgets.Text(description='First Name', style=form_style)
        su_lastname = widgets.Text(description='Last Name', style=form_style)
        su_age = widgets.IntText(description='Age', style=form_style)
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender', style=form_style)

        selected_artists = []

        artist_header = widgets.HTML(f"<h2 style='{subtitle_style}'>🎵 Select Favorite Artists</h2>")
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(description='Add', layout=widgets.Layout(width='80px'))
        artist_box = widgets.VBox()

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(
                    description='Remove',
                    button_style='danger',
                    layout=widgets.Layout(width='80px')
                )
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn]))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)

        su_button = widgets.Button(description='Create Account', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Back', layout=button_layout)

        def sign_up_action(b):
            show_loading("Creating account...")

            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            if len(su_password.value) < 4:
                show_status("Password should be at least 4 characters long.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: go_back())

        display(widgets.VBox([
            label,
            widgets.VBox([
                su_username, su_password, su_firstname, su_lastname, su_age, su_gender
            ], layout=form_layout),
            artist_header,
            widgets.HBox([artist_input, add_button]),
            artist_box,
            widgets.HBox([su_button, back_button], layout=widgets.Layout(justify_content='flex-start', margin='10px 0')),
            output
        ], layout=widgets.Layout(align_items='center',padding='20px')))
        #], layout=widgets.Layout(align_items='flex-start', padding='20px')))

    def show_edit_favorites_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🎨 Edit Favorite Artists</h1>")
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style={'description_width': '100px'}
        )
        add_button = widgets.Button(description='Add', layout=widgets.Layout(width='80px'))
        artist_box = widgets.VBox()

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(
                    description='Remove',
                    button_style='danger',
                    layout=widgets.Layout(width='80px')
                )
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn]))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)
        refresh_artist_table()

        confirm_button = widgets.Button(description='Save Changes', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Cancel', layout=button_layout)

        def save_favorites(b):
            show_loading("Saving your preferences...")

            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists)
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(session['user_id'], selected_artists)
            show_status("Your favorite artists have been updated successfully!", 'success')
            switch_to_page('session')

        confirm_button.on_click(save_favorites)
        back_button.on_click(lambda b: switch_to_page('session'))

        display(widgets.VBox([
            label,
            widgets.HBox([artist_input, add_button]),
            widgets.HTML(f"<p style='{subtitle_style}'>Your Current Favorite Artists:</p>"),
            artist_box,
            widgets.HBox([confirm_button, back_button], layout=widgets.Layout(justify_content='flex-start', margin='10px 0')),
            output
        ], layout=widgets.Layout(align_items='center',justify_content='flex-start', padding='20px')))
        #], layout=widgets.Layout(align_items='flex-start', padding='20px')))


    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<h1 style='{main_title_style}'>🎵 Music Recommender</h1>")
        user_info = widgets.HTML(f"<p style='margin-bottom:15px;'><b>👤 Signed in as:</b> {session['username']}</p>")

        mode_selector = widgets.ToggleButtons(
            options=[
                ('Private Mode', 'none'),
                ('Log My Activity', 'log'),
                ('Recommend Me Music', 'recommend')
            ],
            value=session['mode'],
            description='Mode:',
            button_style='info',
            tooltips=[
                'Private Mode: No logging or recommendations',
                'Log Mode: Track your listening activity',
                'Recommend Mode: Get personalized music recommendations'
            ]
        )

        # Create a mapping for mode labels
        mode_labels = {
            'none': 'Private Mode',
            'log': 'Log My Activity',
            'recommend': 'Recommend Me Music'
        }

        mode_description = widgets.HTML(
            f"""<div style='margin:10px 0; padding:10px; background:#f5f5f5; border-radius:5px; width:auto;'>
                <p id='mode-desc'>Select a mode to continue.</p>
            </div>"""
        )

        # Action buttons with consistent styling
        play_button = widgets.Button(
            description='Play Random Song',
            icon='play',
            button_style='success',
            layout=button_layout
        )
        sad_button = widgets.Button(
            description='I Feel Sad 😢',
            button_style='info',
            layout=button_layout
        )
        happy_button = widgets.Button(
            description='I Feel Happy 😊',
            button_style='info',
            layout=button_layout
        )
        fav_edit_button = widgets.Button(
            description='Edit Favorites',
            icon='edit',
            layout=button_layout
        )
        exit_button = widgets.Button(
            description='Log Out',
            icon='sign-out',
            button_style='danger',
            layout=button_layout
        )

        def change_mode(change):
            session['mode'] = change['new']

            # Update mode description based on selection
            mode_descriptions = {
                'none': "Private Mode: Your activity won't be logged or used for recommendations.",
                'log': "Log Mode: Your listening activity will be tracked to improve recommendations.",
                'recommend': "Recommend Mode: Tell us how you're feeling and get personalized recommendations!"
            }
            mode_description.value = f"""<div style='margin:10px 0; padding:10px; background:#f5f5f5; border-radius:5px; width:auto;'>
                <p id='mode-desc'>{mode_descriptions[session['mode']]}</p>
            </div>"""

            show_status(f"Mode changed to: {mode_labels[session['mode']]}", 'info')

        def play_song(b):
            show_status("")
            show_loading("Finding a song...")

            if session['mode'] == 'none':
                show_status("Privacy mode → not logging song play.", 'info')
                return

            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)

            show_status(f"Now playing: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")

            if session['mode'] != 'recommend':
                show_status("Not in recommend mode — please switch to 'Recommend Me Music' mode first.", 'warning')
                return

            show_loading("Finding songs to lift your mood...")
            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("No songs found to recommend. Try adding more favorite artists.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]

            # Create a nicer song selection widget
            song_header = widgets.HTML("<h3 style='margin-top:15px;'>🎵 Songs to cheer you up:</h3>")
            song_options = widgets.RadioButtons(
                options=options,
                layout=widgets.Layout(width='auto')
            )
            confirm_button = widgets.Button(description='Play Selected Song', button_style='success')

            def on_confirm_click(_):
                if not song_options.value:
                    show_status("Please select a song first", 'warning')
                    return

                show_loading("Playing your selection...")

                selected_label = song_options.value
                selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(
                    session['user_id'],
                    selected_row['song_id'],
                    mood_before,
                    mood_after,
                    is_recommended=1
                )
                show_status(f"Now playing: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

            confirm_button.on_click(on_confirm_click)

            with music_recommender_output:
                clear_output()
                display(widgets.VBox([
                    song_header,
                    song_options,
                    confirm_button
                ]))

        def feel_happy(b):
            show_status("")
            show_status("Thanks for sharing! We're glad you're feeling happy today. 😊", 'success')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        fav_edit_button.on_click(lambda b: switch_to_page('edit_favorites'))
        exit_button.on_click(exit_session)

        # Initialize mode description
        change_mode({'new': session['mode']})

        display(widgets.VBox([
            profile_label,
            user_info,
            widgets.HTML(f"<h2 style='{subtitle_style}'>Select Your Experience Mode:</h2>"),
            mode_selector,
            mode_description,
            widgets.HTML(f"<h2 style='{subtitle_style}'>Actions:</h2>"),
            widgets.HBox([play_button, sad_button, happy_button], layout=widgets.Layout(justify_content='flex-start')),
            widgets.HTML(f"<h2 style='{subtitle_style}'>Profile Settings:</h2>"),
            widgets.HBox([fav_edit_button, exit_button], layout=widgets.Layout(justify_content='flex-start')),
            output
        ], layout=widgets.Layout(align_items='center',justify_content='space-between',padding='20px')))
        #], layout=widgets.Layout(align_items='flex-start', padding='20px')))

    switch_to_page('start')
    print("✅ Enhanced music recommender interface loaded successfully!")

In [ ]:
start_music_recommender()

In [ ]:
#%%writefile "/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/main.py"
import ipywidgets as widgets
from google.colab import drive
drive.mount('/content/drive')
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService  # ✅ import the new class

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager  # ✅ import the new class

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
project_dir = '/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/'
data_dir = project_dir + 'data/'
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################
def start_music_recommender():
    print("DEBUG: Starting enhanced multi-page recommender with improved UI/UX")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    # Define common styles
    main_title_style = "color:#3366cc; font-size:24px; font-weight:bold; margin-bottom:15px; text-align:center;"
    subtitle_style = "color:#333; font-size:18px; font-weight:bold; margin-top:15px; margin-bottom:10px; text-align:center;"

    # Improved button styles with transitions
    button_layout = widgets.Layout(
        margin="8px",
        width="auto",
        min_width="120px"
    )

    # Common container layout - consistently centered with fixed width
    container_layout = widgets.Layout(
        display="flex",
        flex_flow="column",
        align_items="center",
        width="600px",
        max_width="100%",
        margin="0 auto",
        overflow="visible",  # Changed from hidden to visible
        padding="20px",
        border="none"
    )

    # Define CSS for transitions and smooth animations
    display(HTML("""
    <style>
    /* Smooth transitions for all elements */
    .widget-button, .widget-text, .widget-password,
    .widget-dropdown, .widget-label, .widget-html {
        transition: all 0.3s ease-in-out;
    }

    /* Improved Button Hover Effects */
    .widget-button:hover {
        transform: translateY(-2px);
        box-shadow: 0 4px 8px rgba(0,0,0,0.1);
    }

    /* Remove outline focus on widgets */
    .widget-box:focus {
        outline: none !important;
    }

    /* Comprehensive scrollbar hiding */
    ::-webkit-scrollbar {
        width: 0px !important;
        height: 0px !important;
        background: transparent !important;
        display: none !important;
    }

    /* Hide scrollbars everywhere - more aggressive approach */
    * {
        -ms-overflow-style: none !important;  /* IE and Edge */
        scrollbar-width: none !important;  /* Firefox */
    }

    /* Additional scrollbar hiding for specific containers */
    .widget-box, .widget-vbox, .widget-hbox,
    .jupyter-widgets, .widget-container,
    .p-Widget, .p-Panel, .lm-Widget, .lm-Panel {
        overflow: hidden !important;
        scrollbar-width: none !important;
    }

    /* Force all Jupyter output areas to hide scrollbars */
    .jp-OutputArea, .jp-OutputArea-output, .jp-OutputArea-child {
        overflow: hidden !important;
        max-height: none !important;
    }

    /* Force the main output widget to expand as needed without scrollbars */
    .output_scroll {
        height: auto !important;
        max-height: none !important;
        overflow-y: hidden !important;
    }

    /* Fade animation for page transitions */
    @keyframes fadeIn {
        from { opacity: 0; transform: translateY(10px); }
        to { opacity: 1; transform: translateY(0); }
    }

    .fade-in {
        animation: fadeIn 0.4s ease-in-out;
    }

    /* Ensure proper sizing for input elements */
    .widget-text, .widget-textarea, .widget-password {
        box-sizing: border-box !important;
        width: 100% !important;
    }
    </style>
    """))

    # Navigation history
    nav_history = []

    def show_status(message, status='info'):
        colors = {'info': '#ff9900', 'success': '#00aa00', 'error': '#cc0000', 'warning': '#ff6600'}
        icons = {'info': 'ℹ️', 'success': '✅', 'error': '❌', 'warning': '⚠️'}
        with output:
            clear_output()
            if message:
                display(HTML(f"""
                <div style='padding:10px; border-radius:8px; background-color:{colors[status]}15;
                     border-left:4px solid {colors[status]}; margin:15px auto; max-width:500px; text-align:center;'>
                    <span style='color:{colors[status]}; font-weight:bold'>{icons[status]} {message}</span>
                </div>
                """))

    def show_loading(message="Loading..."):
        with output:
            clear_output()
            display(HTML(f"""
            <div style='text-align:center; padding:15px;'>
                <div style='font-weight:bold; margin-bottom:15px;'>{message}</div>
                <div style='display:inline-block; width:40px; height:40px; border:5px solid #f3f3f3;
                     border-top:5px solid #3498db; border-radius:50%; animation:spin 1s linear infinite;'></div>
                <style>@keyframes spin {{0% {{ transform: rotate(0deg); }} 100% {{ transform: rotate(360deg); }}}}</style>
            </div>
            """))

    def switch_to_page(page, save_history=True):
        if save_history and session['current_page'] != page:
            nav_history.append(session['current_page'])
        session['current_page'] = page
        show_status("")

        # Add fade transition between pages
        display(HTML("<div class='fade-in'>"))

        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

        display(HTML("</div>"))

    def go_back():
        if nav_history:
            previous_page = nav_history.pop()
            switch_to_page(previous_page, save_history=False)
        else:
            switch_to_page('start', save_history=False)

    def show_start_page():
        clear_output()
        start_label = widgets.HTML(f"<h1 style='{main_title_style}'>🎵 Music Recommender</h1>")
        welcome_text = widgets.HTML("<p style='margin-bottom:20px; text-align:center;'>Welcome to your personal music recommendation system.</p>")
        signin_button = widgets.Button(description='Sign In', button_style='primary', layout=button_layout)
        signup_button = widgets.Button(description='Sign Up', layout=button_layout)

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            welcome_text,
            widgets.HBox([signin_button, signup_button], layout=widgets.Layout(justify_content='center')),
            output
        ], layout=container_layout))

    def show_signin_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🔑 Sign In</h1>")

        # Consistent form field styling
        form_style = {'description_width': '80px'}
        form_layout = widgets.Layout(width='300px', margin='0 auto')

        si_username = widgets.Text(description='Username', style=form_style)
        si_password = widgets.Password(description='Password', style=form_style)
        si_button = widgets.Button(description='Sign In', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Back', layout=button_layout)

        def sign_in_action(b):
            show_loading("Signing in...")

            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: go_back())

        buttons_layout = widgets.Layout(justify_content='center', margin='15px 0')

        # Create a simplified container without nested layouts
        signin_form = widgets.VBox([
            si_username,
            si_password
        ], layout=form_layout)

        buttons = widgets.HBox([
            si_button,
            back_button
        ], layout=buttons_layout)

        # Use minimal nesting to avoid scroll issues
        main_container = widgets.VBox([
            label,
            signin_form,
            buttons,
            output
        ], layout=container_layout)

        # Direct display without further nesting
        display(main_container)

    def show_signup_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🔐 Sign Up</h1>")

        # Create form fields with consistent styling
        form_style = {'description_width': '100px'}
        form_layout = widgets.Layout(width='300px', margin='0 auto')

        su_username = widgets.Text(description='Username', style=form_style)
        su_password = widgets.Password(description='Password', style=form_style)
        su_firstname = widgets.Text(description='First Name', style=form_style)
        su_lastname = widgets.Text(description='Last Name', style=form_style)
        su_age = widgets.IntText(description='Age', style=form_style)
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender', style=form_style)

        selected_artists = []

        artist_header = widgets.HTML(f"<h2 style='{subtitle_style}'>🎵 Select Favorite Artists</h2>")
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(description='Add', layout=widgets.Layout(width='80px'))

        # Use fixed height with visible overflow only for the artist box
        artist_box = widgets.VBox(layout=widgets.Layout(
            width='400px',
            margin='10px auto',
            # No max-height to prevent scroll issues
        ))

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(
                    description='Remove',
                    button_style='danger',
                    layout=widgets.Layout(width='80px')
                )
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn], layout=widgets.Layout(justify_content='space-between', width='100%')))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)

        su_button = widgets.Button(description='Create Account', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Back', layout=button_layout)

        def sign_up_action(b):
            show_loading("Creating account...")

            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            if len(su_password.value) < 4:
                show_status("Password should be at least 4 characters long.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: go_back())

        artist_input_container = widgets.HBox([artist_input, add_button],
                                             layout=widgets.Layout(width='400px', margin='0 auto', justify_content='space-between'))

        buttons_layout = widgets.Layout(justify_content='center', margin='15px 0')

        # Group form fields to reduce nesting
        user_form = widgets.VBox([
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender
        ], layout=form_layout)

        buttons = widgets.HBox([
            su_button, back_button
        ], layout=buttons_layout)

        # Simplified container structure
        main_container = widgets.VBox([
            label,
            user_form,
            artist_header,
            artist_input_container,
            artist_box,
            buttons,
            output
        ], layout=container_layout)

        display(main_container)

    def show_edit_favorites_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🎨 Edit Favorite Artists</h1>")
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        form_style = {'description_width': '100px'}
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(description='Add', layout=widgets.Layout(width='80px'))

        # Remove fixed height and scroll settings
        artist_box = widgets.VBox(layout=widgets.Layout(
            width='400px',
            margin='10px auto'
        ))

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(
                    description='Remove',
                    button_style='danger',
                    layout=widgets.Layout(width='80px')
                )
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn], layout=widgets.Layout(justify_content='space-between', width='100%')))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)
        refresh_artist_table()

        confirm_button = widgets.Button(description='Save Changes', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Cancel', layout=button_layout)

        def save_favorites(b):
            show_loading("Saving your preferences...")

            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists)
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(session['user_id'], selected_artists)
            show_status("Your favorite artists have been updated successfully!", 'success')
            switch_to_page('session')

        confirm_button.on_click(save_favorites)
        back_button.on_click(lambda b: switch_to_page('session'))

        artist_input_container = widgets.HBox([artist_input, add_button],
                                             layout=widgets.Layout(width='400px', margin='0 auto', justify_content='space-between'))

        artist_header = widgets.HTML(f"<p style='{subtitle_style}'>Your Current Favorite Artists:</p>")

        buttons_layout = widgets.Layout(justify_content='center', margin='15px 0')
        buttons = widgets.HBox([confirm_button, back_button], layout=buttons_layout)

        # Simplified structure
        main_container = widgets.VBox([
            label,
            artist_input_container,
            artist_header,
            artist_box,
            buttons,
            output
        ], layout=container_layout)

        display(main_container)

    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<h1 style='{main_title_style}'>🎵 Music Recommender</h1>")
        user_info = widgets.HTML(f"<p style='text-align:center; margin-bottom:15px;'><b>👤 Signed in as:</b> {session['username']}</p>")

        # Improved mode selector with better spacing
        mode_selector = widgets.ToggleButtons(
            options=[
                ('Private Mode', 'none'),
                ('Log My Activity', 'log'),
                ('Recommend Me Music', 'recommend')
            ],
            value=session['mode'],
            description='Mode:',
            button_style='info',
            tooltips=[
                'Private Mode: No logging or recommendations',
                'Log Mode: Track your listening activity',
                'Recommend Mode: Get personalized music recommendations'
            ],
            layout=widgets.Layout(width='auto', margin='0 auto')
        )

        # Create a mapping for mode labels
        mode_labels = {
            'none': 'Private Mode',
            'log': 'Log My Activity',
            'recommend': 'Recommend Me Music'
        }

        mode_description = widgets.HTML(
            f"""<div style='margin:15px auto; padding:12px; background:#f8f8f8; border-radius:8px; width:500px; text-align:center;'>
                <p id='mode-desc' style='margin:0;'>Select a mode to continue.</p>
            </div>"""
        )

        # Action buttons with consistent styling and better layout
        play_button = widgets.Button(
            description='Play Random Song',
            icon='play',
            button_style='success',
            layout=button_layout
        )
        sad_button = widgets.Button(
            description='I Feel Sad 😢',
            button_style='info',
            layout=button_layout
        )
        happy_button = widgets.Button(
            description='I Feel Happy 😊',
            button_style='info',
            layout=button_layout
        )
        fav_edit_button = widgets.Button(
            description='Edit Favorites',
            icon='edit',
            layout=button_layout
        )
        exit_button = widgets.Button(
            description='Log Out',
            icon='sign-out',
            button_style='danger',
            layout=button_layout
        )

        def change_mode(change):
            session['mode'] = change['new']

            # Update mode description based on selection
            mode_descriptions = {
                'none': "Private Mode: Your activity won't be logged or used for recommendations.",
                'log': "Log Mode: Your listening activity will be tracked to improve recommendations.",
                'recommend': "Recommend Mode: Tell us how you're feeling and get personalized recommendations!"
            }
            mode_description.value = f"""<div style='margin:15px auto; padding:12px; background:#f8f8f8; border-radius:8px; width:500px; text-align:center;'>
                <p id='mode-desc' style='margin:0;'>{mode_descriptions[session['mode']]}</p>
            </div>"""

            show_status(f"Mode changed to: {mode_labels[session['mode']]}", 'info')

        def play_song(b):
            show_status("")
            show_loading("Finding a song...")

            if session['mode'] == 'none':
                show_status("Privacy mode → not logging song play.", 'info')
                return

            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)

            show_status(f"Now playing: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")

            if session['mode'] != 'recommend':
                show_status("Not in recommend mode — please switch to 'Recommend Me Music' mode first.", 'warning')
                return

            show_loading("Finding songs to lift your mood...")
            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("No songs found to recommend. Try adding more favorite artists.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]

            # Create a nicer song selection widget
            song_header = widgets.HTML("<h3 style='margin-top:15px; text-align:center;'>🎵 Songs to cheer you up:</h3>")
            song_options = widgets.RadioButtons(
                options=options,
                layout=widgets.Layout(width='auto', margin='10px auto')
            )
            confirm_button = widgets.Button(
                description='Play Selected Song',
                button_style='success',
                layout=widgets.Layout(margin='10px auto', display='flex', justify_content='center')
            )

            def on_confirm_click(_):
                if not song_options.value:
                    show_status("Please select a song first", 'warning')
                    return

                show_loading("Playing your selection...")

                selected_label = song_options.value
                selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(
                    session['user_id'],
                    selected_row['song_id'],
                    mood_before,
                    mood_after,
                    is_recommended=1
                )
                show_status(f"Now playing: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

            confirm_button.on_click(on_confirm_click)

            with music_recommender_output:
                clear_output()
                display(widgets.VBox([
                    song_header,
                    song_options,
                    confirm_button
                ], layout=widgets.Layout(align_items='center', width='500px', margin='0 auto')))

        def feel_happy(b):
            show_status("")
            show_status("Thanks for sharing! We're glad you're feeling happy today. 😊", 'success')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        fav_edit_button.on_click(lambda b: switch_to_page('edit_favorites'))
        exit_button.on_click(exit_session)

        # Initialize mode description
        change_mode({'new': session['mode']})

        # Action buttons container with centered layout
        action_buttons = widgets.HBox(
            [play_button, sad_button, happy_button],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )

        # Profile buttons container with centered layout
        profile_buttons = widgets.HBox(
            [fav_edit_button, exit_button],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )

        action_title = widgets.HTML(f"<h2 style='{subtitle_style}'>Actions:</h2>")
        profile_title = widgets.HTML(f"<h2 style='{subtitle_style}'>Profile Settings:</h2>")
        mode_title = widgets.HTML(f"<h2 style='{subtitle_style}'>Select Your Experience Mode:</h2>")

        display(widgets.VBox([
            profile_label,
            user_info,
            mode_title,
            mode_selector,
            mode_description,
            action_title,
            action_buttons,
            profile_title,
            profile_buttons,
            output
        ], layout=container_layout))

    switch_to_page('start')
    #print("✅ Enhanced musicalllllll recommender interface loaded successfully!")

if __name__ == "__main__":
    main()

Overwriting /content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/main.py


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService  # ✅ import the new class

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager  # ✅ import the new class

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################
def start_music_recommender():
    print("DEBUG: Starting full multi-page recommender with full error clearing")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    # Define common styles
    main_title_style = "color:#3366cc; font-size:24px; font-weight:bold; margin-bottom:15px; text-align:center;"
    subtitle_style = "color:#333; font-size:18px; font-weight:bold; margin-top:15px; margin-bottom:10px; text-align:center;"

    # Improved button styles with transitions
    button_layout = widgets.Layout(
        margin="8px",
        width="auto",
        min_width="120px"
    )

    # Common container layout - consistently centered with fixed width
    container_layout = widgets.Layout(
        display="flex",
        flex_flow="column",
        align_items="center",
        width="600px",
        margin="0 auto",
        overflow="hidden",  # Prevents scrollbars
        padding="20px"
    )

    # Define CSS for transitions and smooth animations
    display(HTML("""
    <style>
    /* Smooth transitions for all elements */
    .widget-button, .widget-text, .widget-password,
    .widget-dropdown, .widget-label, .widget-html {
        transition: all 0.3s ease-in-out;
    }

    /* Improved Button Hover Effects */
    .widget-button:hover {
        transform: translateY(-2px);
        box-shadow: 0 4px 8px rgba(0,0,0,0.1);
    }

    /* Remove outline focus on widgets */
    .widget-box:focus {
        outline: none !important;
    }

    /* Custom scrollbar style */
    ::-webkit-scrollbar {
        width: 8px;
        height: 8px;
    }

    ::-webkit-scrollbar-track {
        background: #f1f1f1;
        border-radius: 10px;
    }

    ::-webkit-scrollbar-thumb {
        background: #c1c1c1;
        border-radius: 10px;
    }

    ::-webkit-scrollbar-thumb:hover {
        background: #999;
    }

    /* Hide overflow scrollbars in the main output area */
    .jp-OutputArea-output {
        overflow: hidden !important;
    }

    /* Fade animation for page transitions */
    @keyframes fadeIn {
        from { opacity: 0; transform: translateY(10px); }
        to { opacity: 1; transform: translateY(0); }
    }

    .fade-in {
        animation: fadeIn 0.4s ease-in-out;
    }
    </style>
    """))

    # Navigation history
    nav_history = []

    def show_status(message, status='info'):
        colors = {'info': '#ff9900', 'success': '#00aa00', 'error': '#cc0000', 'warning': '#ff6600'}
        icons = {'info': 'ℹ️', 'success': '✅', 'error': '❌', 'warning': '⚠️'}
        with output:
            clear_output()
            if message:
                display(HTML(f"""
                <div style='padding:10px; border-radius:8px; background-color:{colors[status]}15;
                     border-left:4px solid {colors[status]}; margin:15px auto; max-width:500px; text-align:center;'>
                    <span style='color:{colors[status]}; font-weight:bold'>{icons[status]} {message}</span>
                </div>
                """))

    def show_loading(message="Loading..."):
        with output:
            clear_output()
            display(HTML(f"""
            <div style='text-align:center; padding:15px;'>
                <div style='font-weight:bold; margin-bottom:15px;'>{message}</div>
                <div style='display:inline-block; width:40px; height:40px; border:5px solid #f3f3f3;
                     border-top:5px solid #3498db; border-radius:50%; animation:spin 1s linear infinite;'></div>
                <style>@keyframes spin {{0% {{ transform: rotate(0deg); }} 100% {{ transform: rotate(360deg); }}}}</style>
            </div>
            """))

    def switch_to_page(page, save_history=True):
        if save_history and session['current_page'] != page:
            nav_history.append(session['current_page'])
        session['current_page'] = page
        show_status("")

        # Add fade transition between pages
        display(HTML("<div class='fade-in'>"))

        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

        display(HTML("</div>"))

    def go_back():
        if nav_history:
            previous_page = nav_history.pop()
            switch_to_page(previous_page, save_history=False)
        else:
            switch_to_page('start', save_history=False)

    def show_start_page():
        clear_output()
        start_label = widgets.HTML(f"<h1 style='{main_title_style}'>🎵 Music Recommender</h1>")
        welcome_text = widgets.HTML("<p style='margin-bottom:20px; text-align:center;'>Welcome to your personal music recommendation system.</p>")
        signin_button = widgets.Button(description='Sign In', button_style='primary', layout=button_layout)
        signup_button = widgets.Button(description='Sign Up', layout=button_layout)

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            welcome_text,
            widgets.HBox([signin_button, signup_button], layout=widgets.Layout(justify_content='center')),
            output
        ], layout=container_layout))

    def show_signin_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🔑 Sign In</h1>")

        # Consistent form field styling
        form_style = {'description_width': '80px'}
        form_layout = widgets.Layout(width='300px', margin='0 auto')

        si_username = widgets.Text(description='Username', style=form_style)
        si_password = widgets.Password(description='Password', style=form_style)
        si_button = widgets.Button(description='Sign In', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Back', layout=button_layout)

        def sign_in_action(b):
            show_loading("Signing in...")

            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: go_back())

        buttons_layout = widgets.Layout(justify_content='center', margin='15px 0')

        display(widgets.VBox([
            label,
            widgets.VBox([si_username, si_password], layout=form_layout),
            widgets.HBox([si_button, back_button], layout=buttons_layout),
            output
        ], layout=container_layout))

    def show_signup_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🔐 Sign Up</h1>")

        # Create form fields with consistent styling
        form_style = {'description_width': '100px'}
        form_layout = widgets.Layout(width='300px', margin='0 auto')

        su_username = widgets.Text(description='Username', style=form_style)
        su_password = widgets.Password(description='Password', style=form_style)
        su_firstname = widgets.Text(description='First Name', style=form_style)
        su_lastname = widgets.Text(description='Last Name', style=form_style)
        su_age = widgets.IntText(description='Age', style=form_style)
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender', style=form_style)

        selected_artists = []

        artist_header = widgets.HTML(f"<h2 style='{subtitle_style}'>🎵 Select Favorite Artists</h2>")
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(description='Add', layout=widgets.Layout(width='80px'))
        artist_box = widgets.VBox(layout=widgets.Layout(max_height='200px', overflow_y='auto', width='400px', margin='10px auto'))

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(
                    description='Remove',
                    button_style='danger',
                    layout=widgets.Layout(width='80px')
                )
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn], layout=widgets.Layout(justify_content='space-between', width='100%')))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)

        su_button = widgets.Button(description='Create Account', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Back', layout=button_layout)

        def sign_up_action(b):
            show_loading("Creating account...")

            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            if len(su_password.value) < 4:
                show_status("Password should be at least 4 characters long.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: go_back())

        artist_input_container = widgets.HBox([artist_input, add_button],
                                             layout=widgets.Layout(width='400px', margin='0 auto', justify_content='space-between'))

        buttons_layout = widgets.Layout(justify_content='center', margin='15px 0')

        display(widgets.VBox([
            label,
            widgets.VBox([
                su_username, su_password, su_firstname, su_lastname, su_age, su_gender
            ], layout=form_layout),
            artist_header,
            artist_input_container,
            artist_box,
            widgets.HBox([su_button, back_button], layout=buttons_layout),
            output
        ], layout=container_layout))

    def show_edit_favorites_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🎨 Edit Favorite Artists</h1>")
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        form_style = {'description_width': '100px'}
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(description='Add', layout=widgets.Layout(width='80px'))
        artist_box = widgets.VBox(layout=widgets.Layout(max_height='200px', overflow_y='auto', width='400px', margin='10px auto'))

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(
                    description='Remove',
                    button_style='danger',
                    layout=widgets.Layout(width='80px')
                )
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn], layout=widgets.Layout(justify_content='space-between', width='100%')))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)
        refresh_artist_table()

        confirm_button = widgets.Button(description='Save Changes', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Cancel', layout=button_layout)

        def save_favorites(b):
            show_loading("Saving your preferences...")

            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists)
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(session['user_id'], selected_artists)
            show_status("Your favorite artists have been updated successfully!", 'success')
            switch_to_page('session')

        confirm_button.on_click(save_favorites)
        back_button.on_click(lambda b: switch_to_page('session'))

        artist_input_container = widgets.HBox([artist_input, add_button],
                                             layout=widgets.Layout(width='400px', margin='0 auto', justify_content='space-between'))

        artist_header = widgets.HTML(f"<p style='{subtitle_style}'>Your Current Favorite Artists:</p>")

        buttons_layout = widgets.Layout(justify_content='center', margin='15px 0')

        display(widgets.VBox([
            label,
            artist_input_container,
            artist_header,
            artist_box,
            widgets.HBox([confirm_button, back_button], layout=buttons_layout),
            output
        ], layout=container_layout))

    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<h1 style='{main_title_style}'>🎵 Music Recommender</h1>")
        user_info = widgets.HTML(f"<p style='text-align:center; margin-bottom:15px;'><b>👤 Signed in as:</b> {session['username']}</p>")

        # Improved mode selector with better spacing
        mode_selector = widgets.ToggleButtons(
            options=[
                ('Private Mode', 'none'),
                ('Log My Activity', 'log'),
                ('Recommend Me Music', 'recommend')
            ],
            value=session['mode'],
            description='Mode:',
            button_style='info',
            tooltips=[
                'Private Mode: No logging or recommendations',
                'Log Mode: Track your listening activity',
                'Recommend Mode: Get personalized music recommendations'
            ],
            layout=widgets.Layout(width='auto', margin='0 auto')
        )

        # Create a mapping for mode labels
        mode_labels = {
            'none': 'Private Mode',
            'log': 'Log My Activity',
            'recommend': 'Recommend Me Music'
        }

        mode_description = widgets.HTML(
            f"""<div style='margin:15px auto; padding:12px; background:#f8f8f8; border-radius:8px; width:500px; text-align:center;'>
                <p id='mode-desc' style='margin:0;'>Select a mode to continue.</p>
            </div>"""
        )

        # Action buttons with consistent styling and better layout
        play_button = widgets.Button(
            description='Play Random Song',
            icon='play',
            button_style='success',
            layout=button_layout
        )
        sad_button = widgets.Button(
            description='I Feel Sad 😢',
            button_style='info',
            layout=button_layout
        )
        happy_button = widgets.Button(
            description='I Feel Happy 😊',
            button_style='info',
            layout=button_layout
        )
        fav_edit_button = widgets.Button(
            description='Edit Favorites',
            icon='edit',
            layout=button_layout
        )
        exit_button = widgets.Button(
            description='Log Out',
            icon='sign-out',
            button_style='danger',
            layout=button_layout
        )

        def change_mode(change):
            session['mode'] = change['new']

            # Update mode description based on selection
            mode_descriptions = {
                'none': "Private Mode: Your activity won't be logged or used for recommendations.",
                'log': "Log Mode: Your listening activity will be tracked to improve recommendations.",
                'recommend': "Recommend Mode: Tell us how you're feeling and get personalized recommendations!"
            }
            mode_description.value = f"""<div style='margin:15px auto; padding:12px; background:#f8f8f8; border-radius:8px; width:500px; text-align:center;'>
                <p id='mode-desc' style='margin:0;'>{mode_descriptions[session['mode']]}</p>
            </div>"""

            show_status(f"Mode changed to: {mode_labels[session['mode']]}", 'info')

        def play_song(b):
            show_status("")
            show_loading("Finding a song...")

            if session['mode'] == 'none':
                show_status("Privacy mode → not logging song play.", 'info')
                return

            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)

            show_status(f"Now playing: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")

            if session['mode'] != 'recommend':
                show_status("Not in recommend mode — please switch to 'Recommend Me Music' mode first.", 'warning')
                return

            show_loading("Finding songs to lift your mood...")
            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("No songs found to recommend. Try adding more favorite artists.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]

            # Create a nicer song selection widget
            song_header = widgets.HTML("<h3 style='margin-top:15px; text-align:center;'>🎵 Songs to cheer you up:</h3>")
            song_options = widgets.RadioButtons(
                options=options,
                layout=widgets.Layout(width='auto', margin='10px auto')
            )
            confirm_button = widgets.Button(
                description='Play Selected Song',
                button_style='success',
                layout=widgets.Layout(margin='10px auto', display='flex', justify_content='center')
            )

            def on_confirm_click(_):
                if not song_options.value:
                    show_status("Please select a song first", 'warning')
                    return

                show_loading("Playing your selection...")

                selected_label = song_options.value
                selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(
                    session['user_id'],
                    selected_row['song_id'],
                    mood_before,
                    mood_after,
                    is_recommended=1
                )
                show_status(f"Now playing: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

            confirm_button.on_click(on_confirm_click)

            with music_recommender_output:
                clear_output()
                display(widgets.VBox([
                    song_header,
                    song_options,
                    confirm_button
                ], layout=widgets.Layout(align_items='center', width='500px', margin='0 auto')))

        def feel_happy(b):
            show_status("")
            show_status("Thanks for sharing! We're glad you're feeling happy today. 😊", 'success')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        fav_edit_button.on_click(lambda b: switch_to_page('edit_favorites'))
        exit_button.on_click(exit_session)

        # Initialize mode description
        change_mode({'new': session['mode']})

        # Action buttons container with centered layout
        action_buttons = widgets.HBox(
            [play_button, sad_button, happy_button],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )

        # Profile buttons container with centered layout
        profile_buttons = widgets.HBox(
            [fav_edit_button, exit_button],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )

        action_title = widgets.HTML(f"<h2 style='{subtitle_style}'>Actions:</h2>")
        profile_title = widgets.HTML(f"<h2 style='{subtitle_style}'>Profile Settings:</h2>")
        mode_title = widgets.HTML(f"<h2 style='{subtitle_style}'>Select Your Experience Mode:</h2>")

        display(widgets.VBox([
            profile_label,
            user_info,
            mode_title,
            mode_selector,
            mode_description,
            action_title,
            action_buttons,
            profile_title,
            profile_buttons,
            output
        ], layout=container_layout))

    switch_to_page('start')
    print("✅ Enhanced music recommender interface loaded successfully!")

In [ ]:
start_music_recommender()

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService  # ✅ import the new class

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager  # ✅ import the new class

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################
def start_music_recommender():
    print("DEBUG: Starting enhanced multi-page recommender with improved UI/UX")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    # Define common styles
    main_title_style = "color:#3366cc; font-size:24px; font-weight:bold; margin-bottom:15px; text-align:center;"
    subtitle_style = "color:#333; font-size:18px; font-weight:bold; margin-top:15px; margin-bottom:10px; text-align:center;"

    # Improved button styles with transitions
    button_layout = widgets.Layout(
        margin="8px",
        width="auto",
        min_width="120px"
    )

    # Common container layout - consistently centered with fixed width
    container_layout = widgets.Layout(
        display="flex",
        flex_flow="column",
        align_items="center",
        width="600px",
        max_width="100%",
        margin="0 auto",
        overflow="visible",  # Changed from hidden to visible
        padding="20px",
        border="none"
    )

    # Define CSS for transitions and smooth animations
    display(HTML("""
    <style>
    /* Smooth transitions for all elements */
    .widget-button, .widget-text, .widget-password,
    .widget-dropdown, .widget-label, .widget-html {
        transition: all 0.3s ease-in-out;
    }

    /* Improved Button Hover Effects */
    .widget-button:hover {
        transform: translateY(-2px);
        box-shadow: 0 4px 8px rgba(0,0,0,0.1);
    }

    /* Remove outline focus on widgets */
    .widget-box:focus {
        outline: none !important;
    }

    /* Comprehensive scrollbar hiding */
    ::-webkit-scrollbar {
        width: 0px !important;
        height: 0px !important;
        background: transparent !important;
        display: none !important;
    }

    /* Hide scrollbars everywhere - more aggressive approach */
    * {
        -ms-overflow-style: none !important;  /* IE and Edge */
        scrollbar-width: none !important;  /* Firefox */
    }

    /* Additional scrollbar hiding for specific containers */
    .widget-box, .widget-vbox, .widget-hbox,
    .jupyter-widgets, .widget-container,
    .p-Widget, .p-Panel, .lm-Widget, .lm-Panel {
        overflow: hidden !important;
        scrollbar-width: none !important;
    }

    /* Force all Jupyter output areas to hide scrollbars */
    .jp-OutputArea, .jp-OutputArea-output, .jp-OutputArea-child {
        overflow: hidden !important;
        max-height: none !important;
    }

    /* Force the main output widget to expand as needed without scrollbars */
    .output_scroll {
        height: auto !important;
        max-height: none !important;
        overflow-y: hidden !important;
    }

    /* Fade animation for page transitions */
    @keyframes fadeIn {
        from { opacity: 0; transform: translateY(10px); }
        to { opacity: 1; transform: translateY(0); }
    }

    .fade-in {
        animation: fadeIn 0.4s ease-in-out;
    }

    /* Ensure proper sizing for input elements */
    .widget-text, .widget-textarea, .widget-password {
        box-sizing: border-box !important;
        width: 100% !important;
    }
    </style>
    """))

    # Navigation history
    nav_history = []

    def show_status(message, status='info'):
        colors = {'info': '#ff9900', 'success': '#00aa00', 'error': '#cc0000', 'warning': '#ff6600'}
        icons = {'info': 'ℹ️', 'success': '✅', 'error': '❌', 'warning': '⚠️'}
        with output:
            clear_output()
            if message:
                display(HTML(f"""
                <div style='padding:10px; border-radius:8px; background-color:{colors[status]}15;
                     border-left:4px solid {colors[status]}; margin:15px auto; max-width:500px; text-align:center;'>
                    <span style='color:{colors[status]}; font-weight:bold'>{icons[status]} {message}</span>
                </div>
                """))

    def show_loading(message="Loading..."):
        with output:
            clear_output()
            display(HTML(f"""
            <div style='text-align:center; padding:15px;'>
                <div style='font-weight:bold; margin-bottom:15px;'>{message}</div>
                <div style='display:inline-block; width:40px; height:40px; border:5px solid #f3f3f3;
                     border-top:5px solid #3498db; border-radius:50%; animation:spin 1s linear infinite;'></div>
                <style>@keyframes spin {{0% {{ transform: rotate(0deg); }} 100% {{ transform: rotate(360deg); }}}}</style>
            </div>
            """))

    def switch_to_page(page, save_history=True):
        if save_history and session['current_page'] != page:
            nav_history.append(session['current_page'])
        session['current_page'] = page
        show_status("")

        # Add fade transition between pages
        display(HTML("<div class='fade-in'>"))

        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

        display(HTML("</div>"))

    def go_back():
        if nav_history:
            previous_page = nav_history.pop()
            switch_to_page(previous_page, save_history=False)
        else:
            switch_to_page('start', save_history=False)

    def show_start_page():
        clear_output()
        start_label = widgets.HTML(f"<h1 style='{main_title_style}'>🎵 Music Recommender</h1>")
        welcome_text = widgets.HTML("<p style='margin-bottom:20px; text-align:center;'>Welcome to your personal music recommendation system.</p>")
        signin_button = widgets.Button(description='Sign In', button_style='primary', layout=button_layout)
        signup_button = widgets.Button(description='Sign Up', layout=button_layout)

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            welcome_text,
            widgets.HBox([signin_button, signup_button], layout=widgets.Layout(justify_content='center')),
            output
        ], layout=container_layout))

    def show_signin_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🔑 Sign In</h1>")

        # Consistent form field styling
        form_style = {'description_width': '80px'}
        form_layout = widgets.Layout(width='300px', margin='0 auto')

        si_username = widgets.Text(description='Username', style=form_style)
        si_password = widgets.Password(description='Password', style=form_style)
        si_button = widgets.Button(description='Sign In', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Back', layout=button_layout)

        def sign_in_action(b):
            show_loading("Signing in...")

            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: go_back())

        buttons_layout = widgets.Layout(justify_content='center', margin='15px 0')

        # Create a simplified container without nested layouts
        signin_form = widgets.VBox([
            si_username,
            si_password
        ], layout=form_layout)

        buttons = widgets.HBox([
            si_button,
            back_button
        ], layout=buttons_layout)

        # Use minimal nesting to avoid scroll issues
        main_container = widgets.VBox([
            label,
            signin_form,
            buttons,
            output
        ], layout=container_layout)

        # Direct display without further nesting
        display(main_container)

    def show_signup_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🔐 Sign Up</h1>")

        # Create form fields with consistent styling
        form_style = {'description_width': '100px'}
        form_layout = widgets.Layout(width='300px', margin='0 auto')

        su_username = widgets.Text(description='Username', style=form_style)
        su_password = widgets.Password(description='Password', style=form_style)
        su_firstname = widgets.Text(description='First Name', style=form_style)
        su_lastname = widgets.Text(description='Last Name', style=form_style)
        su_age = widgets.IntText(description='Age', style=form_style)
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender', style=form_style)

        selected_artists = []

        artist_header = widgets.HTML(f"<h2 style='{subtitle_style}'>🎵 Select Favorite Artists</h2>")
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(description='Add', layout=widgets.Layout(width='80px'))

        # Use fixed height with visible overflow only for the artist box
        artist_box = widgets.VBox(layout=widgets.Layout(
            width='400px',
            margin='10px auto',
            # No max-height to prevent scroll issues
        ))

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(
                    description='Remove',
                    button_style='danger',
                    layout=widgets.Layout(width='80px')
                )
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn], layout=widgets.Layout(justify_content='space-between', width='100%')))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)

        su_button = widgets.Button(description='Create Account', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Back', layout=button_layout)

        def sign_up_action(b):
            show_loading("Creating account...")

            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            if len(su_password.value) < 4:
                show_status("Password should be at least 4 characters long.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: go_back())

        artist_input_container = widgets.HBox([artist_input, add_button],
                                             layout=widgets.Layout(width='400px', margin='0 auto', justify_content='space-between'))

        buttons_layout = widgets.Layout(justify_content='center', margin='15px 0')

        # Group form fields to reduce nesting
        user_form = widgets.VBox([
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender
        ], layout=form_layout)

        buttons = widgets.HBox([
            su_button, back_button
        ], layout=buttons_layout)

        # Simplified container structure
        main_container = widgets.VBox([
            label,
            user_form,
            artist_header,
            artist_input_container,
            artist_box,
            buttons,
            output
        ], layout=container_layout)

        display(main_container)

    def show_edit_favorites_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🎨 Edit Favorite Artists</h1>")
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        form_style = {'description_width': '100px'}
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(description='Add', layout=widgets.Layout(width='80px'))

        # Remove fixed height and scroll settings
        artist_box = widgets.VBox(layout=widgets.Layout(
            width='400px',
            margin='10px auto'
        ))

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(
                    description='Remove',
                    button_style='danger',
                    layout=widgets.Layout(width='80px')
                )
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn], layout=widgets.Layout(justify_content='space-between', width='100%')))
            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)
        refresh_artist_table()

        confirm_button = widgets.Button(description='Save Changes', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Cancel', layout=button_layout)

        def save_favorites(b):
            show_loading("Saving your preferences...")

            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists)
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(session['user_id'], selected_artists)
            show_status("Your favorite artists have been updated successfully!", 'success')
            switch_to_page('session')

        confirm_button.on_click(save_favorites)
        back_button.on_click(lambda b: switch_to_page('session'))

        artist_input_container = widgets.HBox([artist_input, add_button],
                                             layout=widgets.Layout(width='400px', margin='0 auto', justify_content='space-between'))

        artist_header = widgets.HTML(f"<p style='{subtitle_style}'>Your Current Favorite Artists:</p>")

        buttons_layout = widgets.Layout(justify_content='center', margin='15px 0')
        buttons = widgets.HBox([confirm_button, back_button], layout=buttons_layout)

        # Simplified structure
        main_container = widgets.VBox([
            label,
            artist_input_container,
            artist_header,
            artist_box,
            buttons,
            output
        ], layout=container_layout)

        display(main_container)

    def show_session_page():
        clear_output()
        profile_label = widgets.HTML(f"<h1 style='{main_title_style}'>🎵 Music Recommender</h1>")
        user_info = widgets.HTML(f"<p style='text-align:center; margin-bottom:15px;'><b>👤 Signed in as:</b> {session['username']}</p>")

        # Improved mode selector with better spacing
        mode_selector = widgets.ToggleButtons(
            options=[
                ('Private Mode', 'none'),
                ('Log My Activity', 'log'),
                ('Recommend Me Music', 'recommend')
            ],
            value=session['mode'],
            description='Mode:',
            button_style='info',
            tooltips=[
                'Private Mode: No logging or recommendations',
                'Log Mode: Track your listening activity',
                'Recommend Mode: Get personalized music recommendations'
            ],
            layout=widgets.Layout(width='auto', margin='0 auto')
        )

        # Create a mapping for mode labels
        mode_labels = {
            'none': 'Private Mode',
            'log': 'Log My Activity',
            'recommend': 'Recommend Me Music'
        }

        mode_description = widgets.HTML(
            f"""<div style='margin:15px auto; padding:12px; background:#f8f8f8; border-radius:8px; width:500px; text-align:center;'>
                <p id='mode-desc' style='margin:0;'>Select a mode to continue.</p>
            </div>"""
        )

        # Action buttons with consistent styling and better layout
        play_button = widgets.Button(
            description='Play Random Song',
            icon='play',
            button_style='success',
            layout=button_layout
        )
        sad_button = widgets.Button(
            description='I Feel Sad 😢',
            button_style='info',
            layout=button_layout
        )
        happy_button = widgets.Button(
            description='I Feel Happy 😊',
            button_style='info',
            layout=button_layout
        )
        fav_edit_button = widgets.Button(
            description='Edit Favorites',
            icon='edit',
            layout=button_layout
        )
        exit_button = widgets.Button(
            description='Log Out',
            icon='sign-out',
            button_style='danger',
            layout=button_layout
        )

        def change_mode(change):
            session['mode'] = change['new']

            # Update mode description based on selection
            mode_descriptions = {
                'none': "Private Mode: Your activity won't be logged or used for recommendations.",
                'log': "Log Mode: Your listening activity will be tracked to improve recommendations.",
                'recommend': "Recommend Mode: Tell us how you're feeling and get personalized recommendations!"
            }
            mode_description.value = f"""<div style='margin:15px auto; padding:12px; background:#f8f8f8; border-radius:8px; width:500px; text-align:center;'>
                <p id='mode-desc' style='margin:0;'>{mode_descriptions[session['mode']]}</p>
            </div>"""

            show_status(f"Mode changed to: {mode_labels[session['mode']]}", 'info')

        def play_song(b):
            show_status("")
            show_loading("Finding a song...")

            if session['mode'] == 'none':
                show_status("Privacy mode → not logging song play.", 'info')
                return

            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)

            show_status(f"Now playing: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")

            if session['mode'] != 'recommend':
                show_status("Not in recommend mode — please switch to 'Recommend Me Music' mode first.", 'warning')
                return

            show_loading("Finding songs to lift your mood...")
            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("No songs found to recommend. Try adding more favorite artists.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]

            # Create a nicer song selection widget
            song_header = widgets.HTML("<h3 style='margin-top:15px; text-align:center;'>🎵 Songs to cheer you up:</h3>")
            song_options = widgets.RadioButtons(
                options=options,
                layout=widgets.Layout(width='auto', margin='10px auto')
            )
            confirm_button = widgets.Button(
                description='Play Selected Song',
                button_style='success',
                layout=widgets.Layout(margin='10px auto', display='flex', justify_content='center')
            )

            def on_confirm_click(_):
                if not song_options.value:
                    show_status("Please select a song first", 'warning')
                    return

                show_loading("Playing your selection...")

                selected_label = song_options.value
                selected_row = recommended[recommended['artist'] + ' - ' + recommended['song'] == selected_label].iloc[0]
                mood_before = 'sad'
                mood_after = emotion_service.detect()
                logger.log(
                    session['user_id'],
                    selected_row['song_id'],
                    mood_before,
                    mood_after,
                    is_recommended=1
                )
                show_status(f"Now playing: 🎵 {selected_row['artist']} - {selected_row['song']}", 'success')

            confirm_button.on_click(on_confirm_click)

            with music_recommender_output:
                clear_output()
                display(widgets.VBox([
                    song_header,
                    song_options,
                    confirm_button
                ], layout=widgets.Layout(align_items='center', width='500px', margin='0 auto')))

        def feel_happy(b):
            show_status("")
            show_status("Thanks for sharing! We're glad you're feeling happy today. 😊", 'success')

        def exit_session(b):
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        fav_edit_button.on_click(lambda b: switch_to_page('edit_favorites'))
        exit_button.on_click(exit_session)

        # Initialize mode description
        change_mode({'new': session['mode']})

        # Action buttons container with centered layout
        action_buttons = widgets.HBox(
            [play_button, sad_button, happy_button],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )

        # Profile buttons container with centered layout
        profile_buttons = widgets.HBox(
            [fav_edit_button, exit_button],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )

        action_title = widgets.HTML(f"<h2 style='{subtitle_style}'>Actions:</h2>")
        profile_title = widgets.HTML(f"<h2 style='{subtitle_style}'>Profile Settings:</h2>")
        mode_title = widgets.HTML(f"<h2 style='{subtitle_style}'>Select Your Experience Mode:</h2>")

        display(widgets.VBox([
            profile_label,
            user_info,
            mode_title,
            mode_selector,
            mode_description,
            action_title,
            action_buttons,
            profile_title,
            profile_buttons,
            output
        ], layout=container_layout))

    switch_to_page('start')
    print("✅ Enhanced music recommender interface loaded successfully!")

In [ ]:
start_music_recommender()

In [ ]:
start_music_recommender()

DEBUG: Starting enhanced multi-page recommender with improved UI/UX
✅ Health-based Dummy EmotionService initialized


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################
def start_music_recommender():
    print("DEBUG: Starting enhanced multi-page recommender with improved UI/UX")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    user_manager = UserManager(users_csv)
    emotion_service = EmotionService()
    recommender = RecommendationService(songs_csv, history_csv, users_csv)
    logger = LoggingService(history_csv)

    artist_list = sorted(pd.read_csv(songs_csv)['artist'].dropna().unique().tolist())

    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    # Define common styles with improved readability and spacing
    main_title_style = "color:#3366cc; font-size:28px; font-weight:bold; margin:25px 0 20px; text-align:center;"
    subtitle_style = "color:#555; font-size:20px; font-weight:bold; margin:20px 0 15px; text-align:center; border-bottom: 1px solid #eee; padding-bottom: 8px;"
    section_style = "margin:25px 0; background:#f9f9f9; border-radius:12px; padding:20px; box-shadow:0 2px 8px rgba(0,0,0,0.05);"

    # Improved button styles with more visible transitions
    button_layout = widgets.Layout(
        margin="10px",
        width="auto",
        min_width="140px"
    )

    # Wider container layout with better spacing
    container_layout = widgets.Layout(
        display="flex",
        flex_flow="column",
        align_items="center",
        width="750px",
        max_width="95%",
        margin="0 auto",
        padding="15px 20px 30px",
        border="none"
    )

    # Define enhanced CSS for better UI appearance
    display(HTML("""
    <style>
    /* Smooth transitions for all elements */
    .widget-button, .widget-text, .widget-password,
    .widget-dropdown, .widget-label, .widget-html {
        transition: all 0.3s ease-in-out;
    }

    /* Better Button Styling */
    .widget-button {
        border-radius: 6px !important;
        font-weight: 600 !important;
        box-shadow: 0 1px 3px rgba(0,0,0,0.12) !important;
    }

    /* Improved Button Hover Effects */
    .widget-button:hover {
        transform: translateY(-2px);
        box-shadow: 0 4px 8px rgba(0,0,0,0.15) !important;
    }

    /* Active button state */
    .widget-button:active {
        transform: translateY(1px);
        box-shadow: 0 2px 4px rgba(0,0,0,0.1) !important;
    }

    /* Remove outline focus on widgets */
    .widget-box:focus {
        outline: none !important;
    }

    /* Better form elements styling */
    .widget-text, .widget-password, .widget-dropdown {
        border-radius: 4px !important;
        border: 1px solid #ddd !important;
        padding: 6px 8px !important;
    }

    .widget-text:focus, .widget-password:focus {
        border-color: #3366cc !important;
        box-shadow: 0 0 0 2px rgba(51,102,204,0.2) !important;
    }

    /* Better toggle buttons */
    .widget-toggle-buttons {
        border-radius: 6px !important;
        overflow: hidden !important;
    }

    /* Fade animation for page transitions */
    @keyframes fadeIn {
        from { opacity: 0; transform: translateY(15px); }
        to { opacity: 1; transform: translateY(0); }
    }

    .fade-in {
        animation: fadeIn 0.5s ease-in-out;
    }

    /* Section containers */
    .app-section {
        background: #f9f9f9;
        border-radius: 12px;
        padding: 20px;
        margin: 15px 0;
        width: 100%;
        box-shadow: 0 2px 6px rgba(0,0,0,0.05);
        border: 1px solid #eee;
    }

    /* Mode selector styling */
    #mode-selector .widget-toggle-button {
        min-width: 150px !important;
        border-radius: 4px !important;
        margin: 0 5px !important;
        font-weight: 600 !important;
    }

    /* Status message styling */
    .status-message {
        padding: 12px;
        border-radius: 8px;
        margin: 15px auto;
        text-align: center;
        max-width: 550px;
        border-left: 4px solid;
        font-weight: 500;
    }
    </style>
    """))

    # Navigation history
    nav_history = []

    def show_status(message, status='info'):
        colors = {'info': '#3498db', 'success': '#2ecc71', 'error': '#e74c3c', 'warning': '#f39c12'}
        icons = {'info': 'ℹ️', 'success': '✅', 'error': '❌', 'warning': '⚠️'}
        with output:
            clear_output()
            if message:
                display(HTML(f"""
                <div class="status-message" style='background-color:{colors[status]}15; border-left-color:{colors[status]};'>
                    <span style='color:{colors[status]};'>{icons[status]} {message}</span>
                </div>
                """))

    def show_loading(message="Loading..."):
        with output:
            clear_output()
            display(HTML(f"""
            <div style='text-align:center; padding:20px;'>
                <div style='font-weight:bold; margin-bottom:15px;'>{message}</div>
                <div style='display:inline-block; width:40px; height:40px; border:5px solid #f3f3f3;
                     border-top:5px solid #3498db; border-radius:50%; animation:spin 1s linear infinite;'></div>
                <style>@keyframes spin {{0% {{ transform: rotate(0deg); }} 100% {{ transform: rotate(360deg); }}}}</style>
            </div>
            """))

    def switch_to_page(page, save_history=True):
        if save_history and session['current_page'] != page:
            nav_history.append(session['current_page'])
        session['current_page'] = page
        show_status("")

        # Add fade transition between pages
        display(HTML("<div class='fade-in'>"))

        if page == 'start':
            show_start_page()
        elif page == 'signin':
            show_signin_page()
        elif page == 'signup':
            show_signup_page()
        elif page == 'session':
            show_session_page()
        elif page == 'edit_favorites':
            show_edit_favorites_page()
        else:
            with output:
                clear_output()
                print(f"❌ Unknown page: {page}")

        display(HTML("</div>"))

    def go_back():
        if nav_history:
            previous_page = nav_history.pop()
            switch_to_page(previous_page, save_history=False)
        else:
            switch_to_page('start', save_history=False)

    def show_start_page():
        clear_output()
        start_label = widgets.HTML(f"<h1 style='{main_title_style}'>🎵 Music Recommender</h1>")
        welcome_text = widgets.HTML("<p style='margin-bottom:25px; text-align:center; font-size:16px;'>Welcome to your personal music recommendation system.</p>")

        # Add some visual interest with a decorative element
        decoration = widgets.HTML("""
        <div style="text-align:center; margin:20px 0 30px;">
            <span style="font-size:40px; color:#3366cc;">♪ ♫ ♬</span>
        </div>
        """)

        signin_button = widgets.Button(description='Sign In', button_style='primary', layout=button_layout)
        signup_button = widgets.Button(description='Sign Up', layout=button_layout)

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        display(widgets.VBox([
            start_label,
            welcome_text,
            decoration,
            widgets.HBox([signin_button, signup_button], layout=widgets.Layout(justify_content='center')),
            output
        ], layout=container_layout))

    def show_signin_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🔑 Sign In</h1>")

        # Consistent form field styling
        form_style = {'description_width': '100px'}
        form_layout = widgets.Layout(width='350px', margin='15px auto')

        si_username = widgets.Text(description='Username', style=form_style)
        si_password = widgets.Password(description='Password', style=form_style)
        si_button = widgets.Button(description='Sign In', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Back', layout=button_layout)

        def sign_in_action(b):
            show_loading("Signing in...")

            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username', 'password'])
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: go_back())

        buttons_layout = widgets.Layout(justify_content='center', margin='20px 0')

        # Add a decorative section around the form
        form_container = widgets.HTML(f"""
        <div class="app-section" style="{section_style}">
            <h3 style="text-align:center; margin-top:0; color:#555;">Enter Your Credentials</h3>
        </div>
        """)

        # Create a simplified container without nested layouts
        signin_form = widgets.VBox([
            si_username,
            si_password
        ], layout=form_layout)

        buttons = widgets.HBox([
            si_button,
            back_button
        ], layout=buttons_layout)

        # Use minimal nesting to avoid scroll issues
        main_container = widgets.VBox([
            label,
            form_container,
            signin_form,
            buttons,
            output
        ], layout=container_layout)

        # Direct display without further nesting
        display(main_container)

    def show_signup_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🔐 Sign Up</h1>")

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import os
import importlib
import sys
##################################################################################
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton')
import recommendation_service
importlib.reload(recommendation_service)
from recommendation_service import RecommendationService

import emotion_service
importlib.reload(emotion_service)
from emotion_service import EmotionService

import user_manager
importlib.reload(user_manager)
from user_manager import UserManager

import logging_service
importlib.reload(logging_service)
from logging_service import LoggingService
##################################################################################
# Define data directory - assuming data_dir is defined elsewhere or you need to define it
# For example: data_dir = "/content/drive/MyDrive/Colab Notebooks/project/Music/data/"
# If data_dir is not defined, this will cause a NameError
try:
    data_dir
except NameError:
    # Define a default or handle the case where data_dir is not set
    data_dir = "/content/data/" # Example default path

users_csv=f"{data_dir}users.csv"
songs_csv=f"{data_dir}songs.csv"
history_csv=f"{data_dir}listening_history.csv"
##################################################################################
def start_music_recommender():
    print("DEBUG: Starting enhanced multi-page recommender with improved UI/UX")

    global music_recommender_output
    try:
        music_recommender_output.close()
    except:
        pass
    music_recommender_output = widgets.Output()

    # Instantiate services - assuming UserManager needs paths in its constructor
    user_manager_instance = UserManager(users_csv)
    emotion_service_instance = EmotionService()
    recommender_instance = RecommendationService(songs_csv, history_csv, users_csv)
    logger_instance = LoggingService(history_csv)

    # Check if songs_csv exists before reading
    if os.path.exists(songs_csv):
        songs_df_temp = pd.read_csv(songs_csv)
        artist_list = sorted(songs_df_temp['artist'].dropna().unique().tolist())
    else:
        artist_list = []
        print(f"Warning: {songs_csv} not found. Artist list will be empty.")


    session = {
        'user_id': None,
        'username': '',
        'mode': 'none',
        'current_page': 'start'
    }

    output = music_recommender_output

    # Define common styles with improved readability and spacing
    main_title_style = "color:#3366cc; font-size:28px; font-weight:bold; margin:25px 0 20px; text-align:center;"
    subtitle_style = "color:#555; font-size:20px; font-weight:bold; margin:20px 0 15px; text-align:center; border-bottom: 1px solid #eee; padding-bottom: 8px;"
    section_style = "margin:25px 0; background:#f9f9f9; border-radius:12px; padding:20px; box-shadow:0 2px 8px rgba(0,0,0,0.05);"

    # Improved button styles with more visible transitions
    button_layout = widgets.Layout(
        margin="10px",
        width="auto",
        min_width="140px"
    )

    # Wider container layout with better spacing
    container_layout = widgets.Layout(
        display="flex",
        flex_flow="column",
        align_items="center",
        width="750px",
        max_width="95%",
        margin="0 auto",
        padding="15px 20px 30px",
        border="none"
    )

    # Define enhanced CSS for better UI appearance
    display(HTML("""
    <style>
    /* Smooth transitions for all elements */
    .widget-button, .widget-text, .widget-password,
    .widget-dropdown, .widget-label, .widget-html {
        transition: all 0.3s ease-in-out;
    }

    /* Better Button Styling */
    .widget-button {
        border-radius: 6px !important;
        font-weight: 600 !important;
        box-shadow: 0 1px 3px rgba(0,0,0,0.12) !important;
    }

    /* Improved Button Hover Effects */
    .widget-button:hover {
        transform: translateY(-2px);
        box-shadow: 0 4px 8px rgba(0,0,0,0.15) !important;
    }

    /* Active button state */
    .widget-button:active {
        transform: translateY(1px);
        box-shadow: 0 2px 4px rgba(0,0,0,0.1) !important;
    }

    /* Remove outline focus on widgets */
    .widget-box:focus {
        outline: none !important;
    }

    /* Better form elements styling */
    .widget-text, .widget-password, .widget-dropdown {
        border-radius: 4px !important;
        border: 1px solid #ddd !important;
        padding: 6px 8px !important;
    }

    .widget-text:focus, .widget-password:focus {
        border-color: #3366cc !important;
        box-shadow: 0 0 0 2px rgba(51,102,204,0.2) !important;
    }

    /* Better toggle buttons */
    .widget-toggle-buttons {
        border-radius: 6px !important;
        overflow: hidden !important;
    }

    /* Fade animation for page transitions */
    @keyframes fadeIn {
        from { opacity: 0; transform: translateY(15px); }
        to { opacity: 1; transform: translateY(0); }
    }

    .fade-in {
        animation: fadeIn 0.5s ease-in-out;
    }

    /* Section containers */
    .app-section {
        background: #f9f9f9;
        border-radius: 12px;
        padding: 20px;
        margin: 15px 0;
        width: 100%;
        box-shadow: 0 2px 6px rgba(0,0,0,0.05);
        border: 1px solid #eee;
    }

    /* Mode selector styling */
    #mode-selector .widget-toggle-button {
        min-width: 150px !important;
        border-radius: 4px !important;
        margin: 0 5px !important;
        font-weight: 600 !important;
    }

    /* Status message styling */
    .status-message {
        padding: 12px;
        border-radius: 8px;
        margin: 15px auto;
        text-align: center;
        max-width: 550px;
        border-left: 4px solid;
        font-weight: 500;
    }
    </style>
    """))

    # Navigation history
    nav_history = []

    def show_status(message, status='info'):
        colors = {'info': '#3498db', 'success': '#2ecc71', 'error': '#e74c3c', 'warning': '#f39c12'}
        icons = {'info': 'ℹ️', 'success': '✅', 'error': '❌', 'warning': '⚠️'}
        with output:
            clear_output()
            if message:
                display(HTML(f"""
                <div class="status-message" style='background-color:{colors[status]}15; border-left-color:{colors[status]};'>
                    <span style='color:{colors[status]};'>{icons[status]} {message}</span>
                </div>
                """))

    def show_loading(message="Loading..."):
        with output:
            clear_output()
            display(HTML(f"""
            <div style='text-align:center; padding:20px;'>
                <div style='font-weight:bold; margin-bottom:15px;'>{message}</div>
                <div style='display:inline-block; width:40px; height:40px; border:5px solid #f3f3f3;
                     border-top:5px solid #3498db; border-radius:50%; animation:spin 1s linear infinite;'></div>
                <style>@keyframes spin {{0% {{ transform: rotate(0deg); }} 100% {{ transform: rotate(360deg); }}}}</style>
            </div>
            """))

    def switch_to_page(page, save_history=True):
        if save_history and session['current_page'] != page:
            nav_history.append(session['current_page'])
        session['current_page'] = page
        show_status("")

        # Add fade transition between pages
        # This needs to wrap the content *after* clear_output
        # display(HTML("<div class='fade-in'>")) # This won't work correctly with clear_output

        with output:
            clear_output()
            # Now display the content wrapped in the fade-in div
            if page == 'start':
                content = show_start_page()
            elif page == 'signin':
                content = show_signin_page()
            elif page == 'signup':
                content = show_signup_page()
            elif page == 'session':
                content = show_session_page()
            elif page == 'edit_favorites':
                content = show_edit_favorites_page()
            else:
                content = widgets.VBox([widgets.HTML(f"❌ Unknown page: {page}")])

            display(widgets.VBox([widgets.HTML("<div class='fade-in'>"), content, widgets.HTML("</div>")], layout=widgets.Layout(width='100%', border='none')))


    def go_back():
        if nav_history:
            previous_page = nav_history.pop()
            switch_to_page(previous_page, save_history=False)
        else:
            switch_to_page('start', save_history=False)

    def show_start_page():
        start_label = widgets.HTML(f"<h1 style='{main_title_style}'>🎵 Music Recommender</h1>")
        welcome_text = widgets.HTML("<p style='margin-bottom:25px; text-align:center; font-size:16px;'>Welcome to your personal music recommendation system.</p>")

        # Add some visual interest with a decorative element
        decoration = widgets.HTML("""
        <div style="text-align:center; margin:20px 0 30px;">
            <span style="font-size:40px; color:#3366cc;">♪ ♫ ♬</span>
        </div>
        """)

        signin_button = widgets.Button(description='Sign In', button_style='primary', layout=button_layout)
        signup_button = widgets.Button(description='Sign Up', layout=button_layout)

        signin_button.on_click(lambda b: switch_to_page('signin'))
        signup_button.on_click(lambda b: switch_to_page('signup'))

        # Return the VBox instead of displaying directly
        return widgets.VBox([
            start_label,
            welcome_text,
            decoration,
            widgets.HBox([signin_button, signup_button], layout=widgets.Layout(justify_content='center')),
            output # Keep output here for status messages
        ], layout=container_layout)

    def show_signin_page():
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🔑 Sign In</h1>")

        # Consistent form field styling
        form_style = {'description_width': '100px'}
        form_layout = widgets.Layout(width='350px', margin='15px auto')

        si_username = widgets.Text(description='Username', style=form_style)
        si_password = widgets.Password(description='Password', style=form_style)
        si_button = widgets.Button(description='Sign In', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Back', layout=button_layout)

        def sign_in_action(b):
            show_loading("Signing in...")

            if si_username.value.strip() == "" or si_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            # Check if users_csv exists
            if not os.path.exists(users_csv):
                show_status("User database not found. Please sign up first.", 'error')
                return

            users_df = pd.read_csv(users_csv)
            user_row = users_df[users_df['username'] == si_username.value]
            if user_row.empty:
                show_status("Username not found.", 'error')
            elif user_row.iloc[0]['password'] != si_password.value:
                show_status("Incorrect password.", 'error')
            else:
                session['user_id'] = user_row.iloc[0]['user_id']
                session['username'] = si_username.value
                show_status("")
                switch_to_page('session')

        si_button.on_click(sign_in_action)
        back_button.on_click(lambda b: go_back())

        buttons_layout = widgets.Layout(justify_content='center', margin='20px 0')

        # Add a decorative section around the form
        form_container = widgets.HTML(f"""
        <div class="app-section" style="{section_style}">
            <h3 style="text-align:center; margin-top:0; color:#555;">Enter Your Credentials</h3>
        </div>
        """)

        # Create a simplified container without nested layouts
        signin_form = widgets.VBox([
            si_username,
            si_password
        ], layout=form_layout)

        buttons = widgets.HBox([
            si_button,
            back_button
        ], layout=buttons_layout)

        # Use minimal nesting to avoid scroll issues
        main_container = widgets.VBox([
            label,
            form_container,
            signin_form,
            buttons,
            output # Keep output here
        ], layout=container_layout)

        # Return the main container
        return main_container

    def show_signup_page():
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🔐 Sign Up</h1>")

        # Create form fields with consistent styling
        form_style = {'description_width': '100px'}
        form_layout = widgets.Layout(width='350px', margin='15px auto')

        su_username = widgets.Text(description='Username', style=form_style)
        su_password = widgets.Password(description='Password', style=form_style)
        su_firstname = widgets.Text(description='First Name', style=form_style)
        su_lastname = widgets.Text(description='Last Name', style=form_style)
        su_age = widgets.IntText(description='Age', style=form_style)
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender', style=form_style)

        selected_artists = []

        artist_header = widgets.HTML(f"<h2 style='{subtitle_style}'>🎵 Select Favorite Artists</h2>")
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(description='Add', layout=widgets.Layout(width='80px'))

        # Artist box with better styling
        artist_box = widgets.VBox(layout=widgets.Layout(
            width='450px',
            margin='15px auto',
            background='#f9f9f9',
            padding='15px',
            border='1px solid #eee',
            border_radius='8px'
        ))

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(
                    description='Remove',
                    button_style='danger',
                    layout=widgets.Layout(width='90px')
                )
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn], layout=widgets.Layout(justify_content='space-between', width='100%')))

            if not rows:
                rows = [widgets.HTML("<p style='text-align:center; color:#999; font-style:italic;'>No artists selected yet</p>")]

            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)

        su_button = widgets.Button(description='Create Account', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Back', layout=button_layout)

        def sign_up_action(b):
            show_loading("Creating account...")

            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            if len(su_password.value) < 4:
                show_status("Password should be at least 4 characters long.", 'error')
                return

            # Ensure users_csv exists, create if not
            if not os.path.exists(users_csv):
                users_df = pd.DataFrame(columns=['user_id', 'username', 'password', 'first_name', 'last_name', 'age', 'gender', 'favorite_artists', 'recommended_artists'])
                users_df.to_csv(users_csv, index=False)
            else:
                users_df = pd.read_csv(users_csv)


            if su_username.value in users_df['username'].values:
                show_status("Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager_instance.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: go_back())

        artist_input_container = widgets.HBox([artist_input, add_button],
                                             layout=widgets.Layout(width='450px', margin='15px auto', justify_content='space-between'))

        buttons_layout = widgets.Layout(justify_content='center', margin='25px 0')

        # Group form fields with better visual separation
        personal_details = widgets.HTML(f"""
        <div class="app-section" style="{section_style}">
            <h3 style="text-align:center; margin-top:0; color:#555;">Personal Information</h3>
        </div>
        """)

        user_form = widgets.VBox([
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender
        ], layout=form_layout)

        buttons = widgets.HBox([
            su_button, back_button
        ], layout=buttons_layout)

        # Simplified container structure with visual separation
        main_container = widgets.VBox([
            label,
            personal_details,
            user_form,
            artist_header,
            artist_input_container,
            artist_box,
            buttons,
            output # Keep output here
        ], layout=container_layout)

        refresh_artist_table()
        # Return the main container
        return main_container

    def show_edit_favorites_page():
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🎨 Edit Favorite Artists</h1>")

        # Check if users_csv exists and session['user_id'] is set
        if not os.path.exists(users_csv) or session['user_id'] is None:
            show_status("Cannot edit favorites: User not signed in or user database not found.", 'error')
            return widgets.VBox([label, widgets.Button(description="Back to Session", on_click=lambda b: switch_to_page('session'))], layout=container_layout)


        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        if user_row.empty:
            show_status("Error: User data not found.", 'error')
            return widgets.VBox([label, widgets.Button(description="Back to Session", on_click=lambda b: switch_to_page('session'))], layout=container_layout)

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        form_style = {'description_width': '100px'}
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(description='Add', layout=widgets.Layout(width='80px'))

        # Better styled artist box
        artist_box = widgets.VBox(layout=widgets.Layout(
            width='450px',
            margin='15px auto',
            background='#f9f9f9',
            padding='15px',
            border='1px solid #eee',
            border_radius='8px'
        ))

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(
                    description='Remove',
                    button_style='danger',
                    layout=widgets.Layout(width='90px')
                )
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn], layout=widgets.Layout(justify_content='space-between', width='100%')))

            if not rows:
                rows = [widgets.HTML("<p style='text-align:center; color:#999; font-style:italic;'>No artists selected yet</p>")]

            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)
        refresh_artist_table()

        confirm_button = widgets.Button(description='Save Changes', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Cancel', layout=button_layout)

        def save_favorites(b):
            show_loading("Saving your preferences...")

            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists)
            users_df.to_csv(users_csv, index=False)
            user_manager_instance.update_recommended_artists(session['user_id'], selected_artists)
            show_status("Your favorite artists have been updated successfully!", 'success')
            switch_to_page('session')

        confirm_button.on_click(save_favorites)
        back_button.on_click(lambda b: switch_to_page('session'))

        artist_input_container = widgets.HBox([artist_input, add_button],
                                             layout=widgets.Layout(width='450px', margin='15px auto', justify_content='space-between'))

        artist_header = widgets.HTML(f"<h2 style='{subtitle_style}'>Your Current Favorite Artists:</h2>")

        instructions = widgets.HTML("""
        <div style="text-align:center; margin-bottom:20px; color:#666;">
            Search for artists to add to your favorites, or remove existing ones below.
        </div>
        """)

        buttons_layout = widgets.Layout(justify_content='center', margin='25px 0')
        buttons = widgets.HBox([confirm_button, back_button], layout=buttons_layout)

        # Simplified structure with better visual organization
        main_container = widgets.VBox([
            label,
            instructions,
            artist_input_container,
            artist_header,
            artist_box,
            buttons,
            output # Keep output here
        ], layout=container_layout)

        # Return the main container
        return main_container

    def show_session_page():
        # Profile header with user info
        profile_label = widgets.HTML(f"<h1 style='{main_title_style}'>🎵 Music Recommender</h1>")
        user_info = widgets.HTML(f"""
            <div style="text-align:center; margin-bottom:20px;">
                <span style="background:#f0f7ff; padding:8px 15px; border-radius:20px; font-weight:600;">
                    👤 Signed in as: <span style="color:#3366cc;">{session['username']}</span>
                </span>
            </div>
        """)

        # Mode section with clear visual separation
        mode_section = widgets.HTML(f"""
        <div class="app-section" style="{section_style}">
            <h3 style="text-align:center; margin-top:0; color:#555;">Select Your Experience Mode</h3>
        </div>
        """)

        # Improved mode selector with better spacing and ID for CSS targeting
        mode_selector = widgets.ToggleButtons(
            options=[
                ('Private Mode', 'none'),
                ('Log My Activity', 'log'),
                ('Recommend Me Music', 'recommend')
            ],
            value=session['mode'],
            description='',
            button_style='info',
            tooltips=[
                'Private Mode: No logging or recommendations',
                'Log Mode: Track your listening activity',
                'Recommend Mode: Get personalized music recommendations'
            ],
            layout=widgets.Layout(width='100%', margin='15px auto')
        )

        # Add an ID for CSS targeting (Note: This HTML widget itself doesn't add the ID to the ToggleButtons)
        # You would typically modify the layout of the ToggleButtons or wrap it
        # For simplicity here, we'll just include the HTML for documentation
        mode_selector_html = widgets.HTML('<div id="mode-selector"></div>')


        # Create a mapping for mode labels
        mode_labels = {
            'none': 'Private Mode',
            'log': 'Log My Activity',
            'recommend': 'Recommend Me Music'
        }

        mode_description = widgets.HTML(
            f"""<div style='margin:15px auto; padding:15px; background:#f0f7ff; border-radius:8px; width:100%; text-align:center;'>
                <p id='mode-desc' style='margin:0; font-size:15px;'>Select a mode to continue.</p>
            </div>"""
        )

        # Actions section with better visual grouping
        actions_section = widgets.HTML(f"""
        <div class="app-section" style="{section_style}">
            <h3 style="text-align:center; margin-top:0; color:#555;">Music Player Controls</h3>
        </div>
        """)

        # Action buttons with consistent styling and better layout
        play_button = widgets.Button(
            description='Play Random Song',
            icon='play',
            button_style='success',
            layout=button_layout
        )

        # Mood section with better visual grouping
        mood_section = widgets.HTML(f"""
        <div class="app-section" style="{section_style}">
            <h3 style="text-align:center; margin-top:0; color:#555;">How Are You Feeling Today?</h3>
        </div>
        """)

        sad_button = widgets.Button(
            description='I Feel Sad 😢',
            button_style='info',
            layout=button_layout
        )
        happy_button = widgets.Button(
            description='I Feel Happy 😊',
            button_style='info',
            layout=button_layout
        )

        # Profile section with better visual grouping
        profile_section = widgets.HTML(f"""
        <div class="app-section" style="{section_style}">
            <h3 style="text-align:center; margin-top:0; color:#555;">Account Settings</h3>
        </div>
        """)

        fav_edit_button = widgets.Button(
            description='Edit Favorites',
            icon='edit',
            layout=button_layout
        )
        exit_button = widgets.Button(
            description='Log Out',
            icon='sign-out',
            button_style='danger',
            layout=button_layout
        )

        def change_mode(change):
            session['mode'] = change['new']

            # Update mode description based on selection
            mode_descriptions = {
                'none': "Private Mode: Your activity won't be logged or used for recommendations.",
                'log': "Log Mode: Your listening activity will be tracked to improve recommendations.",
                'recommend': "Recommend Mode: Tell us how you're feeling and get personalized recommendations!"
            }
            mode_description.value = f"""<div style='margin:15px auto; padding:15px; background:#f0f7ff; border-radius:8px; width:100%; text-align:center;'>
                <p id='mode-desc' style='margin:0; font-size:15px;'>{mode_descriptions[session['mode']]}</p>
            </div>"""

            show_status(f"Mode changed to: {mode_labels[session['mode']]}", 'info')

        def play_song(b):
            show_status("")
            show_loading("Finding a song...")

            # Check if songs_csv exists
            if not os.path.exists(songs_csv):
                show_status("Error: Song database not found.", 'error')
                return

            if session['mode'] == 'none':
                show_status("Privacy mode → not logging song play.", 'info')
                return

            songs_df = pd.read_csv(songs_csv)
            if songs_df.empty:
                 show_status("Error: No songs available in the database.", 'error')
                 return

            song = songs_df.sample(1).iloc[0]
            # Assuming emotion_service_instance.detect() works
            mood_before = emotion_service_instance.detect()
            mood_after = emotion_service_instance.detect()
            # Assuming logger_instance.log works
            logger_instance.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)

            show_status(f"Now playing: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")

            if session['mode'] != 'recommend':
                show_status("Not in recommend mode — please switch to 'Recommend Me Music' mode first.", 'warning')
                return

            show_loading("Finding songs to lift your mood...")
            # Assuming recommender_instance.recommend_songs works
            recommended = recommender_instance.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("No songs found to recommend. Try adding more favorite artists.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]

            # Create a nicer song selection widget
            song_header = widgets.HTML("<h3 style='margin-top:15px; text-align:center;'>🎵 Songs to cheer you up:</h3>")
            song_options = widgets.RadioButtons(
                options=options,
                layout=widgets.Layout(width='auto', margin='15px auto')
            )
            confirm_button = widgets.Button(
                description='Play Selected Song',
                button_style='success',
                layout=widgets.Layout(margin='15px auto', display='flex', justify_content='center')
            )

            def on_confirm_click(_):
                if not song_options.value:
                    show_status("Please select a song first", 'warning')
                    return

                show_loading("Playing your selection...")
                selected_label = song_options.value
                selected_artist, selected_song = selected_label.split(' - ', 1)

                # Check if songs_csv exists
                if not os.path.exists(songs_csv):
                    show_status("Error: Song database not found.", 'error')
                    with output: clear_output()
                    return

                songs_df = pd.read_csv(songs_csv)
                song_row = songs_df[(songs_df['artist'] == selected_artist) & (songs_df['song'] == selected_song)]

                if not song_row.empty:
                    song_id = song_row.iloc[0]['song_id']
                    # Assuming emotion_service_instance.detect() works
                    mood_before = emotion_service_instance.detect()
                    mood_after = emotion_service_instance.detect()
                    # Assuming logger_instance.log works
                    logger_instance.log(session['user_id'], song_id, mood_before, mood_after, is_recommended=1)
                    show_status(f"Now playing: {selected_label}", 'success')
                else:
                    show_status("Song not found in database", 'error')

                # Remove the song selection widgets after playing
                with output:
                    clear_output()

            confirm_button.on_click(on_confirm_click)
            with output:
                clear_output()
                display(song_header)
                display(song_options)
                display(confirm_button)

        def feel_happy(b):
            show_status("")
            if session['mode'] != 'recommend':
                show_status("Not in recommend mode — please switch to 'Recommend Me Music' mode first.", 'warning')
                return

            show_loading("Finding songs to match your happy mood...")
            # Assuming recommender_instance.recommend_songs works
            recommended = recommender_instance.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("No songs found to recommend. Try adding more favorite artists.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]

            # Create a nicer song selection widget
            song_header = widgets.HTML("<h3 style='margin-top:15px; text-align:center;'>🎵 Songs to keep your mood up:</h3>")
            song_options = widgets.RadioButtons(
                options=options,
                layout=widgets.Layout(width='auto', margin='15px auto')
                )
            confirm_button = widgets.Button(
                description='Play Selected Song',
                button_style='success',
                layout=widgets.Layout(margin='15px auto', display='flex', justify_content='center')
                )

            def on_confirm_click(_):
                if not song_options.value:
                    show_status("Please select a song first", 'warning')
                    return

                show_loading("Playing your selection...")
                selected_label = song_options.value
                selected_artist, selected_song = selected_label.split(' - ', 1)

                # Check if songs_csv exists
                if not os.path.exists(songs_csv):
                    show_status("Error: Song database not found.", 'error')
                    with output: clear_output()
                    return

                songs_df = pd.read_csv(songs_csv)
                song_row = songs_df[(songs_df['artist'] == selected_artist) & (songs_df['song'] == selected_song)]

                if not song_row.empty:
                    song_id = song_row.iloc[0]['song_id']
                    # Assuming emotion_service_instance.detect() works
                    mood_before = emotion_service_instance.detect()
                    mood_after = emotion_service_instance.detect()
                    # Assuming logger_instance.log works
                    logger_instance.log(session['user_id'], song_id, mood_before, mood_after, is_recommended=1)
                    show_status(f"Now playing: {selected_label}", 'success')
                else:
                    show_status("Song not found in database", 'error')

                with output:
                    clear_output()

            confirm_button.on_click(on_confirm_click)

            with output:
                clear_output()
                display(song_header)
                display(song_options)
                display(confirm_button)

        def exit_app(b):
            show_loading("Logging out...")
            session['user_id'] = None
            session['username'] = ''
            session['mode'] = 'none'
            show_status("")
            switch_to_page('start')

        # Connect buttons to functions
        mode_selector.observe(change_mode, names='value')
        play_button.on_click(play_song)
        sad_button.on_click(feel_sad)
        happy_button.on_click(feel_happy)
        fav_edit_button.on_click(lambda b: switch_to_page('edit_favorites'))
        exit_button.on_click(exit_app)

        # Create the layout with organized button groups
        mood_buttons = widgets.HBox([sad_button, happy_button], layout=widgets.Layout(justify_content='center', margin='15px 0'))
        profile_buttons = widgets.HBox([fav_edit_button, exit_button], layout=widgets.Layout(justify_content='center', margin='15px 0'))

        # Display everything with proper spacing and organization
        main_container = widgets.VBox([
            profile_label,
            user_info,
            mode_section,
            mode_selector,
            mode_description,
            actions_section,
            widgets.HBox([play_button], layout=widgets.Layout(justify_content='center', margin='15px 0')),
            mood_section,
            mood_buttons,
            profile_section,
            profile_buttons,
            output # Keep output here
        ], layout=container_layout)

        # Return the main container
        return main_container


    # Start the application
    # Need to call switch_to_page outside the function definition to actually start
    switch_to_page('start')

# Call the main function to start the application
start_music_recommender()
        su_password = widgets.Password(description='Password', style=form_style)
        su_firstname = widgets.Text(description='First Name', style=form_style)
        su_lastname = widgets.Text(description='Last Name', style=form_style)
        su_age = widgets.IntText(description='Age', style=form_style)
        su_gender = widgets.Dropdown(options=['M', 'F'], description='Gender', style=form_style)

        selected_artists = []

        artist_header = widgets.HTML(f"<h2 style='{subtitle_style}'>🎵 Select Favorite Artists</h2>")
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(description='Add', layout=widgets.Layout(width='80px'))

        # Artist box with better styling
        artist_box = widgets.VBox(layout=widgets.Layout(
            width='450px',
            margin:'15px auto',
            background:'#f9f9f9',
            padding:'15px',
            border:'1px solid #eee',
            border_radius:'8px'
        ))

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(
                    description='Remove',
                    button_style='danger',
                    layout=widgets.Layout(width:'90px')
                )
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn], layout=widgets.Layout(justify_content:'space-between', width:'100%')))

            if not rows:
                rows = [widgets.HTML("<p style='text-align:center; color:#999; font-style:italic;'>No artists selected yet</p>")]

            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)

        su_button = widgets.Button(description='Create Account', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Back', layout=button_layout)

        def sign_up_action(b):
            show_loading("Creating account...")

            if su_username.value.strip() == "" or su_password.value.strip() == "":
                show_status("Please enter both username and password.", 'error')
                return

            if len(su_password.value) < 4:
                show_status("Password should be at least 4 characters long.", 'error')
                return

            users_df = pd.read_csv(users_csv) if os.path.exists(users_csv) else pd.DataFrame(columns=['user_id', 'username'])
            if su_username.value in users_df['username'].values:
                show_status("Username already taken.", 'error')
            else:
                new_user_id = users_df['user_id'].max() + 1 if not users_df.empty else 1
                new_user = pd.DataFrame([{
                    'user_id': new_user_id,
                    'username': su_username.value,
                    'password': su_password.value,
                    'first_name': su_firstname.value,
                    'last_name': su_lastname.value,
                    'age': su_age.value,
                    'gender': su_gender.value,
                    'favorite_artists': ', '.join(selected_artists),
                    'recommended_artists': ''
                }])
                users_df = pd.concat([users_df, new_user], ignore_index=True)
                users_df.to_csv(users_csv, index=False)
                user_manager.update_recommended_artists(new_user_id, selected_artists)
                session['user_id'] = new_user_id
                session['username'] = su_username.value
                show_status("")
                switch_to_page('session')

        su_button.on_click(sign_up_action)
        back_button.on_click(lambda b: go_back())

        artist_input_container = widgets.HBox([artist_input, add_button],
                                             layout=widgets.Layout(width='450px', margin:'15px auto', justify_content:'space-between'))

        buttons_layout = widgets.Layout(justify_content='center', margin:'25px 0')

        # Group form fields with better visual separation
        personal_details = widgets.HTML(f"""
        <div class="app-section" style="{section_style}">
            <h3 style="text-align:center; margin-top:0; color:#555;">Personal Information</h3>
        </div>
        """)

        user_form = widgets.VBox([
            su_username, su_password, su_firstname, su_lastname, su_age, su_gender
        ], layout=form_layout)

        buttons = widgets.HBox([
            su_button, back_button
        ], layout=buttons_layout)

        # Simplified container structure with visual separation
        main_container = widgets.VBox([
            label,
            personal_details,
            user_form,
            artist_header,
            artist_input_container,
            artist_box,
            buttons,
            output
        ], layout=container_layout)

        refresh_artist_table()
        display(main_container)

    def show_edit_favorites_page():
        clear_output()
        label = widgets.HTML(f"<h1 style='{main_title_style}'>🎨 Edit Favorite Artists</h1>")
        users_df = pd.read_csv(users_csv)
        user_row = users_df[users_df['user_id'] == session['user_id']]

        fav_str = user_row.iloc[0].get('favorite_artists', '')
        if pd.isna(fav_str):
            fav_str = ''
        selected_artists = [a.strip() for a in fav_str.split(',') if a.strip()]

        form_style = {'description_width': '100px'}
        artist_input = widgets.Combobox(
            placeholder='Type to search artist...',
            options=artist_list,
            description='Artist:',
            ensure_option=True,
            style=form_style
        )
        add_button = widgets.Button(description='Add', layout=widgets.Layout(width:'80px'))

        # Better styled artist box
        artist_box = widgets.VBox(layout=widgets.Layout(
            width:'450px',
            margin:'15px auto',
            background:'#f9f9f9',
            padding:'15px',
            border:'1px solid #eee',
            border_radius:'8px'
        ))

        def refresh_artist_table():
            rows = []
            for a in selected_artists:
                label = widgets.Label(a)
                remove_btn = widgets.Button(
                    description='Remove',
                    button_style='danger',
                    layout=widgets.Layout(width:'90px')
                )
                def make_remove_callback(artist):
                    return lambda b: (selected_artists.remove(artist), refresh_artist_table())
                remove_btn.on_click(make_remove_callback(a))
                rows.append(widgets.HBox([label, remove_btn], layout=widgets.Layout(justify_content:'space-between', width:'100%')))

            if not rows:
                rows = [widgets.HTML("<p style='text-align:center; color:#999; font-style:italic;'>No artists selected yet</p>")]

            artist_box.children = rows

        def on_add(b):
            val = artist_input.value.strip()
            if val in artist_list and val not in selected_artists:
                selected_artists.append(val)
                refresh_artist_table()
                artist_input.value = ''
            elif val in selected_artists:
                show_status("This artist is already in your favorites", 'warning')
            elif val not in artist_list:
                show_status("Artist not found in our database", 'warning')

        add_button.on_click(on_add)
        refresh_artist_table()

        confirm_button = widgets.Button(description='Save Changes', button_style='primary', layout=button_layout)
        back_button = widgets.Button(description='Cancel', layout=button_layout)

        def save_favorites(b):
            show_loading("Saving your preferences...")

            users_df.loc[users_df['user_id'] == session['user_id'], 'favorite_artists'] = ', '.join(selected_artists)
            users_df.to_csv(users_csv, index=False)
            user_manager.update_recommended_artists(session['user_id'], selected_artists)
            show_status("Your favorite artists have been updated successfully!", 'success')
            switch_to_page('session')

        confirm_button.on_click(save_favorites)
        back_button.on_click(lambda b: switch_to_page('session'))

        artist_input_container = widgets.HBox([artist_input, add_button],
                                             layout=widgets.Layout(width:'450px', margin:'15px auto', justify_content:'space-between'))

        artist_header = widgets.HTML(f"<h2 style='{subtitle_style}'>Your Current Favorite Artists:</h2>")

        instructions = widgets.HTML("""
        <div style="text-align:center; margin-bottom:20px; color:#666;">
            Search for artists to add to your favorites, or remove existing ones below.
        </div>
        """)

        buttons_layout = widgets.Layout(justify_content:'center', margin:'25px 0')
        buttons = widgets.HBox([confirm_button, back_button], layout=buttons_layout)

        # Simplified structure with better visual organization
        main_container = widgets.VBox([
            label,
            instructions,
            artist_input_container,
            artist_header,
            artist_box,
            buttons,
            output
        ], layout=container_layout)

        display(main_container)

    def show_session_page():
        clear_output()

        # Profile header with user info
        profile_label = widgets.HTML(f"<h1 style='{main_title_style}'>🎵 Music Recommender</h1>")
        user_info = widgets.HTML(f"""
            <div style="text-align:center; margin-bottom:20px;">
                <span style="background:#f0f7ff; padding:8px 15px; border-radius:20px; font-weight:600;">
                    👤 Signed in as: <span style="color:#3366cc;">{session['username']}</span>
                </span>
            </div>
        """)

        # Mode section with clear visual separation
        mode_section = widgets.HTML(f"""
        <div class="app-section" style="{section_style}">
            <h3 style="text-align:center; margin-top:0; color:#555;">Select Your Experience Mode</h3>
        </div>
        """)

        # Improved mode selector with better spacing and ID for CSS targeting
        mode_selector = widgets.ToggleButtons(
            options=[
                ('Private Mode', 'none'),
                ('Log My Activity', 'log'),
                ('Recommend Me Music', 'recommend')
            ],
            value=session['mode'],
            description='',
            button_style='info',
            tooltips=[
                'Private Mode: No logging or recommendations',
                'Log Mode: Track your listening activity',
                'Recommend Mode: Get personalized music recommendations'
            ],
            layout=widgets.Layout(width:'100%', margin:'15px auto')
        )

        # Add an ID for CSS targeting
        mode_selector_html = widgets.HTML('<div id="mode-selector"></div>')

        # Create a mapping for mode labels
        mode_labels = {
            'none': 'Private Mode',
            'log': 'Log My Activity',
            'recommend': 'Recommend Me Music'
        }

        mode_description = widgets.HTML(
            f"""<div style='margin:15px auto; padding:15px; background:#f0f7ff; border-radius:8px; width:100%; text-align:center;'>
                <p id='mode-desc' style='margin:0; font-size:15px;'>Select a mode to continue.</p>
            </div>"""
        )

        # Actions section with better visual grouping
        actions_section = widgets.HTML(f"""
        <div class="app-section" style="{section_style}">
            <h3 style="text-align:center; margin-top:0; color:#555;">Music Player Controls</h3>
        </div>
        """)

        # Action buttons with consistent styling and better layout
        play_button = widgets.Button(
            description='Play Random Song',
            icon='play',
            button_style='success',
            layout=button_layout
        )

        # Mood section with better visual grouping
        mood_section = widgets.HTML(f"""
        <div class="app-section" style="{section_style}">
            <h3 style="text-align:center; margin-top:0; color:#555;">How Are You Feeling Today?</h3>
        </div>
        """)

        sad_button = widgets.Button(
            description='I Feel Sad 😢',
            button_style='info',
            layout=button_layout
        )
        happy_button = widgets.Button(
            description='I Feel Happy 😊',
            button_style='info',
            layout=button_layout
        )

        # Profile section with better visual grouping
        profile_section = widgets.HTML(f"""
        <div class="app-section" style="{section_style}">
            <h3 style="text-align:center; margin-top:0; color:#555;">Account Settings</h3>
        </div>
        """)

        fav_edit_button = widgets.Button(
            description='Edit Favorites',
            icon='edit',
            layout=button_layout
        )
        exit_button = widgets.Button(
            description='Log Out',
            icon='sign-out',
            button_style='danger',
            layout=button_layout
        )

        def change_mode(change):
            session['mode'] = change['new']

            # Update mode description based on selection
            mode_descriptions = {
                'none': "Private Mode: Your activity won't be logged or used for recommendations.",
                'log': "Log Mode: Your listening activity will be tracked to improve recommendations.",
                'recommend': "Recommend Mode: Tell us how you're feeling and get personalized recommendations!"
            }
            mode_description.value = f"""<div style='margin:15px auto; padding:15px; background:#f0f7ff; border-radius:8px; width:100%; text-align:center;'>
                <p id='mode-desc' style='margin:0; font-size:15px;'>{mode_descriptions[session['mode']]}</p>
            </div>"""

            show_status(f"Mode changed to: {mode_labels[session['mode']]}", 'info')

        def play_song(b):
            show_status("")
            show_loading("Finding a song...")

            if session['mode'] == 'none':
                show_status("Privacy mode → not logging song play.", 'info')
                return

            songs_df = pd.read_csv(songs_csv)
            song = songs_df.sample(1).iloc[0]
            mood_before = emotion_service.detect()
            mood_after = emotion_service.detect()
            logger.log(session['user_id'], song['song_id'], mood_before, mood_after, is_recommended=0)

            show_status(f"Now playing: {song['artist']} - {song['song']}", 'success')

        def feel_sad(b):
            show_status("")

            if session['mode'] != 'recommend':
                show_status("Not in recommend mode — please switch to 'Recommend Me Music' mode first.", 'warning')
                return

            show_loading("Finding songs to lift your mood...")
            recommended = recommender.recommend_songs(session['user_id'])

            if recommended.empty:
                show_status("No songs found to recommend. Try adding more favorite artists.", 'error')
                return

            options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]

            # Create a nicer song selection widget
            song_header = widgets.HTML("<h3 style='margin-top:15px; text-align:center;'>🎵 Songs to cheer you up:</h3>")
            song_options = widgets.RadioButtons(
                options=options,
                layout=widgets.Layout(width:'auto', margin:'15px auto')
            )
            confirm_button = widgets.Button(
                description='Play Selected Song',
                button_style='success',
                layout=widgets.Layout(margin:'15px auto', display:'flex', justify_content:'center')
            )

            def on_confirm_click(_):
              if not song_options.value:
                show_status("Please select a song first", 'warning')
                return

              show_loading("Playing your selection...")
              selected_label = song_options.value
              selected_artist, selected_song = selected_label.split(' - ', 1)
              songs_df = pd.read_csv(songs_csv)
              song_row = songs_df[(songs_df['artist'] == selected_artist) & (songs_df['song'] == selected_song)]

              if not song_row.empty:
                song_id = song_row.iloc[0]['song_id']
                mood_before = emotion_service.detect()
                mood_after = emotion_service.detect()
                logger.log(session['user_id'], song_id, mood_before, mood_after, is_recommended=1)
                show_status(f"Now playing: {selected_label}", 'success')
              else:
                show_status("Song not found in database", 'error')

              # Remove the song selection widgets after playing
              with output:
                clear_output()
            confirm_button.on_click(on_confirm_click)
            with output:
              clear_output()
              display(song_header)
              display(song_options)
              display(confirm_button)

            def feel_happy(b):
              show_status("")
              if session['mode'] != 'recommend':
                show_status("Not in recommend mode — please switch to 'Recommend Me Music' mode first.", 'warning')
                return

              show_loading("Finding songs to match your happy mood...")
              recommended = recommender.recommend_songs(session['user_id'])

              if recommended.empty:
                show_status("No songs found to recommend. Try adding more favorite artists.", 'error')
                return

              options = [f"{row['artist']} - {row['song']}" for _, row in recommended.iterrows()]

              # Create a nicer song selection widget
              song_header = widgets.HTML("<h3 style='margin-top:15px; text-align:center;'>🎵 Songs to keep your mood up:</h3>")
              song_options = widgets.RadioButtons(
                  options=options,
                  layout=widgets.Layout(width='auto', margin='15px auto')
                  )
              confirm_button = widgets.Button(
                  description='Play Selected Song',
                  button_style='success',
                  layout=widgets.Layout(margin='15px auto', display='flex', justify_content='center')
                  )

              confirm_button.on_click(on_confirm_click)

              with output:
                clear_output()
                display(song_header)
                display(song_options)
                display(confirm_button)

            def exit_app(b):
              show_loading("Logging out...")
              session['user_id'] = None
              session['username'] = ''
              session['mode'] = 'none'
              show_status("")
              switch_to_page('start')

# Connect buttons to functions
          mode_selector.observe(change_mode, names='value')
          play_button.on_click(play_song)
          sad_button.on_click(feel_sad)
          happy_button.on_click(feel_happy)
          fav_edit_button.on_click(lambda b: switch_to_page('edit_favorites'))
          exit_button.on_click(exit_app)

# Create the layout with organized button groups
        mood_buttons = widgets.HBox([sad_button, happy_button], layout=widgets.Layout(justify_content='center', margin='15px 0'))
        profile_buttons = widgets.HBox([fav_edit_button, exit_button], layout=widgets.Layout(justify_content='center', margin='15px 0'))

# Display everything with proper spacing and organization
        main_container = widgets.VBox([
            profile_label,
            user_info,
            mode_section,
            mode_selector,
            mode_description,
            actions_section,
            widgets.HBox([play_button], layout=widgets.Layout(justify_content='center', margin='15px 0')),
            mood_section,
            mood_buttons,
            profile_section,
            profile_buttons,
            output
        ], layout=container_layout)

        display(main_container)

# Start the application
        switch_to_page('start')

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 1414)

In [ ]:
launch_admin_ui()

# Run

# Clearing history

In [ ]:
import pandas as pd

def clear_user_and_history_data(users_csv_path, history_csv_path):
    # Clear users.csv
    users_df = pd.read_csv(users_csv_path)
    empty_users = pd.DataFrame(columns=users_df.columns)
    empty_users.to_csv(users_csv_path, index=False)
    print(f"✅ Cleared all data in {users_csv_path} (kept columns).")

    # Clear listening_history.csv
    history_df = pd.read_csv(history_csv_path)
    empty_history = pd.DataFrame(columns=history_df.columns)
    empty_history.to_csv(history_csv_path, index=False)
    print(f"✅ Cleared all data in {history_csv_path} (kept columns).")
clear_user_and_history_data(f"{data_dir}users.csv",f"{data_dir}listening_history.csv")

✅ Cleared all data in /content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/data/users.csv (kept columns).
✅ Cleared all data in /content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/data/listening_history.csv (kept columns).


In [ ]:
'''import pandas as pd

# Load the full history
history_csv = '/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/data/listening_history.csv'
df = pd.read_csv(history_csv)

# Keep only the original columns
original_columns = ['user_id', 'song_id', 'mood_before', 'mood_after','start_date','end_date','is_recommended']
df_cleaned = df[original_columns]

# Overwrite the original file
df_cleaned.to_csv(history_csv, index=False)

print("✅ History CSV cleaned back to original structure.")'''


'import pandas as pd\n\n# Load the full history\nhistory_csv = \'/content/drive/MyDrive/Colab Notebooks/project/Music/skeleton/data/listening_history.csv\'\ndf = pd.read_csv(history_csv)\n\n# Keep only the original columns\noriginal_columns = [\'user_id\', \'song_id\', \'mood_before\', \'mood_after\',\'start_date\',\'end_date\',\'is_recommended\']\ndf_cleaned = df[original_columns]\n\n# Overwrite the original file\ndf_cleaned.to_csv(history_csv, index=False)\n\nprint("✅ History CSV cleaned back to original structure.")'

# Admin

In [ ]:
def launch_admin_ui():
    from IPython.display import display, clear_output, HTML
    import pandas as pd
    import os
    import ipywidgets as widgets

    output = widgets.Output()
    users_table = widgets.Output()
    history_table = widgets.Output()

    def load_users(_=None):
        users_table.clear_output()
        with users_table:
            if os.path.exists(users_csv):
                df = pd.read_csv(users_csv)
                display(df)
            else:
                display(HTML("<i>No users.csv file found</i>"))

    def load_history(_=None):
        history_table.clear_output()
        with history_table:
            if os.path.exists(history_csv):
                df = pd.read_csv(history_csv)
                display(df)
            else:
                display(HTML("<i>No listening_history.csv file found</i>"))

    refresh_users = widgets.Button(description="🔄 Refresh Users")
    refresh_users.on_click(load_users)

    refresh_history = widgets.Button(description="🔄 Refresh History")
    refresh_history.on_click(load_history)

    title = widgets.HTML("<h3>🛠 Admin Panel: View Databases</h3>")

    load_users()
    load_history()

    display(widgets.VBox([
        title,
        widgets.HBox([refresh_users, refresh_history]),
        widgets.HTML("<h4>👥 Users Table:</h4>"),
        users_table,
        widgets.HTML("<h4>🎧 Listening History:</h4>"),
        history_table,
        output
    ]))


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
from datetime import datetime, timedelta
import os

def simulated_log_ui(songs_csv, history_csv):
    songs_df = pd.read_csv(songs_csv)

    user_id = widgets.IntText(description="User ID")
    artist = widgets.Text(description="Artist")
    song = widgets.Text(description="Song")
    mood_before = widgets.Dropdown(options=["happy", "sad", "relaxed", "anger"], description="Mood Before")
    mood_after = widgets.Dropdown(options=["happy", "sad", "relaxed", "anger"], description="Mood After")
    is_recommended = widgets.Checkbox(value=False, description="Recommended?")
    duration = widgets.IntText(description="Duration (s)", value=120)
    submit_btn = widgets.Button(description="➕ Add Entry")
    output = widgets.Output()

    def on_submit_clicked(b):
        with output:
            clear_output()
            match = songs_df[
                (songs_df['artist'].str.lower() == artist.value.strip().lower()) &
                (songs_df['song'].str.lower() == song.value.strip().lower())
            ]
            if match.empty:
                print("❌ No matching song found.")
                return

            song_id = match.iloc[0]['song_id']
            start_time = datetime.now()
            end_time = start_time + timedelta(seconds=duration.value)

            entry = {
                'user_id': user_id.value,
                'song_id': song_id,
                'mood_before': mood_before.value,
                'mood_after': mood_after.value,
                'start_date': start_time,
                'end_date': end_time,
                'is_recommended': int(is_recommended.value)
            }

            if os.path.exists(history_csv):
                hist_df = pd.read_csv(history_csv)
            else:
                hist_df = pd.DataFrame(columns=entry.keys())

            hist_df = pd.concat([hist_df, pd.DataFrame([entry])], ignore_index=True)
            hist_df.to_csv(history_csv, index=False)

            print(f"✅ Logged entry for song ID {song_id} with {duration.value} sec duration.")

    submit_btn.on_click(on_submit_clicked)

    display(widgets.VBox([
        user_id, artist, song,
        mood_before, mood_after,
        is_recommended, duration,
        submit_btn, output
    ]))


In [ ]:
simulated_log_ui(songs_csv, history_csv)

In [ ]:
'''import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler

# Load your songs.csv
df = pd.read_csv(data_dir+'songs.csv')

# Define numeric audio feature columns
ignore_cols = ['song_id', 'artist', 'song', 'title', 'genre', 'emotion']
feature_cols = [c for c in df.columns if c not in ignore_cols and df[c].dtype in [np.float64, np.int64]]

# Fit a scaler
scaler = StandardScaler()
scaler.fit(df[feature_cols].fillna(0))

# Save it
joblib.dump(scaler, 'scaler.joblib')
print("✅ Saved scaler.joblib")'''

'import pandas as pd\nimport numpy as np\nimport joblib\nfrom sklearn.preprocessing import StandardScaler\n\n# Load your songs.csv\ndf = pd.read_csv(data_dir+\'songs.csv\')\n\n# Define numeric audio feature columns\nignore_cols = [\'song_id\', \'artist\', \'song\', \'title\', \'genre\', \'emotion\']\nfeature_cols = [c for c in df.columns if c not in ignore_cols and df[c].dtype in [np.float64, np.int64]]\n\n# Fit a scaler\nscaler = StandardScaler()\nscaler.fit(df[feature_cols].fillna(0))\n\n# Save it\njoblib.dump(scaler, \'scaler.joblib\')\nprint("✅ Saved scaler.joblib")'